In [1]:
import flask
import time
from telegram.ext import Updater
import logging
from telegram.ext import CommandHandler
import math
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import numpy as np
import requests

from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [17]:
#获取历史币价 不同交易所，不同币种api获取的时间不同 要根据情况指定
def get_crypto_hisprice(name):
    
    #bitcoin
    date_start = '2013-04-28 08:00:00'
    date_end = '2021-08-19 08:00:00'
    
    #dogecoin
#     date_start = '2013-12-15 08:00:00'
#     date_end = '2021-08-17 08:00:00'

    output = cg.get_coin_market_chart_range_by_id(id=name, vs_currency='usd',from_timestamp=time.mktime(time.strptime(date_start, '%Y-%m-%d %H:%M:%S')),to_timestamp=time.mktime(time.strptime(date_end, '%Y-%m-%d %H:%M:%S')))
    return str(output['prices'])
# get_crypto_hisprice('bitcoin')

'[[1367107200000, 135.3], [1367193600000, 141.96], [1367280000000, 135.3], [1367366400000, 117.0], [1367452800000, 103.43], [1367539200000, 91.01], [1367625600000, 111.25], [1367712000000, 116.79], [1367798400000, 118.33], [1367884800000, 106.4], [1367971200000, 112.64], [1368057600000, 113.0], [1368144000000, 118.78], [1368230400000, 113.01], [1368316800000, 114.713], [1368403200000, 117.18], [1368489600000, 114.5], [1368576000000, 114.156], [1368662400000, 115.5], [1368748800000, 123.1], [1368835200000, 123.88], [1368921600000, 120.501], [1369008000000, 122.58], [1369094400000, 122.9], [1369180800000, 123.0], [1369267200000, 125.748], [1369353600000, 131.7], [1369440000000, 130.77], [1369526400000, 134.6], [1369612800000, 128.985], [1369699200000, 129.179], [1369785600000, 132.13], [1369872000000, 127.401], [1369958400000, 127.98], [1370044800000, 129.09], [1370131200000, 121.3], [1370217600000, 121.16], [1370390400000, 121.309], [1370476800000, 121.714], [1370563200000, 119.0], [137

In [3]:
#将元数据转成给前端页面展示的可读数据 包括了日期和价格
def get_rawdata_webshow():
    #通过api获取的源数据 get_crypto_hisprice
    rawdata = [[1367107200000, 135.3], [1367193600000, 141.96], [1367280000000, 135.3], [1367366400000, 117.0], [1367452800000, 103.43], [1367539200000, 91.01], [1367625600000, 111.25], [1367712000000, 116.79], [1367798400000, 118.33], [1367884800000, 106.4], [1367971200000, 112.64], [1368057600000, 113.0], [1368144000000, 118.78], [1368230400000, 113.01], [1368316800000, 114.713], [1368403200000, 117.18], [1368489600000, 114.5], [1368576000000, 114.156], [1368662400000, 115.5], [1368748800000, 123.1], [1368835200000, 123.88], [1368921600000, 120.501], [1369008000000, 122.58], [1369094400000, 122.9], [1369180800000, 123.0], [1369267200000, 125.748], [1369353600000, 131.7], [1369440000000, 130.77], [1369526400000, 134.6], [1369612800000, 128.985], [1369699200000, 129.179], [1369785600000, 132.13], [1369872000000, 127.401], [1369958400000, 127.98], [1370044800000, 129.09], [1370131200000, 121.3], [1370217600000, 121.16], [1370390400000, 121.309], [1370476800000, 121.714], [1370563200000, 119.0], [1370649600000, 110.09], [1370736000000, 108.401], [1370822400000, 99.7497], [1370908800000, 105.99], [1370995200000, 108.9], [1371081600000, 108.4], [1371168000000, 102.769], [1371254400000, 98.399], [1371340800000, 99.99], [1371427200000, 100.42], [1371513600000, 102.0], [1371600000000, 106.55], [1371686400000, 107.9], [1371772800000, 110.91], [1371859200000, 109.015], [1371945600000, 107.25], [1372032000000, 108.0], [1372118400000, 102.443], [1372204800000, 103.691], [1372291200000, 103.39], [1372377600000, 99.66], [1372464000000, 93.3311], [1372550400000, 95.3886], [1372636800000, 97.504], [1372723200000, 89.489], [1372809600000, 90.797], [1372896000000, 78.8], [1372982400000, 78.48], [1373068800000, 67.809], [1373155200000, 68.0831], [1373241600000, 77.502], [1373328000000, 76.3915], [1373414400000, 77.0], [1373500800000, 85.75], [1373587200000, 93.804], [1373673600000, 88.0622], [1373760000000, 95.58], [1373846400000, 94.0], [1373932800000, 97.301], [1374019200000, 98.9796], [1374105600000, 97.8], [1374192000000, 88.7869], [1374278400000, 91.572], [1374364800000, 89.28], [1374451200000, 90.3672], [1374537600000, 91.7024], [1374624000000, 94.5806], [1374710400000, 94.525], [1374796800000, 96.9499], [1374883200000, 94.13], [1374969600000, 94.005], [1375056000000, 98.5759], [1375142400000, 101.5], [1375228800000, 110.34], [1375315200000, 106.23], [1375401600000, 101.61], [1375488000000, 103.7], [1375574400000, 104.86], [1375660800000, 106.56], [1375747200000, 105.841], [1375833600000, 106.75], [1375920000000, 101.672], [1376006400000, 103.4], [1376092800000, 102.71], [1376179200000, 103.77], [1376265600000, 105.18], [1376352000000, 106.068], [1376438400000, 111.621], [1376524800000, 111.87], [1376611200000, 109.867], [1376697600000, 112.74], [1376784000000, 114.4], [1376870400000, 118.782], [1376956800000, 120.5], [1377043200000, 121.51], [1377129600000, 122.499], [1377216000000, 120.0], [1377302400000, 120.44], [1377388800000, 122.0], [1377475200000, 122.04], [1377561600000, 125.67], [1377648000000, 123.02], [1377734400000, 123.44], [1377820800000, 129.2], [1377907200000, 137.76], [1377993600000, 138.17], [1378080000000, 137.17], [1378166400000, 137.51], [1378252800000, 135.21], [1378339200000, 129.96], [1378425600000, 126.13], [1378512000000, 123.73], [1378598400000, 121.77], [1378684800000, 126.4], [1378771200000, 127.78], [1378857600000, 131.65], [1378944000000, 134.22], [1379030400000, 135.05], [1379116800000, 131.39], [1379203200000, 129.95], [1379289600000, 131.48], [1379376000000, 131.31], [1379462400000, 132.06], [1379548800000, 130.23], [1379635200000, 128.57], [1379721600000, 126.78], [1379808000000, 128.37], [1379894400000, 126.73], [1379980800000, 126.09], [1380067200000, 127.99], [1380153600000, 128.02], [1380240000000, 131.84], [1380326400000, 133.93], [1380412800000, 140.28], [1380499200000, 131.65], [1380585600000, 132.55], [1380672000000, 132.04], [1380758400000, 123.07], [1380844800000, 128.26], [1380931200000, 129.57], [1381017600000, 127.56], [1381104000000, 128.38], [1381190400000, 125.72], [1381276800000, 130.9], [1381363200000, 129.68], [1381449600000, 130.27], [1381536000000, 135.35], [1381622400000, 135.48], [1381708800000, 140.48], [1381795200000, 143.89], [1381881600000, 143.61], [1381968000000, 146.15], [1382054400000, 151.34], [1382140800000, 170.51], [1382227200000, 172.01], [1382313600000, 181.41], [1382400000000, 193.45], [1382486400000, 206.78], [1382572800000, 198.8], [1382659200000, 172.48], [1382745600000, 178.96], [1382832000000, 189.83], [1382918400000, 198.55], [1383004800000, 204.39], [1383091200000, 204.2], [1383177600000, 203.88], [1383264000000, 205.26], [1383350400000, 206.89], [1383436800000, 212.45], [1383523200000, 226.28], [1383609600000, 230.63], [1383696000000, 260.85], [1383782400000, 295.63], [1383868800000, 332.02], [1383955200000, 358.0], [1384041600000, 313.0], [1384128000000, 340.55], [1384214400000, 355.48], [1384300800000, 404.17], [1384387200000, 408.84], [1384473600000, 424.16], [1384560000000, 437.41], [1384646400000, 488.36], [1384732800000, 580.87], [1384819200000, 614.4], [1384905600000, 565.34], [1384992000000, 675.06], [1385078400000, 731.15], [1385164800000, 819.31], [1385251200000, 764.43], [1385337600000, 793.07], [1385424000000, 832.93], [1385510400000, 914.13], [1385596800000, 979.08], [1385683200000, 1101.83], [1385769600000, 1127.45], [1385856000000, 1033.39], [1385942400000, 974.03], [1386028800000, 1078.64], [1386115200000, 1121.48], [1386201600000, 989.04], [1386288000000, 1019.66], [1386374400000, 811.98], [1386460800000, 743.11], [1386547200000, 897.89], [1386633600000, 927.78], [1386720000000, 887.08], [1386806400000, 878.17], [1386892800000, 897.27], [1386979200000, 866.99], [1387065600000, 854.62], [1387152000000, 789.15], [1387238400000, 691.65], [1387324800000, 559.36], [1387411200000, 677.84], [1387497600000, 694.25], [1387584000000, 596.12], [1387670400000, 650.27], [1387756800000, 645.75], [1387843200000, 659.94], [1387929600000, 668.74], [1388016000000, 734.59], [1388102400000, 734.27], [1388188800000, 738.81], [1388275200000, 726.47], [1388361600000, 760.52], [1388448000000, 755.16], [1388534400000, 767.74], [1388620800000, 772.53], [1388707200000, 825.47], [1388793600000, 849.14], [1388880000000, 919.41], [1388966400000, 936.38], [1389052800000, 826.5], [1389139200000, 838.32], [1389225600000, 853.29], [1389312000000, 863.3], [1389398400000, 905.72], [1389484800000, 867.45], [1389571200000, 840.96], [1389657600000, 838.05], [1389744000000, 860.44], [1389830400000, 837.98], [1389916800000, 812.9], [1390003200000, 840.0], [1390089600000, 838.17], [1390176000000, 869.15], [1390262400000, 869.57], [1390348800000, 865.03], [1390435200000, 846.97], [1390521600000, 808.32], [1390608000000, 804.39], [1390694400000, 853.51], [1390780800000, 891.86], [1390867200000, 777.0], [1390953600000, 816.85], [1391040000000, 817.17], [1391126400000, 817.93], [1391212800000, 832.52], [1391299200000, 836.41], [1391385600000, 813.38], [1391472000000, 832.36], [1391558400000, 830.92], [1391644800000, 805.08], [1391731200000, 721.32], [1391817600000, 701.61], [1391904000000, 672.07], [1391990400000, 689.65], [1392076800000, 697.76], [1392163200000, 655.26], [1392249600000, 651.91], [1392336000000, 623.57], [1392422400000, 644.1], [1392508800000, 613.85], [1392595200000, 645.14], [1392681600000, 625.01], [1392768000000, 620.99], [1392854400000, 593.89], [1392940800000, 582.75], [1393027200000, 571.06], [1393113600000, 622.97], [1393200000000, 572.39], [1393286400000, 489.31], [1393372800000, 567.25], [1393459200000, 586.28], [1393545600000, 567.2], [1393632000000, 560.11], [1393718400000, 561.35], [1393804800000, 585.79], [1393891200000, 675.0], [1393977600000, 658.24], [1394064000000, 654.55], [1394150400000, 617.62], [1394236800000, 610.37], [1394323200000, 636.17], [1394409600000, 621.17], [1394496000000, 624.23], [1394582400000, 638.07], [1394668800000, 639.03], [1394755200000, 629.67], [1394841600000, 638.07], [1394928000000, 629.55], [1395014400000, 623.705], [1395100800000, 612.622], [1395187200000, 610.636], [1395273600000, 592.07], [1395360000000, 582.303], [1395446400000, 561.22], [1395532800000, 567.779], [1395619200000, 571.418], [1395705600000, 582.55], [1395792000000, 584.442], [1395878400000, 567.64], [1395964800000, 503.661], [1396051200000, 502.302], [1396137600000, 469.938], [1396224000000, 457.399], [1396310400000, 478.7163], [1396396800000, 437.515], [1396483200000, 447.0822], [1396569600000, 448.8792], [1396656000000, 464.8259], [1396742400000, 460.7028], [1396828800000, 457.792], [1396915200000, 450.4638], [1397001600000, 440.1983], [1397088000000, 360.8407], [1397174400000, 420.0563], [1397260800000, 420.66], [1397347200000, 414.9495], [1397433600000, 457.6338], [1397520000000, 520.1233], [1397606400000, 529.1625], [1397692800000, 494.4], [1397779200000, 478.2312], [1397865600000, 501.5515], [1397952000000, 497.3177], [1398038400000, 492.6733], [1398124800000, 484.8533], [1398211200000, 487.525], [1398297600000, 494.2733], [1398384000000, 458.8067], [1398470400000, 453.8], [1398556800000, 437.9216], [1398643200000, 436.92], [1398729600000, 443.27], [1398816000000, 444.9968], [1398902400000, 455.0913], [1398988800000, 443.4734], [1399075200000, 434.0333], [1399161600000, 433.9432], [1399248000000, 429.0761], [1399334400000, 426.7634], [1399420800000, 439.3], [1399507200000, 437.3633], [1399593600000, 447.4933], [1399680000000, 452.505], [1399766400000, 436.2833], [1399852800000, 439.19], [1399939200000, 437.305], [1400025600000, 444.2467], [1400112000000, 444.25], [1400198400000, 445.165], [1400284800000, 445.8533], [1400371200000, 444.2133], [1400457600000, 443.87], [1400544000000, 483.4167], [1400630400000, 488.7717], [1400716800000, 522.6083], [1400803200000, 524.475], [1400889600000, 523.1083], [1400976000000, 569.3767], [1401062400000, 579.6217], [1401148800000, 569.475], [1401235200000, 571.5533], [1401321600000, 565.26], [1401408000000, 621.1467], [1401494400000, 620.5], [1401580800000, 640.1617], [1401667200000, 656.41], [1401753600000, 667.3683], [1401840000000, 641.1833], [1401926400000, 658.1], [1402012800000, 648.63], [1402099200000, 651.7083], [1402185600000, 649.8983], [1402272000000, 643.905], [1402358400000, 649.0567], [1402444800000, 630.0367], [1402531200000, 567.175], [1402617600000, 594.6833], [1402704000000, 570.0033], [1402790400000, 583.41], [1402876800000, 592.52], [1402963200000, 607.1583], [1403049600000, 604.19], [1403136000000, 593.3883000000002], [1403222400000, 590.055], [1403308800000, 591.2567], [1403395200000, 595.0367], [1403481600000, 586.1482999999998], [1403568000000, 582.4149999999998], [1403654400000, 561.4716999999999], [1403740800000, 579.5867], [1403827200000, 598.2367], [1403913600000, 594.8717000000001], [1404000000000, 598.4883], [1404086400000, 637.7549999999999], [1404172800000, 643.9350000000001], [1404259200000, 648.1075000000002], [1404345600000, 640.2838000000003], [1404432000000, 627.1288], [1404518400000, 627.5812999999999], [1404604800000, 631.0025], [1404691200000, 619.4950000000002], [1404777600000, 620.0474999999996], [1404864000000, 620.16], [1404950400000, 615.1275000000002], [1405036800000, 630.8350000000002], [1405123200000, 633.6389951679741], [1405209600000, 627.4623228193354], [1405296000000, 618.2050265064788], [1405382400000, 618.9524999999999], [1405468800000, 614.1474999999997], [1405555200000, 622.8950000000001], [1405641600000, 627.0188000000003], [1405728000000, 626.1763000000001], [1405814400000, 620.5138], [1405900800000, 619.9088000000002], [1405987200000, 619.0062999999999], [1406073600000, 617.9425000000001], [1406160000000, 599.8088], [1406246400000, 600.0963000000002], [1406332800000, 593.7750000000001], [1406419200000, 590.3788], [1406505600000, 585.2787999999998], [1406592000000, 581.7138], [1406678400000, 560.7912999999999], [1406764800000, 583.1821057431899], [1406851200000, 593.8655273953669], [1406937600000, 588.2306991611412], [1407024000000, 585.5731940766995], [1407110400000, 586.3686859113967], [1407196800000, 582.4186267649761], [1407283200000, 583.3249999999999], [1407369600000, 587.2031590188054], [1407456000000, 590.9072702245192], [1407542400000, 588.1078559835756], [1407628800000, 588.5004825541756], [1407715200000, 573.4063], [1407801600000, 567.4023482208208], [1407888000000, 543.6674999999999], [1407974400000, 503.7087999999998], [1408060800000, 492.2844982828295], [1408147200000, 521.0463000000001], [1408233600000, 494.7813], [1408320000000, 459.8863000000001], [1408406400000, 484.1549999999999], [1408492800000, 514.1863000000001], [1408579200000, 515.605], [1408665600000, 515.8263000000001], [1408752000000, 496.5688000000001], [1408838400000, 508.1138], [1408924800000, 500.3863], [1409011200000, 510.6387999999999], [1409097600000, 510.4463], [1409184000000, 506.9150000000001], [1409270400000, 508.0963], [1409356800000, 501.5300000000001], [1409443200000, 480.0262999999999], [1409529600000, 475.8200000000002], [1409616000000, 474.6938], [1409702400000, 474.7263000000001], [1409788800000, 489.5475000000001], [1409875200000, 480.2425000000001], [1409961600000, 482.3588], [1410048000000, 480.6025], [1410134400000, 471.1250000000001], [1410220800000, 471.7538], [1410307200000, 477.1138], [1410393600000, 476.3775000000001], [1410480000000, 473.6874999999999], [1410566400000, 476.0013], [1410652800000, 474.795], [1410739200000, 473.0313], [1410825600000, 462.5925], [1410912000000, 454.6238000000001], [1410998400000, 423.565], [1411084800000, 393.2624999999999], [1411171200000, 408.5025], [1411257600000, 396.62], [1411344000000, 398.3113000000001], [1411430400000, 433.7512999999999], [1411516800000, 420.92], [1411603200000, 407.9775], [1411689600000, 402.88], [1411776000000, 398.0074999999999], [1411862400000, 375.9887999999999], [1411948800000, 373.0888], [1412035200000, 387.3512999999999], [1412121600000, 380.8949999999999], [1412208000000, 371.8413], [1412294400000, 356.1899999999999], [1412380800000, 327.8813], [1412467200000, 325.1099999999999], [1412553600000, 328.73], [1412640000000, 331.7238], [1412726400000, 353.3901000000001], [1412812800000, 358.395], [1412899200000, 357.5549999999999], [1412985600000, 359.515], [1413072000000, 374.1675000000001], [1413158400000, 387.0387999999999], [1413244800000, 398.2888], [1413331200000, 392.1900000000001], [1413417600000, 379.3912999999999], [1413504000000, 380.2925], [1413590400000, 389.1163], [1413676800000, 386.945], [1413763200000, 379.7699999999999], [1413849600000, 382.4137999999999], [1413936000000, 379.4425000000001], [1414022400000, 356.2000000000001], [1414108800000, 354.3175000000001], [1414195200000, 347.0304940564094], [1414281600000, 350.1888000000001], [1414368000000, 348.29], [1414454400000, 351.5638000000001], [1414540800000, 333.1474999999999], [1414627200000, 343.6674999999999], [1414713600000, 336.5879758057807], [1414800000000, 323.9763], [1414886400000, 323.9849999999999], [1414972800000, 323.995], [1415059200000, 328.0083], [1415145600000, 337.5933], [1415232000000, 347.8183], [1415318400000, 341.2317], [1415404800000, 344.1783], [1415491200000, 363.4241291015933], [1415577600000, 363.7649999999999], [1415664000000, 366.15], [1415750400000, 418.5783], [1415836800000, 414.7083], [1415923200000, 395.245], [1416009600000, 377.3267], [1416096000000, 387.6233], [1416182400000, 384.9], [1416268800000, 374.8683], [1416355200000, 376.7883], [1416441600000, 355.3888345541618], [1416528000000, 349.3882854323945], [1416614400000, 351.5500000000001], [1416700800000, 366.7817], [1416787200000, 377.7417], [1416873600000, 375.248], [1416960000000, 365.7660000000001], [1417046400000, 368.448], [1417132800000, 376.951], [1417219200000, 374.9649999999999], [1417305600000, 376.472], [1417392000000, 379.489], [1417478400000, 381.3768000000001], [1417564800000, 374.103], [1417651200000, 368.2282000000001], [1417737600000, 376.379], [1417824000000, 373.671], [1417910400000, 374.448], [1417996800000, 361.433], [1418083200000, 350.9280000000001], [1418169600000, 346.9798718363186], [1418256000000, 346.9089999999999], [1418342400000, 351.502], [1418428800000, 347.484], [1418515200000, 350.133], [1418601600000, 344.615], [1418688000000, 328.5340000000001], [1418774400000, 318.828], [1418860800000, 309.765], [1418947200000, 317.54], [1419033600000, 328.7740000000001], [1419120000000, 319.501], [1419206400000, 329.886], [1419292800000, 333.97], [1419379200000, 322.194], [1419465600000, 318.166], [1419552000000, 327.7169708710151], [1419638400000, 314.568], [1419724800000, 316.491], [1419811200000, 312.6329999999999], [1419897600000, 310.079], [1419984000000, 317.379], [1420070400000, 313.9920000000001], [1420156800000, 314.446], [1420243200000, 286.572], [1420329600000, 260.936], [1420416000000, 273.22], [1420502400000, 285.3738000000001], [1420588800000, 295.8725], [1420675200000, 284.4525], [1420761600000, 289.5738000000001], [1420848000000, 274.533], [1420934400000, 267.184], [1421020800000, 268.353667700943], [1421107200000, 221.436], [1421193600000, 172.1479], [1421280000000, 210.5559], [1421366400000, 205.816], [1421452800000, 199.4914], [1421539200000, 208.9892], [1421625600000, 214.545], [1421712000000, 210.5137], [1421798400000, 224.0451], [1421884800000, 232.0423000000001], [1421971200000, 232.898], [1422057600000, 248.096], [1422144000000, 253.8388999999999], [1422230400000, 271.2882000000001], [1422316800000, 259.2356], [1422489600000, 232.5329422304941], [1422576000000, 228.6354], [1422662400000, 218.0233], [1422748800000, 226.2697], [1422835200000, 240.0193], [1422921600000, 226.0119], [1423008000000, 225.3083], [1423094400000, 216.9136], [1423180800000, 222.4872], [1423267200000, 227.2050999999999], [1423353600000, 222.9061], [1423440000000, 220.0922], [1423526400000, 220.6493], [1423612800000, 219.0956], [1423699200000, 222.0401587875259], [1423785600000, 235.6774], [1423872000000, 259.0086000000001], [1423958400000, 232.9962], [1424044800000, 235.9199], [1424131200000, 242.4568], [1424217600000, 234.6238], [1424304000000, 240.2988], [1424390400000, 244.0821], [1424476800000, 244.1459], [1424563200000, 235.5529999999999], [1424649600000, 238.1319], [1424736000000, 238.7314], [1424822400000, 237.378], [1424908800000, 236.1711557723409], [1424995200000, 253.8991], [1425081600000, 253.5991], [1425168000000, 257.7075], [1425254400000, 269.9614999999999], [1425340800000, 280.6241999999999], [1425427200000, 273.0192999999999], [1425513600000, 274.8255], [1425600000000, 272.3412999999999], [1425686400000, 275.5846387340832], [1425772800000, 274.578], [1425859200000, 289.2957000000001], [1425945600000, 290.3057], [1426032000000, 295.961], [1426118400000, 293.9203], [1426204800000, 288.0364999999999], [1426291200000, 281.4887], [1426377600000, 285.2645], [1426464000000, 290.0955000000001], [1426550400000, 284.3491], [1426636800000, 255.6062], [1426723200000, 260.0037], [1426809600000, 260.9075], [1426896000000, 259.4899], [1426982400000, 267.7980999999999], [1427068800000, 266.0272], [1427155200000, 245.1679], [1427241600000, 246.2715], [1427328000000, 247.5595], [1427414400000, 247.1476], [1427500800000, 252.0497], [1427587200000, 241.9136999999999], [1427673600000, 246.5418], [1427760000000, 243.2692000000001], [1427846400000, 246.7755], [1427932800000, 252.341], [1428019200000, 253.6226999999999], [1428105600000, 253.3213], [1428192000000, 259.7117999999999], [1428278400000, 255.4233], [1428364800000, 253.2085], [1428451200000, 244.2709], [1428537600000, 243.2006548544864], [1428624000000, 234.5253], [1428710400000, 236.4501], [1428796800000, 235.7386], [1428883200000, 223.9464], [1428969600000, 217.9995], [1429056000000, 223.0161], [1429142400000, 227.9995], [1429228800000, 222.4171382852381], [1429315200000, 222.9536999999999], [1429401600000, 222.6812], [1429488000000, 224.3086], [1429574400000, 234.6867000000001], [1429660800000, 233.7807], [1429747200000, 235.1735], [1429833600000, 230.9863], [1429920000000, 225.9568], [1430006400000, 218.5065], [1430092800000, 228.5712], [1430179200000, 225.6422999999999], [1430265600000, 225.9978], [1430352000000, 236.2208], [1430438400000, 232.7402999999999], [1430524800000, 235.1563], [1430611200000, 239.882], [1430697600000, 238.9705999999999], [1430784000000, 236.0015000000001], [1430870400000, 229.8152], [1430956800000, 236.8450000000001], [1431043200000, 243.657], [1431129600000, 241.5321], [1431216000000, 239.5548], [1431302400000, 241.8506], [1431388800000, 241.5189000000001], [1431475200000, 235.4793], [1431561600000, 236.704], [1431648000000, 236.9908], [1431734400000, 235.9781], [1431820800000, 236.0847], [1431907200000, 232.4757], [1431993600000, 231.9446000000001], [1432080000000, 234.048], [1432166400000, 235.4204999999999], [1432252800000, 240.9352], [1432339200000, 239.0329940983288], [1432425600000, 241.2417000000001], [1432512000000, 237.441], [1432598400000, 237.4971], [1432684800000, 237.2305], [1432771200000, 237.4891000000001], [1432857600000, 237.0714], [1432944000000, 233.4572], [1433030400000, 229.6785], [1433116800000, 223.3178], [1433203200000, 225.7387], [1433289600000, 225.4831], [1433376000000, 224.1748], [1433462400000, 224.7413], [1433548800000, 225.2889], [1433635200000, 223.6157], [1433721600000, 228.7013000000001], [1433808000000, 229.5263999999999], [1433894400000, 229.0022], [1433980800000, 229.9343000000001], [1434067200000, 230.5181], [1434153600000, 232.6301], [1434240000000, 233.6681], [1434326400000, 237.2013], [1434412800000, 250.4743], [1434499200000, 247.6944000000001], [1434585600000, 248.5519999999999], [1434672000000, 242.9967], [1434758400000, 245.4754], [1434844800000, 243.9959], [1434931200000, 247.2790000000001], [1435017600000, 243.8486], [1435104000000, 240.5377000000001], [1435190400000, 242.4769], [1435276800000, 242.4819], [1435363200000, 250.621], [1435449600000, 248.3451], [1435536000000, 256.3038739667178], [1435622400000, 262.4063254601778], [1435708800000, 257.5058], [1435795200000, 254.6537509187558], [1435881600000, 255.161], [1435968000000, 260.415], [1436054400000, 271.0401208003443], [1436140800000, 268.595], [1436227200000, 265.3844519614875], [1436313600000, 269.6130164214266], [1436400000000, 268.8928000000001], [1436486400000, 285.0223], [1436572800000, 291.8907431569518], [1436659200000, 310.1915636277567], [1436745600000, 290.6655], [1436832000000, 286.6653999999999], [1436918400000, 283.9491880809251], [1437004800000, 276.6800999999999], [1437091200000, 278.3728], [1437177600000, 274.4838999999999], [1437264000000, 272.806179588859], [1437350400000, 277.2454], [1437436800000, 274.9428876558478], [1437523200000, 276.1756000000001], [1437609600000, 275.6135], [1437696000000, 288.4447167883336], [1437782400000, 288.3881586253327], [1437868800000, 291.7037645294952], [1437955200000, 292.6148], [1438041600000, 293.9933], [1438128000000, 288.7194990885924], [1438214400000, 286.8366], [1438300800000, 283.6760591480217], [1438387200000, 280.7014], [1438473600000, 281.7267], [1438560000000, 281.829870770124], [1438646400000, 284.2393], [1438732800000, 281.4010623650275], [1438819200000, 278.0134], [1438905600000, 278.5088137572824], [1438992000000, 259.8007283854695], [1439078400000, 264.3384833893257], [1439164800000, 263.5782], [1439251200000, 269.8668], [1439337600000, 267.7126], [1439424000000, 263.6558186169038], [1439510400000, 265.1281000000001], [1439596800000, 260.4762322897643], [1439683200000, 257.8158000000001], [1439769600000, 257.0634000000001], [1439856000000, 253.0766], [1439942400000, 225.9602], [1440028800000, 234.6379], [1440115200000, 232.3324000000001], [1440201600000, 229.1488], [1440288000000, 227.2453999999999], [1440374400000, 210.0943850637937], [1440460800000, 221.3282999999999], [1440547200000, 225.2666000000001], [1440633600000, 222.9704], [1440720000000, 231.6583], [1440806400000, 228.4683], [1440892800000, 228.0672], [1440979200000, 230.0781], [1441065600000, 227.1631], [1441152000000, 228.6427999999999], [1441238400000, 226.5658], [1441324800000, 230.0509], [1441411200000, 234.7495], [1441497600000, 239.1288999999999], [1441584000000, 240.503], [1441670400000, 243.5587], [1441756800000, 237.9452], [1441843200000, 238.7763], [1441929600000, 240.0007956438866], [1442016000000, 234.9098], [1442102400000, 230.5048000000001], [1442188800000, 230.21], [1442275200000, 229.9118], [1442361600000, 228.4640000000001], [1442448000000, 232.746], [1442534400000, 232.354], [1442620800000, 231.2732999999999], [1442707200000, 230.6597], [1442793600000, 226.4528], [1442880000000, 229.8831], [1442966400000, 229.8658], [1443052800000, 233.9255], [1443139200000, 234.9767], [1443225600000, 234.5278999999999], [1443312000000, 232.8346999999999], [1443398400000, 239.2765], [1443484800000, 237.2262], [1443571200000, 236.0463], [1443657600000, 237.7358], [1443744000000, 237.2592999999999], [1443830400000, 238.9718], [1443916800000, 238.6566], [1444003200000, 240.3238], [1444089600000, 246.444], [1444176000000, 243.2229000000001], [1444262400000, 242.9008], [1444348800000, 244.2051], [1444435200000, 245.3209], [1444521600000, 247.5392], [1444608000000, 245.5661], [1444694400000, 249.4051], [1444780800000, 252.6956], [1444867200000, 254.6295530746221], [1444953600000, 262.3735], [1445040000000, 270.3646], [1445126400000, 262.2345], [1445212800000, 264.2094], [1445299200000, 270.2663999999999], [1445385600000, 267.0187], [1445472000000, 273.5729999999999], [1445558400000, 277.3883999999999], [1445644800000, 282.9566], [1445731200000, 284.749], [1445817600000, 285.5159], [1445904000000, 294.8093999999999], [1445990400000, 303.6366], [1446076800000, 313.9073], [1446163200000, 325.8005999999999], [1446249600000, 312.2783], [1446336000000, 324.9970999999999], [1446422400000, 350.7678], [1446508800000, 398.2369], [1446595200000, 404.3588], [1446681600000, 387.473], [1446768000000, 372.8260000000001], [1446854400000, 387.0599999999999], [1446940800000, 371.5020000000001], [1447027200000, 380.6439999999999], [1447113600000, 336.8729999999999], [1447200000000, 305.9829999999999], [1447286400000, 337.955], [1447372800000, 337.372], [1447459200000, 331.501], [1447545600000, 320.484], [1447632000000, 331.353], [1447718400000, 335.111], [1447804800000, 335.447], [1447891200000, 325.5680000000001], [1447977600000, 321.417], [1448064000000, 325.159], [1448150400000, 323.395], [1448236800000, 322.8290000000001], [1448323200000, 319.9789999999999], [1448409600000, 328.1689999999999], [1448496000000, 356.245], [1448582400000, 359.011], [1448668800000, 356.212], [1448755200000, 371.4880000000001], [1448841600000, 378.265], [1448928000000, 363.4499999999999], [1449014400000, 358.8199999999999], [1449100800000, 360.954], [1449187200000, 362.7669999999999], [1449273600000, 390.645], [1449360000000, 394.0109999999999], [1449446400000, 394.355], [1449532800000, 412.9300000000001], [1449619200000, 417.3679999999999], [1449705600000, 416.91], [1449792000000, 452.4149999999999], [1449878400000, 434.903], [1449964800000, 435.8150000000001], [1450051200000, 443.277], [1450137600000, 464.9989999999999], [1450224000000, 453.9139999999999], [1450310400000, 456.473], [1450396800000, 463.816], [1450483200000, 462.0529999999999], [1450569600000, 441.1289999999999], [1450656000000, 437.8090000000001], [1450742400000, 436.2370000000001], [1450828800000, 442.821], [1450915200000, 454.907], [1451001600000, 455.7419999999999], [1451088000000, 417.092], [1451174400000, 423.925], [1451260800000, 420.708], [1451347200000, 432.1540000000001], [1451433600000, 426.811], [1451520000000, 431.139], [1451606400000, 434.4269999999999], [1451692800000, 434.3260000000001], [1451779200000, 430.198], [1451865600000, 433.5339999999999], [1451952000000, 432.115], [1452038400000, 431.3430000000001], [1452124800000, 456.434], [1452211200000, 454.6963000000001], [1452297600000, 448.7090000000001], [1452384000000, 447.2069999999999], [1452470400000, 448.3490000000001], [1452556800000, 445.792], [1452643200000, 432.1200000000001], [1452729600000, 429.2429999999999], [1452816000000, 365.912], [1452902400000, 384.813], [1452988800000, 386.3560000000001], [1453075200000, 385.1320000000001], [1453161600000, 379.015], [1453248000000, 418.848], [1453334400000, 407.8150000000002], [1453420800000, 377.2100000000001], [1453507200000, 389.6409999999999], [1453593600000, 403.0559999999999], [1453680000000, 392.3260000000001], [1453766400000, 392.4559999999998], [1453852800000, 394.491], [1453939200000, 379.5519999999999], [1454025600000, 378.912], [1454112000000, 376.921], [1454198400000, 364.352], [1454284800000, 371.041], [1454371200000, 372.8679999999999], [1454457600000, 368.124], [1454544000000, 388.3759999999999], [1454630400000, 385.7330000000001], [1454716800000, 374.9049999999999], [1454803200000, 375.5170000000002], [1454889600000, 371.1359999999999], [1454976000000, 373.909], [1455062400000, 379.829], [1455148800000, 377.058], [1455235200000, 382.72], [1455321600000, 390.4240000000001], [1455408000000, 404.1440000000001], [1455494400000, 399.379], [1455580800000, 407.0129999999998], [1455667200000, 416.8499999999999], [1455753600000, 421.4220000000001], [1455840000000, 419.377], [1455926400000, 441.1199999999998], [1456012800000, 438.6499999999999], [1456099200000, 437.8029999999999], [1456185600000, 420.3020000000001], [1456272000000, 423.1870000000001], [1456358400000, 423.142], [1456444800000, 427.029], [1456531200000, 432.1390000000001], [1456617600000, 432.7970000000001], [1456704000000, 436.919], [1456790400000, 431.371], [1456876800000, 424.485], [1456963200000, 420.1709999999999], [1457049600000, 409.408], [1457136000000, 397.2410000000001], [1457222400000, 404.912], [1457308800000, 413.74], [1457395200000, 411.8770000000001], [1457481600000, 412.3190000000001], [1457568000000, 416.2920000000001], [1457654400000, 419.299], [1457740800000, 410.8010000000001], [1457827200000, 412.4949999999999], [1457913600000, 414.585], [1458000000000, 415.6410000000001], [1458086400000, 416.5440000000001], [1458172800000, 418.4000000000001], [1458259200000, 408.28], [1458345600000, 406.7389999999999], [1458432000000, 411.8820000000001], [1458518400000, 411.091], [1458604800000, 416.36], [1458691200000, 418.076], [1458777600000, 414.8829999999999], [1458864000000, 415.743], [1458950400000, 416.7149999999999], [1459036800000, 425.349], [1459123200000, 422.756], [1459209600000, 415.5470000000001], [1459296000000, 412.597], [1459382400000, 415.366], [1459468800000, 416.754], [1459555200000, 419.6700000000001], [1459641600000, 418.1850000000001], [1459728000000, 419.437], [1459814400000, 422.651], [1459900800000, 421.65599999999995], [1459987200000, 421.076], [1460073600000, 418.4740000000001], [1460160000000, 416.145], [1460246400000, 421.133], [1460332800000, 422.5840000000001], [1460419200000, 426.181], [1460505600000, 424.26900000000006], [1460592000000, 424.932], [1460678400000, 429.966], [1460764800000, 430.941], [1460851200000, 428.027], [1460937600000, 428.5059999999998], [1461024000000, 436.37], [1461110400000, 441.916], [1461196800000, 451.0830000000002], [1461283200000, 447.488], [1461369600000, 451.818], [1461456000000, 459.9229999999999], [1461542400000, 460.4659999999999], [1461628800000, 467.8559999999999], [1461715200000, 444.789], [1461801600000, 449.3809999999999], [1461888000000, 456.5660000000001], [1461974400000, 447.90299999999985], [1462060800000, 452.592], [1462147200000, 444.488], [1462233600000, 450.3], [1462320000000, 447.119], [1462406400000, 448.9219999999999], [1462492800000, 461.6200000000001], [1462579200000, 459.2479999999998], [1462665600000, 459.283], [1462752000000, 461.74700000000007], [1462838400000, 450.3259999999999], [1462924800000, 452.179], [1463011200000, 454.7310000000001], [1463097600000, 456.0070000000001], [1463184000000, 455.5030000000001], [1463270400000, 457.837], [1463356800000, 454.893], [1463443200000, 452.422], [1463529600000, 453.3710000000001], [1463616000000, 440.5389999999998], [1463702400000, 441.888], [1463788800000, 443.1469999999999], [1463875200000, 438.6750000000001], [1463961600000, 443.175], [1464048000000, 445.769], [1464134400000, 449.0549999999999], [1464220800000, 453.73199999999997], [1464307200000, 473.8640000000001], [1464393600000, 528.545], [1464480000000, 524.624], [1464566400000, 534.2240000000002], [1464652800000, 531.4070000000002], [1464739200000, 539.1679999999999], [1464825600000, 538.832], [1464912000000, 570.4449999999999], [1464998400000, 573.5259999999998], [1465084800000, 574.4756747207706], [1465171200000, 585.563], [1465257600000, 577.9106942699359], [1465344000000, 583.1389999999999], [1465430400000, 577.607], [1465516800000, 579.2276478801407], [1465603200000, 597.746561993518], [1465689600000, 671.7122784950529], [1465776000000, 704.2918521711082], [1465862400000, 685.932131564762], [1465948800000, 693.2269999999999], [1466035200000, 765.4629999999999], [1466121600000, 749.4189999999999], [1466208000000, 756.7996012580627], [1466294400000, 764.0459999999999], [1466380800000, 734.1617358035669], [1466467200000, 664.9965293554344], [1466553600000, 591.895], [1466640000000, 623.2062563646158], [1466726400000, 661.4166051593036], [1466812800000, 666.0566978558096], [1466899200000, 629.3059999999999], [1466985600000, 648.2999999999998], [1467072000000, 646.8429999999997], [1467158400000, 637.437], [1467244800000, 671.6159999999998], [1467331200000, 677.7099999999999], [1467417600000, 693.0470000000001], [1467504000000, 664.7451003433932], [1467590400000, 678.7005716794641], [1467676800000, 667.5305180233454], [1467763200000, 675.9190000000001], [1467849600000, 638.6200000000001], [1467936000000, 665.744506272113], [1468022400000, 653.8705124993986], [1468108800000, 650.3210000000003], [1468195200000, 649.07], [1468281600000, 670.9480000000001], [1468368000000, 658.1500000000001], [1468454400000, 659.5760000000002], [1468540800000, 665.6869915031888], [1468627200000, 663.5010000000002], [1468713600000, 678.3905582459545], [1468800000000, 674.7874378266207], [1468886400000, 674.5573889807015], [1468972800000, 670.8934740599931], [1469059200000, 665.3643845142738], [1469145600000, 651.0899999999999], [1469232000000, 655.4230000000001], [1469318400000, 660.848923533057], [1469404800000, 654.5186383481073], [1469491200000, 651.3099999999998], [1469577600000, 655.618263277738], [1469664000000, 655.8904301748952], [1469750400000, 657.3423791946653], [1469836800000, 654.5489803055398], [1469923200000, 623.7457712533022], [1470009600000, 605.7469999999995], [1470096000000, 517.1349294487607], [1470182400000, 567.3810616156014], [1470268800000, 576.6037067248545], [1470355200000, 576.0338000000002], [1470441600000, 587.6349999999999], [1470528000000, 592.6675], [1470614400000, 590.7125], [1470700800000, 585.3024999999998], [1470787200000, 592.266843297098], [1470873600000, 587.6138000000002], [1470960000000, 586.5299999999999], [1471046400000, 584.4688683346069], [1471132800000, 570.7366999999999], [1471219200000, 567.7342836594349], [1471305600000, 578.2231511023564], [1471392000000, 571.461914957742], [1471478400000, 572.2684198487193], [1471564800000, 573.4919906794345], [1471651200000, 581.9462730812136], [1471737600000, 580.5870271371734], [1471824000000, 585.2175000000001], [1471910400000, 582.0038], [1471996800000, 577.8880531621712], [1472083200000, 576.1163000000001], [1472169600000, 578.1037999999998], [1472256000000, 568.5750000000002], [1472342400000, 573.5288], [1472428800000, 572.5102283617088], [1472515200000, 576.08], [1472601600000, 571.7582826763035], [1472688000000, 571.59], [1472774400000, 573.9624999999999], [1472860800000, 596.1950000000002], [1472947200000, 608.6887999999999], [1473033600000, 606.6438], [1473120000000, 610.6837999999999], [1473206400000, 614.3264885962195], [1473292800000, 625.4675000000001], [1473379200000, 621.9124999999999], [1473465600000, 623.0588000000001], [1473552000000, 605.3154151790802], [1473638400000, 605.8312052420321], [1473724800000, 606.6450000000001], [1473811200000, 608.4250000000002], [1473897600000, 605.5838000000001], [1473984000000, 606.5533658936915], [1474070400000, 605.3438], [1474156800000, 608.7950000000002], [1474243200000, 608.6176342973245], [1474329600000, 607.7688125632665], [1474416000000, 595.8402492264345], [1474502400000, 594.9587999999999], [1474588800000, 601.8925], [1474675200000, 601.2725000000003], [1474761600000, 599.3112999999997], [1474848000000, 606.0970645363005], [1474934400000, 604.2074999999999], [1475020800000, 603.5549999999997], [1475107200000, 604.6375], [1475193600000, 608.2575], [1475280000000, 614.0598845176643], [1475366400000, 610.3588000000001], [1475452800000, 611.9609026561799], [1475539200000, 609.2425], [1475625600000, 611.6363], [1475712000000, 610.9800000000001], [1475798400000, 616.2188000000002], [1475884800000, 616.5338], [1475971200000, 614.6238000000001], [1476057600000, 614.3138], [1476144000000, 640.7], [1476230400000, 635.6163], [1476316800000, 634.6223388041233], [1476403200000, 638.1799999999996], [1476489600000, 637.2088], [1476576000000, 640.1862999999997], [1476662400000, 637.4825000000001], [1476748800000, 635.465], [1476835200000, 628.4375], [1476921600000, 628.5425], [1477008000000, 629.7187999999995], [1477094400000, 652.0875], [1477180800000, 649.6263000000002], [1477267200000, 648.7663], [1477353600000, 652.6338], [1477440000000, 673.4175000000001], [1477526400000, 683.3724999999998], [1477612800000, 687.5587999999999], [1477699200000, 715.4103243232163], [1477785600000, 697.2773624560577], [1477872000000, 698.5762999999998], [1477958400000, 729.2049999999999], [1478044800000, 734.1638], [1478131200000, 686.0612999999995], [1478217600000, 702.11], [1478304000000, 704.0224999999997], [1478390400000, 712.1263000000001], [1478476800000, 706.0925], [1478563200000, 708.9399999999999], [1478649600000, 721.1774999999999], [1478736000000, 713.2141433353438], [1478822400000, 715.6425000000002], [1478908800000, 703.76], [1478995200000, 702.3463169808624], [1479081600000, 705.55], [1479168000000, 712.158899800779], [1479254400000, 740.8993548679168], [1479340800000, 738.4799999999999], [1479427200000, 748.4263], [1479513600000, 750.4100480749124], [1479600000000, 728.3713], [1479686400000, 736.5049999999998], [1479772800000, 748.1225], [1479859200000, 741.1662465178737], [1479945600000, 736.7486937230678], [1480032000000, 738.2039432073005], [1480118400000, 732.7338000000001], [1480204800000, 727.7574999999999], [1480291200000, 731.8424613583487], [1480377600000, 731.81], [1480464000000, 742.3587999999997], [1480550400000, 753.0550000000001], [1480636800000, 771.1713], [1480723200000, 764.8338233239153], [1480809600000, 766.3687999999997], [1480896000000, 752.2399999999997], [1480982400000, 758.6924999999999], [1481068800000, 763.9149999999998], [1481155200000, 768.6749999999998], [1481241600000, 769.8412999999998], [1481328000000, 773.1262933845235], [1481414400000, 768.2324999999998], [1481500800000, 778.4323012225746], [1481587200000, 778.2150000000004], [1481673600000, 775.8913000000002], [1481760000000, 775.5724999999998], [1481846400000, 781.3375000000001], [1481932800000, 788.8774999999998], [1482019200000, 788.4413000000003], [1482105600000, 790.1162999999996], [1482192000000, 799.7212999999999], [1482278400000, 826.6207679031857], [1482364800000, 859.3662999999999], [1482451200000, 917.0988], [1482537600000, 891.6813], [1482624000000, 894.3371331688621], [1482710400000, 902.6063], [1482796800000, 929.1150000000001], [1482883200000, 977.1399999999999], [1482969600000, 971.7212999999998], [1483056000000, 960.9250000000001], [1483142400000, 966.5689708732557], [1483228800000, 998.05], [1483315200000, 1019.1988000000001], [1483401600000, 1035.5313], [1483488000000, 1130.8495163207272], [1483574400000, 990.6674999999997], [1483660800000, 894.0325], [1483747200000, 900.0787999999999], [1483833600000, 908.0850000000004], [1483920000000, 901.8788880390637], [1484006400000, 905.7594626984339], [1484092800000, 784.2775000000003], [1484179200000, 799.4275], [1484265600000, 824.4988000000003], [1484352000000, 817.8250000000003], [1484438400000, 821.9124999999998], [1484524800000, 828.3099999999998], [1484611200000, 904.1525000000003], [1484697600000, 873.9349999999996], [1484784000000, 896.89], [1484870400000, 891.4588], [1484956800000, 919.6775000000004], [1485043200000, 918.7788120146554], [1485129600000, 918.9075000000001], [1485216000000, 885.7887999999999], [1485302400000, 893.4737999999999], [1485388800000, 915.3599999999999], [1485475200000, 918.2875], [1485561600000, 920.245], [1485648000000, 914.1424999999998], [1485734400000, 920.9911458822621], [1485820800000, 968.2662999999999], [1485907200000, 986.5728903454353], [1485993600000, 1008.4033182231417], [1486080000000, 1015.4425000000001], [1486166400000, 1033.63], [1486252800000, 1016.9261585954309], [1486339200000, 1024.5110886063283], [1486425600000, 1051.5000000000005], [1486512000000, 1055.3013], [1486598400000, 981.7049999999999], [1486684800000, 992.1113], [1486771200000, 1010.9462999999998], [1486857600000, 1000.5237691031117], [1486944000000, 996.1039358698114], [1487030400000, 1010.1021618594351], [1487116800000, 1008.8812999999999], [1487203200000, 1034.9038000000003], [1487289600000, 1056.5580910529798], [1487376000000, 1058.5809306486606], [1487462400000, 1053.5024999999998], [1487548800000, 1082.0168234958396], [1487635200000, 1125.4088148356532], [1487721600000, 1123.3049999999998], [1487808000000, 1185.6547846887681], [1487894400000, 1180.8709999999999], [1487980800000, 1149.4036781698383], [1488067200000, 1180.6730000000002], [1488153600000, 1193.1295150369579], [1488240000000, 1190.3309999999997], [1488326400000, 1227.4771489660905], [1488412800000, 1262.0810000000001], [1488499200000, 1289.5406116191473], [1488585600000, 1268.8145320151446], [1488672000000, 1276.2330000000002], [1488758400000, 1280.9859999999996], [1488844800000, 1233.0565084079958], [1488931200000, 1149.6310000000003], [1489017600000, 1191.2367224851023], [1489104000000, 1100.0105899712148], [1489190400000, 1181.2041227520244], [1489276800000, 1230.56], [1489363200000, 1243.2649223901926], [1489449600000, 1246.5567394807624], [1489536000000, 1258.4569999999999], [1489622400000, 1217.3145410078776], [1489708800000, 1074.0938695062455], [1489795200000, 961.9620000000001], [1489881600000, 1023.4901008756748], [1489968000000, 1036.3320123738727], [1490054400000, 1112.701532272989], [1490140800000, 1039.8401726478428], [1490227200000, 1029.1394137431018], [1490313600000, 940.0206265591036], [1490400000000, 965.2650626610691], [1490486400000, 954.4766599717783], [1490572800000, 1042.6050726198062], [1490659200000, 1044.7219999999998], [1490745600000, 1038.617197616254], [1490832000000, 1031.4929056461594], [1490918400000, 1078.274711090609], [1491004800000, 1085.2529999999997], [1491091200000, 1098.0679999999998], [1491177600000, 1139.3082754455547], [1491264000000, 1140.4975903801683], [1491350400000, 1132.339904992776], [1491436800000, 1194.2109999999996], [1491523200000, 1189.775123703212], [1491609600000, 1182.271], [1491696000000, 1187.6989999999998], [1491782400000, 1210.515], [1491868800000, 1226.618], [1491955200000, 1218.299], [1492041600000, 1174.209], [1492128000000, 1183.687], [1492214400000, 1180.323], [1492300800000, 1186.31], [1492387200000, 1205.143], [1492473600000, 1217.58], [1492560000000, 1225.721], [1492646400000, 1253.904], [1492732800000, 1254.9875], [1492819200000, 1249.67], [1492905600000, 1247.8388], [1492992000000, 1249.9563], [1493078400000, 1263.045], [1493164800000, 1287.32], [1493251200000, 1331.3125], [1493337600000, 1321.205], [1493424000000, 1327.6863], [1493510400000, 1349.8863], [1493596800000, 1408.445], [1493683200000, 1442.245], [1493769600000, 1485.8175], [1493856000000, 1499.36], [1493942400000, 1506.0438], [1494028800000, 1547.815], [1494115200000, 1557.8838], [1494201600000, 1633.38], [1494288000000, 1700.1775], [1494374400000, 1747.3175], [1494460800000, 1808.7788], [1494547200000, 1700.7725], [1494633600000, 1751.73], [1494720000000, 1767.7088], [1494806400000, 1696.13], [1494892800000, 1716.315], [1494979200000, 1804.4513], [1495065600000, 1889.4088], [1495152000000, 1972.9013], [1495238400000, 2043.42], [1495324800000, 2053.8788], [1495411200000, 2109.3963], [1495497600000, 2292.7025], [1495584000000, 2448.1313], [1495670400000, 2366.9725], [1495756800000, 2262.36], [1495843200000, 2082.845], [1495929600000, 2219.4875], [1496016000000, 2286.2663], [1496102400000, 2167.6438], [1496188800000, 2329.0775], [1496275200000, 2436.0363], [1496361600000, 2514.2275], [1496448000000, 2562.54], [1496534400000, 2556.405], [1496620800000, 2721.5525], [1496791795701, 2903.4438], [1496793600000, 2716.0588], [1496965161692, 2824.2038], [1496966400000, 2817.0638], [1497052800000, 2937.4538], [1497139200000, 3013.7138], [1497225600000, 2701.255], [1497312000000, 2736.1888], [1497398400000, 2503.40127294964], [1497484800000, 2444.49371230854], [1497571200000, 2513.81034764638], [1497657600000, 2683.5713440378], [1497744000000, 2577.21936114026], [1497830400000, 2620.13645139993], [1497916800000, 2777.79265567999], [1498003200000, 2717.00310788798], [1498089600000, 2724.91221331707], [1498176000000, 2737.74623109829], [1498262400000, 2618.41673570888], [1498348800000, 2606.8632352265], [1498435200000, 2488.19416337737], [1498521600000, 2531.34899878484], [1498608000000, 2575.43777025493], [1498694400000, 2561.68143454198], [1498780800000, 2490.75348798041], [1498867200000, 2439.82055973389], [1498953600000, 2518.54569431263], [1499040000000, 2576.7146569746], [1499126400000, 2610.99757474179], [1499212800000, 2607.21930206989], [1499299200000, 2612.67042830497], [1499385600000, 2509.21820981689], [1499472000000, 2563.19551359687], [1499558400000, 2519.21739188349], [1499644800000, 2363.51568155487], [1499731200000, 2353.91066125207], [1499817600000, 2387.32891171674], [1499904000000, 2364.32126576467], [1499990400000, 2228.72879099011], [1500076800000, 2033.29108803266], [1500163200000, 1927.09906562122], [1500249600000, 2181.28608069932], [1500336000000, 2303.88935545646], [1500422400000, 2274.62136530529], [1500508800000, 2766.82511908244], [1500595200000, 2658.07491237597], [1500681600000, 2802.34788667742], [1500768000000, 2731.60400741206], [1500854400000, 2751.3470702262], [1500940800000, 2563.74789523364], [1501027200000, 2507.38968460757], [1501113600000, 2649.44120113807], [1501200000000, 2798.82931330334], [1501286400000, 2720.10226250836], [1501372800000, 2740.32359181287], [1501459200000, 2871.3680152452], [1501545600000, 2737.76379241919], [1501632000000, 2697.8509023017], [1501718400000, 2777.36799394604], [1501804800000, 2845.89716230278], [1501891200000, 3237.07069450803], [1501977600000, 3189.7324534615], [1502064000000, 3356.70973808691], [1502150400000, 3410.45008044914], [1502236800000, 3330.89782529283], [1502323200000, 3367.90538708821], [1502409600000, 3562.58766284089], [1502496000000, 3800.14062506336], [1502582400000, 4023.25452207874], [1502668800000, 4175.69529134641], [1502755200000, 4149.30875941234], [1502841600000, 4262.5587912331], [1502928000000, 4286.91743450516], [1503014400000, 4097.68580247375], [1503100800000, 4115.21198836208], [1503187200000, 4001.22863247285], [1503273600000, 3935.22993283277], [1503360000000, 4008.41668521521], [1503446400000, 4069.0802390251], [1503532800000, 4167.28917680219], [1503619200000, 4199.89203158457], [1503705600000, 4123.51337132546], [1503792000000, 4116.46873916326], [1503878400000, 4216.24277330547], [1503964800000, 4394.06343624723], [1504051200000, 4553.80652713686], [1504137600000, 4702.93816308458], [1504224000000, 4863.19304868405], [1504310400000, 4556.5977768913], [1504396800000, 4632.46233076675], [1504483200000, 4317.18518771612], [1504569600000, 4447.87559221093], [1504656000000, 4592.25027789782], [1504742400000, 4584.82027519271], [1504828800000, 4285.20793872097], [1504915200000, 4245.60734112386], [1505001600000, 4132.6646422833], [1505088000000, 4164.97621199409], [1505174400000, 4130.95628475182], [1505260800000, 3883.65200134942], [1505347200000, 3100.66815470405], [1505433600000, 3587.9431219159], [1505520000000, 3606.27519721432], [1505606400000, 3553.43231625865], [1505692800000, 4012.28582499344], [1505779200000, 3898.93473248234], [1505865600000, 3858.02095060051], [1505952000000, 3628.05858753461], [1506038400000, 3622.1830393146], [1506124800000, 3763.41140627887], [1506211200000, 3672.30507973319], [1506297600000, 3866.70515274174], [1506384000000, 3852.93520487706], [1506470400000, 4134.21241496119], [1506556800000, 4134.88495085993], [1506643200000, 4130.41524600294], [1506729600000, 4297.79117003372], [1506816000000, 4347.57873545332], [1506902400000, 4383.32549736353], [1506988800000, 4319.06921246641], [1507075200000, 4231.46418703684], [1507161600000, 4290.38040438142], [1507248000000, 4347.54723950876], [1507334400000, 4405.0162007055], [1507420800000, 4562.32193109848], [1507507200000, 4758.72164979337], [1507593600000, 4769.47734676417], [1507680000000, 4807.92589489803], [1507766400000, 5331.61930195449], [1507852800000, 5610.49316222874], [1507939200000, 5679.21622165638], [1508025600000, 5684.04710411913], [1508112000000, 5728.93275055115], [1508198400000, 5643.85907705172], [1508284800000, 5592.78222421231], [1508371200000, 5696.38014134367], [1508457600000, 5952.7713587923], [1508544000000, 6043.96673129542], [1508630400000, 6003.4640189335], [1508716800000, 5959.98304059248], [1508803200000, 5550.08158528565], [1508889600000, 5704.87480650775], [1508976000000, 5883.05318447111], [1509062400000, 5775.51666530736], [1509148800000, 5734.30745398276], [1509235200000, 6123.54977135535], [1509321600000, 6104.38870915078], [1509408000000, 6368.65164135745], [1509494400000, 6714.78898671902], [1509580800000, 7059.7938605488], [1509667200000, 7234.73838202036], [1509753600000, 7390.53522652395], [1509840000000, 7416.26795098255], [1509926400000, 7103.04608116249], [1510012800000, 7166.74137167631], [1510099200000, 7461.82443491069], [1510185600000, 7186.29277509382], [1510272000000, 6656.83969350681], [1510358400000, 6365.39785208573], [1510444800000, 5866.51312938529], [1510531200000, 6423.09140246102], [1510617600000, 6652.21677956026], [1510704000000, 7297.66883633256], [1510790400000, 7814.9444036566], [1510876800000, 7783.54246476388], [1510963200000, 7827.63146946899], [1511049600000, 8011.14742989044], [1511136000000, 8220.50996617654], [1511222400000, 8114.13367954416], [1511308800000, 8257.12389528835], [1511395200000, 8117.66434762603], [1511481600000, 8278.12843633886], [1511568000000, 8743.1593779107], [1511654400000, 9342.98864142167], [1511740800000, 9831.40292992241], [1511827200000, 10257.0674169803], [1511913600000, 9875.89230818829], [1512000000000, 10406.18772373], [1512086400000, 10995.7235531179], [1512172800000, 11162.5450307617], [1512259200000, 11256.7018287538], [1512345600000, 11634.9849554034], [1512432000000, 12174.2537397926], [1512518400000, 13876.4735794148], [1512604800000, 18491.1815542282], [1512691200000, 16908.0049521112], [1512777600000, 15266.4263894651], [1512864000000, 15808.5098961544], [1512950400000, 17106.5051488871], [1513036800000, 17249.920214656], [1513123200000, 16525.0444892988], [1513209600000, 16865.3616648753], [1513296000000, 17978.6305581344], [1513382400000, 19665.3949272416], [1513468800000, 19423.5801532525], [1513555200000, 19188.0539569348], [1513641600000, 18174.1867596316], [1513728000000, 17059.8883502646], [1513814400000, 16355.2382257401], [1513900800000, 14084.8249267767], [1513987200000, 15241.9164656487], [1514073600000, 14451.1145466743], [1514160000000, 14302.7537081839], [1514246400000, 16616.6820570844], [1514332800000, 16373.0810795739], [1514419200000, 15067.8994980567], [1514505600000, 15096.5594360709], [1514592000000, 13620.3618741461], [1514678400000, 14839.5907854184], [1514764800000, 14093.6068305274], [1514851200000, 15321.9328517642], [1514937600000, 15583.8855377593], [1515024000000, 15976.365194174], [1515110400000, 18336.9229801691], [1515196800000, 18343.6624934071], [1515283200000, 17039.8739138488], [1515369600000, 15900.3650216006], [1515456000000, 15604.7234082405], [1515542400000, 15509.2701268168], [1515628800000, 14050.5696063543], [1515715200000, 14759.294629915], [1515801600000, 15193.3148979847], [1515888000000, 14476.5889713691], [1515974400000, 14411.9977069182], [1516060800000, 11723.9135206658], [1516147200000, 11415.3928442768], [1516233600000, 11569.4249920858], [1516320000000, 11758.4693327652], [1516406400000, 13260.2711872786], [1516492800000, 11703.5048628599], [1516579200000, 10991.5950130802], [1516665600000, 10894.0009053405], [1516752000000, 11277.1159704489], [1516838400000, 11225.2544568853], [1516924800000, 11044.9072365744], [1517011200000, 11456.2135062799], [1517097600000, 11665.6969300847], [1517184000000, 11157.6638878978], [1517270718718, 11095.2106891266], [1517356800000, 9989.68977659669], [1517443200000, 10114.495086226], [1517529600000, 9070.55544937356], [1517616000000, 8830.16351005472], [1517702400000, 8994.91486607664], [1517788800000, 8315.51749143615], [1517875200000, 6852.05270051491], [1517961600000, 7711.2203289411], [1518048328226, 7611.61027778882], [1518134400000, 8208.57208449673], [1518220800000, 8650.71675042998], [1518307200000, 8574.67763849023], [1518393600000, 8168.63777979963], [1518480000000, 8911.3985742083], [1518566400000, 8518.26907619091], [1518652800000, 9428.02560716167], [1518739200000, 10112.5801434467], [1518825600000, 10155.2770555258], [1518912000000, 11006.647685962], [1518998400000, 10523.429305801], [1519084800000, 11103.0900361497], [1519171200000, 11502.3770496396], [1519257600000, 9973.59211951887], [1519344000000, 10032.863112314], [1519430400000, 10301.883609336], [1519516800000, 9742.62819006538], [1519603200000, 9699.51718151529], [1519689600000, 10355.5781264856], [1519776000000, 10709.3884310463], [1519862400000, 10346.664931035], [1519948800000, 10859.5521943893], [1520035200000, 11098.0758092511], [1520121600000, 11356.0416016724], [1520208000000, 11286.6762891691], [1520294400000, 11526.4209302906], [1520380800000, 10727.716249205], [1520467200000, 9999.68248604677], [1520553600000, 9420.85488958591], [1520640000000, 9276.33759031006], [1520726400000, 8890.86580399612], [1520812800000, 9512.45611286134], [1520899200000, 9232.70587979953], [1520985600000, 9216.53097626842], [1521072000000, 8307.22338626638], [1521158400000, 8359.36528334063], [1521244800000, 8520.00815917187], [1521331200000, 8063.4097921227], [1521417600000, 8217.84219996114], [1521504000000, 8640.52143024258], [1521590400000, 8926.82049620007], [1521676800000, 8970.31790627893], [1521763200000, 8736.38576909829], [1521849600000, 8824.59283177853], [1521936000000, 8621.15079225785], [1522022400000, 8478.63819558905], [1522108800000, 8186.19606281355], [1522195200000, 7836.76304425289], [1522281600000, 7930.77954092755], [1522368000000, 7126.89519256736], [1522454400000, 6897.26857786945], [1522540800000, 6975.27531402076], [1522627200000, 6836.57232729678], [1522713600000, 7080.41723231299], [1522800000000, 7464.35609406668], [1522886400000, 6845.31597826858], [1522972800000, 6855.42230460675], [1523059200000, 6628.75445585626], [1523145600000, 6884.47723505995], [1523232000000, 6984.8089659738], [1523318400000, 6778.4915700774], [1523404800000, 6829.31650379539], [1523491200000, 6925.10199547663], [1523577600000, 7836.39699399927], [1523664000000, 7845.75284010594], [1523750400000, 7897.71993085806], [1523836800000, 8258.06708369755], [1523923200000, 8030.99926195254], [1524009600000, 7873.63551010255], [1524096000000, 8097.41823310301], [1524182400000, 8277.46063916253], [1524268800000, 8499.45000871487], [1524355200000, 8878.42826029723], [1524441600000, 8769.01199947348], [1524528000000, 8881.39170434545], [1524614400000, 9716.48975733072], [1524700800000, 8825.2025000693], [1524787200000, 9221.86802618436], [1524873600000, 8940.38112200719], [1524960000000, 9249.65952508923], [1525046400000, 9325.62885104679], [1525132800000, 9198.18950997327], [1525219200000, 9024.3875117766], [1525305600000, 9122.65301383019], [1525392000000, 9551.13518381566], [1525478400000, 9634.53822597227], [1525564800000, 9766.24448548887], [1525651200000, 9583.87174550137], [1525737600000, 9308.26502572185], [1525824000000, 9149.51783314723], [1525910400000, 9248.96039558449], [1525996800000, 8969.94037574258], [1526083200000, 8405.5605632418], [1526169600000, 8480.58037612559], [1526256000000, 8683.9862031429], [1526342400000, 8696.97400387283], [1526428800000, 8473.34864905366], [1526515200000, 8343.1252216955], [1526601600000, 8056.05319237475], [1526688000000, 8202.82540955214], [1526774400000, 8180.62700991409], [1526860800000, 8443.26024591542], [1526947200000, 8354.99633591748], [1527033600000, 8009.2995067828], [1527120000000, 7556.31604292077], [1527206400000, 7563.43546544346], [1527292800000, 7472.02260174872], [1527379200000, 7328.87167436352], [1527465600000, 7326.64394017867], [1527552000000, 7111.06892466996], [1527638400000, 7451.2308372653], [1527724800000, 7380.8515577098], [1527811200000, 7460.16694986577], [1527897600000, 7487.07630453815], [1527984000000, 7567.51293761927], [1528070400000, 7639.87944568278], [1528156800000, 7463.65499554503], [1528243200000, 7569.01133988542], [1528329600000, 7605.57848860797], [1528416000000, 7613.77739337755], [1528502400000, 7561.16610052452], [1528588800000, 7415.00137858946], [1528675200000, 6767.49948454925], [1528761600000, 6880.13364875604], [1528848000000, 6581.82807522796], [1528934400000, 6334.09890642618], [1529020800000, 6644.72067973844], [1529107200000, 6406.80704950998], [1529193600000, 6478.81592433021], [1529280000000, 6418.85000188746], [1529366400000, 6677.53340291194], [1529452800000, 6708.68182027534], [1529539200000, 6714.37546123014], [1529625600000, 6690.832151608], [1529712000000, 6088.57944239075], [1529798400000, 6169.43726386169], [1529884800000, 6159.70719379414], [1529971200000, 6241.84871170546], [1530057600000, 6069.87260616628], [1530144000000, 6117.03640962264], [1530230400000, 5879.98897440643], [1530316800000, 6182.25674792295], [1530403200000, 6366.49946751097], [1530489600000, 6312.87724313135], [1530576000000, 6595.5473128085], [1530662400000, 6497.60735494217], [1530748800000, 6562.7217371211], [1530835200000, 6537.74087162192], [1530921600000, 6603.29470396542], [1531008000000, 6759.83608914527], [1531094400000, 6700.63547905209], [1531180800000, 6648.256795903321], [1531267200000, 6303.380664729323], [1531353600000, 6373.234804689936], [1531440000000, 6230.5901971047815], [1531526400000, 6248.397313033797], [1531612800000, 6246.13741162634], [1531699200000, 6349.695836710773], [1531785600000, 6741.6919602852695], [1531872000000, 7303.373059795855], [1531958400000, 7334.771372076191], [1532044800000, 7442.800738748321], [1532131200000, 7331.871523239401], [1532217600000, 7379.137271821949], [1532304000000, 7395.8664054401], [1532390400000, 7704.99567890457], [1532476800000, 8422.37392335148], [1532563200000, 8144.553482041877], [1532649600000, 7929.079243602936], [1532736000000, 8145.333168120666], [1532822400000, 8192.13570223742], [1532908800000, 8197.593575814208], [1532995200000, 8185.921138302329], [1533081600000, 7738.249304586212], [1533168000000, 7612.300753683117], [1533254400000, 7540.633150617145], [1533340800000, 7411.66638248389], [1533427200000, 7009.012011793916], [1533513600000, 7036.60933127318], [1533600000000, 6926.116778774001], [1533686400000, 6711.73242235858], [1533772800000, 6259.527709667595], [1533859200000, 6531.249786737056], [1533945600000, 6166.590417501742], [1534032000000, 6205.218479944854], [1534118400000, 6318.407221774424], [1534204800000, 6283.99625058153], [1534291200000, 6212.2260400556115], [1534377600000, 6267.76870877378], [1534464000000, 6309.385226218996], [1534550400000, 6566.715163377382], [1534636800000, 6382.0605914076095], [1534723200000, 6475.494019667379], [1534809600000, 6242.8824384007985], [1534896000000, 6467.271730309675], [1534982400000, 6336.6563271087025], [1535068800000, 6521.065718268509], [1535155200000, 6684.379144206679], [1535241600000, 6736.036057516459], [1535328000000, 6728.2880026290095], [1535414400000, 6911.096028949572], [1535500800000, 7075.9834051051], [1535587200000, 7046.748947280631], [1535673600000, 7024.14510718162], [1535760000000, 7027.631279401852], [1535846400000, 7211.270767868822], [1535932800000, 7305.272801003753], [1536019200000, 7260.897516647864], [1536105600000, 7359.462992923027], [1536192000000, 6764.926773778225], [1536278400000, 6498.2796126096555], [1536364800000, 6397.81887024023], [1536451200000, 6181.19445075213], [1536537600000, 6260.764806062476], [1536624000000, 6308.135474616629], [1536710400000, 6278.917069628559], [1536796800000, 6330.574238022664], [1536883200000, 6486.461630393616], [1536969600000, 6500.01225461663], [1537056000000, 6530.221146234664], [1537142400000, 6506.248493217292], [1537228800000, 6250.121727322729], [1537315200000, 6347.069255451608], [1537401600000, 6383.753841158382], [1537488000000, 6491.885242276883], [1537574400000, 6753.8344721403255], [1537660800000, 6747.032172960024], [1537747200000, 6711.029451815934], [1537833600000, 6596.187313045577], [1537920000000, 6441.311544542553], [1538006400000, 6462.795635292239], [1538092800000, 6704.704245729721], [1538179200000, 6634.765964572905], [1538265600000, 6606.073795422076], [1538352000000, 6626.037956098487], [1538438400000, 6598.841259755177], [1538524800000, 6524.60769791521], [1538611200000, 6499.262379115219], [1538697600000, 6578.701809284849], [1538784000000, 6621.189363919919], [1538870400000, 6589.124681678888], [1538956800000, 6584.832153052514], [1539043200000, 6653.089450830426], [1539129600000, 6633.745729917172], [1539216000000, 6582.455082387102], [1539302400000, 6234.367641454442], [1539388800000, 6285.592982799844], [1539475200000, 6314.883384422573], [1539561600000, 6328.554760034424], [1539648000000, 6659.87550840943], [1539734400000, 6635.125667516839], [1539820800000, 6635.741832487943], [1539907200000, 6517.511087710534], [1539993600000, 6483.827769471798], [1540080000000, 6532.678750383806], [1540166400000, 6529.670021370842], [1540252800000, 6512.589059298922], [1540339200000, 6472.799660641591], [1540425600000, 6481.443589558082], [1540512000000, 6458.191521836801], [1540598400000, 6468.826424965618], [1540684800000, 6457.058635186671], [1540771200000, 6481.323141667608], [1540857600000, 6292.6479239344], [1540944000000, 6331.781807909807], [1541030400000, 6366.720109554257], [1541116800000, 6379.972972168429], [1541203200000, 6425.4415830402095], [1541289600000, 6388.1897883073725], [1541376000000, 6351.061941056285], [1541462400000, 6415.443408541094], [1541548800000, 6474.847290336688], [1541635200000, 6574.801180290013], [1541721600000, 6490.454730761757], [1541808000000, 6413.78203857173], [1541894400000, 6429.898792074219], [1541980800000, 6434.2845969255495], [1542067200000, 6411.353436205148], [1542153600000, 6394.289276264062], [1542240000000, 5788.612494046974], [1542326400000, 5686.541131058274], [1542412800000, 5609.956182334204], [1542499200000, 5599.1915956893035], [1542585600000, 5638.861481452398], [1542672000000, 4889.197859239111], [1542758400000, 4450.711454417822], [1542844800000, 4613.44841218385], [1542931200000, 4307.388731717267], [1543017600000, 4376.904946801552], [1543104000000, 3895.8368129589876], [1543190400000, 4039.836551137432], [1543276800000, 3811.9235292664935], [1543363200000, 3832.92451590069], [1543449600000, 4267.243628990657], [1543536000000, 4279.475592548712], [1543622400000, 4009.6895798259993], [1543708800000, 4189.573822999859], [1543795200000, 4124.593548643215], [1543881600000, 3878.8392347832714], [1543968000000, 3943.6928120656007], [1544054400000, 3751.691966395661], [1544140800000, 3466.763863771732], [1544227200000, 3408.5236170431267], [1544313600000, 3424.7058606626006], [1544400000000, 3550.546433896889], [1544486400000, 3433.8084404312785], [1544572800000, 3378.808778413106], [1544659200000, 3438.1000735289817], [1544745600000, 3294.636156545267], [1544832000000, 3216.6271499079407], [1544918400000, 3221.9762037129963], [1545004800000, 3217.0468349697426], [1545091200000, 3506.2170132794818], [1545177600000, 3655.775045729801], [1545264000000, 3687.750553192723], [1545350400000, 4065.097238010126], [1545436800000, 3846.2274979885774], [1545523200000, 3960.567773408038], [1545609600000, 3941.617740158326], [1545696000000, 4021.990255345401], [1545782400000, 3759.1784268965694], [1545868800000, 3781.613081593353], [1545955200000, 3587.703880138819], [1546041600000, 3854.4210969825917], [1546128000000, 3778.193939889601], [1546214400000, 3809.7282286410905], [1546300800000, 3692.5315655246986], [1546387200000, 3794.2642537397164], [1546473600000, 3872.1721842228935], [1546560000000, 3776.525807694106], [1546646400000, 3815.268079882635], [1546732800000, 3787.3078571242845], [1546819200000, 4013.5200912973864], [1546905600000, 3989.6687175387638], [1546992000000, 3973.5912546157865], [1547078400000, 3982.2259127624075], [1547164800000, 3623.8477291056797], [1547251200000, 3632.849306434781], [1547337600000, 3598.4363465618803], [1547424000000, 3495.832311307438], [1547510400000, 3648.04828565649], [1547596800000, 3567.801243199206], [1547683200000, 3597.6473233295915], [1547769600000, 3620.9812811457246], [1547856000000, 3597.7932517701506], [1547942400000, 3674.587270857304], [1548028800000, 3539.167531041029], [1548115200000, 3531.004182799933], [1548201600000, 3571.2669080928695], [1548288000000, 3555.7000025226243], [1548374400000, 3565.6420046170365], [1548460800000, 3564.822417111422], [1548547200000, 3563.614563419144], [1548633600000, 3553.121681129778], [1548720000000, 3432.215668301249], [1548806400000, 3413.3754832497975], [1548892800000, 3457.8826491759864], [1548979200000, 3431.943637864456], [1549065600000, 3464.490147562154], [1549152000000, 3486.581616258029], [1549238400000, 3454.025843825187], [1549324800000, 3464.003508861087], [1549411200000, 3469.8946770792636], [1549497600000, 3405.836861256719], [1549584000000, 3394.0146529587223], [1549670400000, 3664.2821026589836], [1549756800000, 3667.4082642899566], [1549843200000, 3675.6916291247258], [1549929600000, 3631.444539729949], [1550016000000, 3633.9650467461183], [1550102400000, 3610.0622730634022], [1550188800000, 3589.661829626318], [1550275200000, 3601.2291913968643], [1550361600000, 3622.5916530750715], [1550448000000, 3665.075530800723], [1550534400000, 3886.5202004751386], [1550620800000, 3915.0099712770552], [1550707200000, 3976.808906528991], [1550793600000, 3934.813593894817], [1550880000000, 3973.6413227746357], [1550966400000, 4115.3379267876635], [1551052800000, 3742.868946700387], [1551139200000, 3824.683504217102], [1551225600000, 3809.209774682024], [1551312000000, 3813.377891881294], [1551398400000, 3815.6076075249844], [1551484800000, 3815.753939922755], [1551571200000, 3819.645730582585], [1551657600000, 3806.9542737979564], [1551744000000, 3712.812457297799], [1551830400000, 3852.461665785486], [1551916800000, 3860.510762380684], [1552003200000, 3871.637247965991], [1552089600000, 3861.879307396147], [1552176000000, 3941.600873271771], [1552262400000, 3923.559788806432], [1552348800000, 3860.3592300287123], [1552435200000, 3876.480852754649], [1552521600000, 3865.817782663389], [1552608000000, 3876.6987080329527], [1552694400000, 3925.460579704576], [1552780800000, 4007.709463818873], [1552867200000, 3980.9901047320204], [1552953600000, 3986.7068940461468], [1553040000000, 4013.5769693454813], [1553126400000, 4045.57263350974], [1553212800000, 3983.317781297362], [1553299200000, 3989.1584912154526], [1553385600000, 4008.945386314754], [1553472000000, 3993.394900459549], [1553558400000, 3927.577296521434], [1553644800000, 3938.659743156943], [1553731200000, 4036.1609179207603], [1553817600000, 4028.3913011796208], [1553904000000, 4103.322859513963], [1553990400000, 4103.856421333923], [1554076800000, 4103.521738549332], [1554163200000, 4146.321927706636], [1554249600000, 4862.230427766917], [1554336000000, 4955.831824018296], [1554422400000, 4899.021553783287], [1554508800000, 5014.955826888765], [1554595200000, 5041.869075888309], [1554681600000, 5176.97788362072], [1554768000000, 5266.89477647303], [1554854400000, 5172.1752647597095], [1554940800000, 5308.905683164441], [1555027200000, 5047.79316241402], [1555113600000, 5082.46515242293], [1555200000000, 5072.987191337883], [1555286400000, 5149.314492798127], [1555372800000, 5039.881017404402], [1555459200000, 5213.598376962082], [1555545600000, 5218.957150384528], [1555632000000, 5274.767013336503], [1555718400000, 5280.354444383007], [1555804800000, 5307.689183437135], [1555891200000, 5311.164840299274], [1555977600000, 5380.935789575588], [1556064000000, 5557.9257276108365], [1556150400000, 5452.101753394462], [1556236800000, 5203.088598730703], [1556323200000, 5295.847320048228], [1556409600000, 5256.977802474059], [1556496000000, 5257.803249232503], [1556582400000, 5199.755008563461], [1556668800000, 5292.803974562054], [1556755200000, 5354.5868001739655], [1556841600000, 5450.706620486424], [1556928000000, 5731.493099606555], [1557014400000, 5803.943309048437], [1557100800000, 5749.323780460327], [1557187200000, 5715.7571359661015], [1557273600000, 5841.053771273751], [1557360000000, 5957.856732858399], [1557446400000, 6168.268996398258], [1557532800000, 6370.233758607852], [1557619200000, 7258.261280655222], [1557705600000, 6953.75138858638], [1557792000000, 7805.98164948938], [1557878400000, 7990.055553534356], [1557964800000, 8192.223138952013], [1558051200000, 7875.913199526066], [1558137600000, 7343.371457162994], [1558224000000, 7300.655158341227], [1558310400000, 8168.730689783752], [1558396800000, 7976.851712899625], [1558483200000, 7958.365526075088], [1558569600000, 7665.79685383355], [1558656000000, 7861.812792465447], [1558742400000, 7977.244882467973], [1558828800000, 8037.627431860584], [1558915200000, 8631.080577844017], [1559001600000, 8816.03391493351], [1559088000000, 8726.978110456395], [1559174400000, 8650.677405968292], [1559260800000, 8310.891063057417], [1559347200000, 8575.646353354166], [1559433600000, 8554.261728092017], [1559520000000, 8743.705469134235], [1559606400000, 8173.632761796738], [1559692800000, 7683.636261419786], [1559779200000, 7812.44885473215], [1559865600000, 7821.123897094859], [1559952000000, 8036.108159122215], [1560038400000, 7942.539284064289], [1560124800000, 7668.029627860183], [1560211200000, 8002.729982894047], [1560297600000, 7897.975476598364], [1560384000000, 8145.05909812096], [1560470400000, 8227.625725590904], [1560556800000, 8671.505388660793], [1560643200000, 8818.609506932296], [1560729600000, 8978.77258780693], [1560816000000, 9312.868294808708], [1560902400000, 9094.021746079217], [1560988800000, 9285.928674251472], [1561075200000, 9548.31456526146], [1561161600000, 10118.359455626854], [1561248000000, 10704.922148115673], [1561334400000, 10846.12509964766], [1561420800000, 11045.550419814825], [1561507200000, 11808.419904075861], [1561593600000, 12996.124342788677], [1561680000000, 11178.682998186501], [1561766400000, 12352.19686591269], [1561852800000, 11972.555667220913], [1561939200000, 10888.097181679726], [1562025600000, 10639.046292232071], [1562112000000, 10822.764178376405], [1562198400000, 11975.637393818817], [1562284800000, 11229.60561912491], [1562371200000, 10987.248856214646], [1562457600000, 11279.476377672418], [1562544000000, 11436.430746923896], [1562630400000, 12260.28269787262], [1562716800000, 12544.167261709219], [1562803200000, 12166.398288868606], [1562889600000, 11385.060490564521], [1562976000000, 11804.562793905448], [1563062400000, 11418.511855491488], [1563148800000, 10285.461109066944], [1563235200000, 10911.345158309634], [1563321600000, 9538.827124243868], [1563408000000, 9657.303101319074], [1563494400000, 10636.155728360647], [1563580800000, 10536.565849576118], [1563667200000, 10774.281137633849], [1563753600000, 10607.463560702123], [1563840000000, 10343.318139598357], [1563926400000, 9936.380762347033], [1564012800000, 9799.306312967936], [1564099200000, 9878.23193291115], [1564185600000, 9845.999447078504], [1564272000000, 9447.678401413306], [1564358400000, 9534.755426833233], [1564444800000, 9510.593552181648], [1564531200000, 9568.437452164315], [1564617600000, 10062.996285663326], [1564704000000, 10361.312000080692], [1564790400000, 10529.171907427639], [1564876800000, 10791.200523159], [1564963200000, 10951.593834840049], [1565049600000, 11818.525531066103], [1565136000000, 11510.164154730011], [1565222400000, 11970.657556451994], [1565308800000, 12024.331570998236], [1565395200000, 11852.561606995518], [1565481600000, 11327.842642233223], [1565568000000, 11549.998354322472], [1565654400000, 11381.241355678554], [1565740800000, 10903.6549918801], [1565827200000, 10016.217389275296], [1565913600000, 10295.87714290961], [1566000000000, 10324.993597415598], [1566086400000, 10224.218706633936], [1566172800000, 10309.872435928555], [1566259200000, 10895.97184442924], [1566345600000, 10743.789313302059], [1566432000000, 10124.058214168905], [1566518400000, 10127.78985767835], [1566604800000, 10385.509090033882], [1566691200000, 10145.01247970698], [1566777600000, 10090.061576030994], [1566864000000, 10334.911854471331], [1566950400000, 10167.889390020262], [1567036800000, 9753.48704889705], [1567123200000, 9485.141423780755], [1567209600000, 9588.98750205815], [1567296000000, 9607.289264302583], [1567382400000, 9729.266231867814], [1567468800000, 10323.022660378076], [1567555200000, 10620.165189356907], [1567641600000, 10573.515179027892], [1567728000000, 10570.249940786205], [1567814400000, 10350.748325717243], [1567900800000, 10461.547731202416], [1567987200000, 10404.333900445994], [1568073600000, 10321.931613525778], [1568160000000, 10121.102297214318], [1568246400000, 10150.71002363705], [1568332800000, 10401.319919342226], [1568419200000, 10355.626898620032], [1568505600000, 10349.551958231765], [1568592000000, 10305.675020156998], [1568678400000, 10268.421696882157], [1568764800000, 10203.071046816189], [1568851200000, 10158.854517376085], [1568937600000, 10253.232374680329], [1569024000000, 10178.442572698175], [1569110400000, 9997.199887816778], [1569196800000, 10041.304200624943], [1569283200000, 9699.504938095957], [1569369600000, 8702.713528026236], [1569456000000, 8446.244115548528], [1569542400000, 8098.80982048931], [1569628800000, 8183.482347567026], [1569715200000, 8225.11037185192], [1569801600000, 8064.380983385267], [1569888000000, 8271.290855465355], [1569974400000, 8366.503905663569], [1570060800000, 8350.132163242944], [1570147200000, 8176.354679406582], [1570233600000, 8156.519553840962], [1570320000000, 8140.242484268429], [1570406400000, 7890.234349864658], [1570492800000, 8182.251593084191], [1570579200000, 8180.69113018462], [1570665600000, 8573.060086448735], [1570752000000, 8561.51932921772], [1570838400000, 8277.848351085042], [1570924800000, 8308.470967433434], [1571011200000, 8289.497236618896], [1571097600000, 8347.350350085451], [1571184000000, 8175.8631121051185], [1571270400000, 8010.3021797941665], [1571356800000, 8079.931459479744], [1571443200000, 7957.314093995798], [1571529600000, 7949.802268609832], [1571616000000, 8208.362621758712], [1571702400000, 8220.91853113097], [1571788800000, 8045.294246807262], [1571875200000, 7468.5325324369], [1571961600000, 7433.915345679171], [1572048000000, 8652.030029867461], [1572134400000, 9217.236031536286], [1572220800000, 9502.020114436129], [1572307200000, 9239.897283779665], [1572393600000, 9407.163351459427], [1572480000000, 9171.874474054188], [1572566400000, 9151.824594459767], [1572652800000, 9233.413801605904], [1572739200000, 9302.83461784109], [1572825600000, 9204.286350451268], [1572912000000, 9378.953914330921], [1572998400000, 9300.373963047077], [1573084800000, 9340.048651574896], [1573171200000, 9221.073784983802], [1573257600000, 8795.896069400225], [1573344000000, 8809.78367388588], [1573430400000, 9043.673292397065], [1573516800000, 8729.917418490728], [1573603200000, 8806.751998954014], [1573689600000, 8770.78082523136], [1573776000000, 8652.025288272329], [1573862400000, 8480.435541874604], [1573948800000, 8496.376959145864], [1574035200000, 8508.499434051764], [1574121600000, 8202.834446107912], [1574208000000, 8138.831605359057], [1574294400000, 8095.865638011639], [1574380800000, 7632.494765498649], [1574467200000, 7293.013949172297], [1574553600000, 7309.689243566324], [1574640000000, 6935.73862740252], [1574726400000, 7118.264883393205], [1574812800000, 7157.684731054473], [1574899200000, 7483.754192176799], [1574985600000, 7438.621769308849], [1575072000000, 7720.865011260837], [1575158400000, 7556.397585815771], [1575244800000, 7405.808478752477], [1575331200000, 7308.982444423794], [1575417600000, 7286.080951511441], [1575504000000, 7209.971417454778], [1575590400000, 7373.782498577651], [1575676800000, 7530.922830404948], [1575763200000, 7493.545154556748], [1575849600000, 7508.315669139108], [1575936000000, 7347.036951245358], [1576022400000, 7233.950487171398], [1576108800000, 7197.426825880834], [1576195200000, 7200.7070180298015], [1576281600000, 7247.350277010915], [1576368000000, 7081.740255265727], [1576454400000, 7108.563021778046], [1576540800000, 6901.036915336086], [1576627200000, 6627.162514553135], [1576713600000, 7274.768503792503], [1576800000000, 7152.558691632884], [1576886400000, 7185.439179630756], [1576972800000, 7141.554857046259], [1577059200000, 7488.196174601466], [1577145600000, 7335.207616177015], [1577232000000, 7266.60819662888], [1577318400000, 7204.1848756885975], [1577404800000, 7208.489307892152], [1577491200000, 7247.62371236525], [1577577600000, 7307.887522683333], [1577664000000, 7394.029285530991], [1577750400000, 7240.4289122518185], [1577836800000, 7195.153895430029], [1577923200000, 7193.7546679601], [1578009600000, 6963.853353983485], [1578096000000, 7300.34693530511], [1578182400000, 7351.377053166627], [1578268800000, 7349.972248050628], [1578355200000, 7734.675515785289], [1578441600000, 8147.415237636371], [1578528000000, 8045.789124592971], [1578614400000, 7832.795643840387], [1578700800000, 8180.513169803805], [1578787200000, 8012.085781814529], [1578873600000, 8171.401185320944], [1578960000000, 8110.647964503772], [1579046400000, 8801.871213137745], [1579132800000, 8820.85010599613], [1579219200000, 8720.91950137424], [1579305600000, 8902.568995225329], [1579392000000, 8906.902675151074], [1579478400000, 8690.380569544623], [1579564800000, 8631.253954333763], [1579651200000, 8724.818622931474], [1579737600000, 8665.436914096897], [1579824000000, 8415.956027335538], [1579910400000, 8432.096675789418], [1579996800000, 8348.665665079325], [1580083200000, 8595.728298493115], [1580169600000, 8879.335548714553], [1580256000000, 9320.805853868467], [1580342400000, 9276.893821086494], [1580428800000, 9509.80513557522], [1580515200000, 9308.060285564778], [1580601600000, 9363.22420680057], [1580688000000, 9338.416631939404], [1580774400000, 9308.91270229121], [1580860800000, 9179.95928107711], [1580947200000, 9592.18387389791], [1581033600000, 9744.636578159387], [1581120000000, 9799.448455824684], [1581206400000, 9890.701658194865], [1581292800000, 10148.379719131783], [1581379200000, 9890.121224019967], [1581465600000, 10226.925769215322], [1581552000000, 10328.897565836414], [1581638400000, 10221.260309201634], [1581724800000, 10320.157140513991], [1581811200000, 9894.136123789158], [1581897600000, 9943.673602578743], [1581984000000, 9723.605026422496], [1582070400000, 10133.817417084678], [1582156800000, 9618.409248250406], [1582243200000, 9608.16918128246], [1582329600000, 9673.011800270346], [1582416000000, 9658.606058375473], [1582502400000, 9946.716123467522], [1582588800000, 9670.9608064677], [1582675200000, 9345.342701734848], [1582761600000, 8801.657129472504], [1582848000000, 8781.175282202359], [1582934400000, 8717.331431359424], [1583020800000, 8552.989118581636], [1583107200000, 8567.802248679225], [1583193600000, 8905.876104262194], [1583280000000, 8756.718576742549], [1583366400000, 8758.646993191916], [1583452800000, 9038.870323233425], [1583539200000, 9135.84506603321], [1583625600000, 8902.20195010789], [1583712000000, 8041.365538071835], [1583798400000, 7921.33200691072], [1583884800000, 7906.731528510632], [1583971200000, 7935.522040170545], [1584057600000, 5142.990459018316], [1584144000000, 5542.819542373153], [1584230400000, 5214.189112383918], [1584316800000, 5397.93335743919], [1584403200000, 5032.501351487721], [1584489600000, 5389.41577503409], [1584576000000, 5376.280747845438], [1584662400000, 6170.200600237675], [1584748800000, 6195.603505257629], [1584835200000, 6145.770891252399], [1584921600000, 5859.647430299388], [1585008000000, 6456.347485963419], [1585094400000, 6730.173782371188], [1585180800000, 6695.9006183977235], [1585267200000, 6765.56207892991], [1585353600000, 6397.826328325604], [1585440000000, 6255.000398693634], [1585526400000, 5915.337154791903], [1585612800000, 6403.141235565223], [1585699200000, 6421.70541388854], [1585785600000, 6640.797666310131], [1585872000000, 6807.897017890566], [1585958400000, 6732.852018394612], [1586044800000, 6859.424923721944], [1586131200000, 6788.048272605917], [1586217600000, 7297.635558289496], [1586304000000, 7196.782202442051], [1586390400000, 7342.291601148024], [1586476800000, 7294.488875121554], [1586563200000, 6864.694257006497], [1586649600000, 6878.781212589853], [1586736000000, 6913.158787469097], [1586822400000, 6857.538537511484], [1586908800000, 6860.17853570111], [1586995200000, 6629.431738031291], [1587081600000, 7059.92622475854], [1587168000000, 7035.261503989225], [1587254400000, 7242.5109294929825], [1587340800000, 7127.511949689152], [1587427200000, 6856.456278354705], [1587513600000, 6842.038597634602], [1587600000000, 7109.995291181778], [1587686400000, 7382.793144116689], [1587772800000, 7495.393587498606], [1587859200000, 7538.557687279841], [1587945600000, 7683.867415083342], [1588032000000, 7774.281554448049], [1588118400000, 7758.230255185947], [1588204800000, 8744.430287016561], [1588291200000, 8610.63580374089], [1588377600000, 8824.818413551968], [1588464000000, 8966.307014689282], [1588550400000, 8888.671912686868], [1588636800000, 8884.407813577056], [1588723200000, 9003.240557621584], [1588809600000, 9144.68703972007], [1588896000000, 9959.166416261767], [1588982400000, 9821.81131529702], [1589068800000, 9566.777187205966], [1589155200000, 8752.617087745832], [1589241600000, 8604.75159101983], [1589328000000, 8788.466749414652], [1589414400000, 9283.08601265873], [1589500800000, 9796.494527024528], [1589587200000, 9309.29535940684], [1589673600000, 9375.29710843331], [1589760000000, 9666.32719340344], [1589846400000, 9708.439858793108], [1589932800000, 9760.198937162193], [1590019200000, 9526.50759300584], [1590105600000, 9059.962506871727], [1590192000000, 9131.767275081993], [1590278400000, 9170.361063506127], [1590364800000, 8731.848525870651], [1590451200000, 8883.691769863415], [1590537600000, 8839.130663273247], [1590624000000, 9174.118563996424], [1590710400000, 9546.04563503715], [1590796800000, 9427.120373393418], [1590883200000, 9662.70587254818], [1590969600000, 9466.961781429516], [1591056000000, 10167.93069332851], [1591142400000, 9515.243858655718], [1591228800000, 9645.227869360308], [1591315200000, 9776.20299178848], [1591401600000, 9636.965527050057], [1591488000000, 9662.858709002241], [1591574400000, 9738.603356828593], [1591660800000, 9773.02951309516], [1591747200000, 9767.00531665552], [1591833600000, 9874.898681832236], [1591920000000, 9325.996856202635], [1592006400000, 9469.533297509908], [1592092800000, 9469.473456163696], [1592179200000, 9345.960907722063], [1592265600000, 9431.719262201745], [1592352000000, 9524.92661691022], [1592438400000, 9463.361414311787], [1592524800000, 9399.767217129216], [1592611200000, 9312.780104497786], [1592697600000, 9360.247968201687], [1592784000000, 9298.360829121417], [1592870400000, 9678.683208975835], [1592956800000, 9624.684291831398], [1593043200000, 9288.061774486938], [1593129600000, 9258.667161007706], [1593216000000, 9166.486360416233], [1593302400000, 9013.90556467614], [1593388800000, 9139.903276297824], [1593475200000, 9185.166540651147], [1593561600000, 9149.721996758017], [1593648000000, 9230.672998590804], [1593734400000, 9094.318072166905], [1593820800000, 9071.3850427828], [1593907200000, 9132.908369533492], [1593993600000, 9087.407312582163], [1594080000000, 9342.376492626678], [1594166400000, 9253.630980242333], [1594252800000, 9432.172515827939], [1594339200000, 9235.716302064242], [1594425600000, 9282.913638839902], [1594512000000, 9234.314674712627], [1594598400000, 9297.479635872663], [1594684800000, 9240.76251972468], [1594771200000, 9247.060695963813], [1594857600000, 9203.371435179699], [1594944000000, 9136.483376363976], [1595030400000, 9156.276583115488], [1595116800000, 9168.402736564132], [1595203200000, 9202.615839500108], [1595289600000, 9163.159654576915], [1595376000000, 9384.379751903267], [1595462400000, 9514.304987626969], [1595548800000, 9589.81771944117], [1595635200000, 9535.93879573746], [1595721600000, 9691.825138917147], [1595808000000, 9925.751397476346], [1595894400000, 10962.258481207355], [1595980800000, 10904.916526918994], [1596067200000, 11093.612240442404], [1596153600000, 11116.307163685275], [1596240000000, 11325.5515272739], [1596326400000, 11812.094307268515], [1596412800000, 11066.306240590267], [1596499200000, 11230.907762749297], [1596585600000, 11181.917508034885], [1596672000000, 11719.26352395155], [1596758400000, 11768.127742240009], [1596844800000, 11571.487980683192], [1596931200000, 11739.131006414418], [1597017600000, 11682.851469154939], [1597104000000, 11862.938012702563], [1597190400000, 11398.671060896633], [1597276800000, 11579.867951602135], [1597363200000, 11817.164038803397], [1597449600000, 11777.391322489924], [1597536000000, 11864.905810156475], [1597622400000, 11901.776488302461], [1597708800000, 12272.465808160425], [1597795200000, 11949.610970628193], [1597881600000, 11733.278970862082], [1597968000000, 11861.83657727968], [1598054400000, 11515.124298729217], [1598140800000, 11676.385305081287], [1598227200000, 11647.928120934363], [1598313600000, 11758.828120368864], [1598400000000, 11350.753473213], [1598486400000, 11465.002564032086], [1598572800000, 11300.398363810944], [1598659200000, 11519.118388160729], [1598745600000, 11481.481823317012], [1598832000000, 11701.004008657852], [1598918400000, 11672.324104943627], [1599004800000, 11895.225345345636], [1599091200000, 11418.254756916149], [1599177600000, 10197.459822768922], [1599264000000, 10484.470392265588], [1599350400000, 10177.789718049991], [1599436800000, 10260.0177277544], [1599523200000, 10359.445216989981], [1599609600000, 10125.014956069688], [1599696000000, 10230.154699360752], [1599782400000, 10342.159391205681], [1599868800000, 10378.223044584596], [1599955200000, 10439.38467226404], [1600041600000, 10328.866065987393], [1600128000000, 10661.096235144483], [1600214400000, 10787.58020807624], [1600300800000, 10952.249969107099], [1600387200000, 10937.996396960929], [1600473600000, 10927.150310293275], [1600560000000, 11083.99836119821], [1600646400000, 10923.32666818137], [1600732800000, 10439.522488792423], [1600819200000, 10527.58784673943], [1600905600000, 10223.784106392732], [1600992000000, 10726.530083321983], [1601078400000, 10681.993848113701], [1601164800000, 10743.19287975935], [1601251200000, 10765.300521660893], [1601337600000, 10672.02854425897], [1601424000000, 10837.516586513004], [1601510400000, 10770.881347466415], [1601596800000, 10627.976063963179], [1601683200000, 10572.674843011133], [1601769600000, 10546.65625537982], [1601856000000, 10670.700811335208], [1601942400000, 10784.94940309411], [1602028800000, 10605.95779130622], [1602115200000, 10668.011010796896], [1602201600000, 10888.666546835811], [1602288000000, 11063.2523862591], [1602374400000, 11286.541942043945], [1602460800000, 11377.814876756858], [1602547200000, 11543.183760771468], [1602633600000, 11426.105544220427], [1602720000000, 11435.57243757972], [1602806400000, 11495.584995733887], [1602892800000, 11319.097785677623], [1602979200000, 11365.814131026178], [1603065600000, 11495.899975689872], [1603152000000, 11752.146863919786], [1603238400000, 11908.708923246453], [1603324800000, 12806.435726727443], [1603411200000, 12951.044060038126], [1603497600000, 12928.072296348406], [1603584000000, 13106.04601925958], [1603670400000, 13021.410842124118], [1603756800000, 13060.79221208406], [1603843200000, 13655.185415883081], [1603929600000, 13282.999348292438], [1604016000000, 13441.878496031612], [1604102400000, 13537.174271524887], [1604188800000, 13778.637638352931], [1604275200000, 13720.361465146289], [1604361600000, 13558.361795524168], [1604448000000, 13989.976358180447], [1604534400000, 14101.716050813722], [1604620800000, 15553.331701081443], [1604707200000, 15548.312145063144], [1604793600000, 14818.45891122149], [1604880000000, 15496.316149147417], [1604966400000, 15335.342198178862], [1605052800000, 15278.70965979927], [1605139200000, 15686.905986213233], [1605225600000, 16265.149252483357], [1605312000000, 16326.81487507908], [1605398400000, 16095.561510630652], [1605484800000, 15984.673784156614], [1605571200000, 16714.40386322188], [1605657600000, 17651.82042768171], [1605744000000, 17829.934958345126], [1605830400000, 17819.75743096837], [1605916800000, 18628.813489036198], [1606003200000, 18689.528655825852], [1606089600000, 18390.61184379203], [1606176000000, 18360.693300385865], [1606262400000, 19091.524181728702], [1606348800000, 18753.288921684914], [1606435200000, 17138.029512395206], [1606521600000, 17140.27555252362], [1606608000000, 17718.972832629497], [1606694400000, 18169.95420978285], [1606780800000, 19609.52143957559], [1606867200000, 18857.417371118165], [1606953600000, 19208.398191994624], [1607040000000, 19435.20484740911], [1607126400000, 18711.226549886927], [1607212800000, 19139.111575237486], [1607299200000, 19325.55411859109], [1607385600000, 19198.806493935343], [1607472000000, 18336.514581297808], [1607558400000, 18564.986455688137], [1607644800000, 18262.058384653155], [1607731200000, 18058.027339569555], [1607817600000, 18807.09540322016], [1607904000000, 19151.13442340493], [1607990400000, 19259.97047153096], [1608076800000, 19432.362456862407], [1608163200000, 21317.66213678684], [1608249600000, 22805.65580598891], [1608336000000, 23120.57055156752], [1608422400000, 23863.85484171969], [1608508800000, 23518.31842054723], [1608595200000, 22840.991707188354], [1608681600000, 23794.79330262859], [1608768000000, 23308.12723217715], [1608854400000, 23760.287048542956], [1608940800000, 24671.107714415415], [1609027200000, 26476.130137019492], [1609113600000, 26423.22879190229], [1609200000000, 27125.384120651503], [1609286400000, 27424.538954606684], [1609372800000, 28837.28852895899], [1609459200000, 29022.41839530417], [1609545600000, 29352.12679194895], [1609632000000, 32163.824935335215], [1609718400000, 33008.226203489285], [1609804800000, 31515.575966658354], [1609891200000, 34082.20604904424], [1609977600000, 36933.520137309126], [1610064000000, 39547.08408135567], [1610150400000, 40815.96185478254], [1610236800000, 40296.5290038294], [1610323200000, 38397.895985418174], [1610409600000, 35669.90668663349], [1610496000000, 33938.33587810088], [1610582400000, 37456.00772548642], [1610668800000, 39232.74997140346], [1610755200000, 36787.52121668496], [1610841600000, 36254.713079410845], [1610928000000, 35804.263885674685], [1611014400000, 36595.4647692826], [1611100800000, 36104.86385775514], [1611187200000, 35587.489202735545], [1611273600000, 30913.695736053804], [1611360000000, 32957.908782655926], [1611446400000, 32068.08737440817], [1611532800000, 32273.51735032927], [1611619200000, 32375.320062981456], [1611705600000, 32582.10944502123], [1611792000000, 30445.52586511351], [1611878400000, 33128.34752861149], [1611964800000, 34150.81876737221], [1612051200000, 34199.51981036302], [1612137600000, 33064.78676701507], [1612224000000, 33405.99035714327], [1612310400000, 35485.98593382442], [1612396800000, 37494.71762460426], [1612483200000, 36816.50808203406], [1612569600000, 38007.83222908396], [1612656000000, 39279.41286897885], [1612742400000, 38833.34026455532], [1612828800000, 46307.57412159395], [1612915200000, 46569.56483], [1613001600000, 44848.69344950499], [1613088000000, 47815.96175554316], [1613174400000, 47414.1862550079], [1613260800000, 46941.29208582354], [1613347200000, 48607.8745224845], [1613433600000, 47898.487027633135], [1613520000000, 49238.13690672487], [1613606400000, 52143.6788446198], [1613692800000, 51733.075539032034], [1613779200000, 56038.72775892988], [1613865600000, 56377.63347790987], [1613952000000, 57669.3035269179], [1614038400000, 54410.85629451601], [1614124800000, 48691.894831704405], [1614211200000, 49849.378713583144], [1614297600000, 46992.665450048495], [1614384000000, 46551.48747675472], [1614470400000, 46653.52639939581], [1614556800000, 44970.163459946736], [1614643200000, 49787.33995949672], [1614729600000, 48532.23678981703], [1614816000000, 50577.45746915202], [1614902400000, 48727.45120080115], [1614988800000, 49091.81488232977], [1615075200000, 49019.36859236582], [1615161600000, 51313.09262027362], [1615248000000, 52328.5359126325], [1615334400000, 54700.273439277444], [1615420800000, 56020.4877195857], [1615507200000, 57788.86810329514], [1615593600000, 57353.86147197248], [1615680000000, 61315.197932766816], [1615766400000, 59428.967566027764], [1615852800000, 55805.32924923426], [1615939200000, 56830.253335076224], [1616025600000, 59014.92563205099], [1616112000000, 57922.40959262911], [1616198400000, 58243.27060892363], [1616284800000, 58376.15847137206], [1616371200000, 57573.539344778685], [1616457600000, 54370.137557895185], [1616544000000, 54584.86994699455], [1616630400000, 52526.86659704657], [1616716800000, 51416.908304980054], [1616803200000, 55033.100480447036], [1616889600000, 55832.418109618324], [1616976000000, 55728.10132780653], [1617062400000, 57634.92869447539], [1617148800000, 58668.628336761], [1617235200000, 58817.3738884188], [1617321600000, 58801.194843878126], [1617408000000, 59060.02678333801], [1617494400000, 57060.40665471088], [1617580800000, 58228.802623466196], [1617667200000, 58706.81559311396], [1617753600000, 58062.17872433649], [1617840000000, 56134.41738866435], [1617926400000, 58065.644024815316], [1618012800000, 58152.993262141834], [1618099200000, 59979.39281571831], [1618185600000, 59988.02095852983], [1618272000000, 59911.020594847316], [1618358400000, 63576.676041048275], [1618444800000, 62807.12323259299], [1618531200000, 63179.772446084695], [1618617600000, 61497.299569441195], [1618704000000, 60273.86667694734], [1618790400000, 56288.94534001195], [1618876800000, 55721.166052695626], [1618963200000, 56294.728558586925], [1619049600000, 54190.07026283405], [1619136000000, 51965.5664304852], [1619222400000, 51191.1246964079], [1619308800000, 50133.1073963654], [1619395200000, 48981.4413932161], [1619481600000, 53978.80554728962], [1619568000000, 54991.81484262107], [1619654400000, 54810.6138506792], [1619740800000, 53596.70402266675], [1619827200000, 57828.511814425874], [1619913600000, 57812.96915967891], [1620000000000, 56600.74528738432], [1620086400000, 57200.30029871162], [1620172800000, 53464.37021950372], [1620259200000, 57432.100227087576], [1620345600000, 56507.759439869595], [1620432000000, 57361.56904328887], [1620518400000, 58771.881689509355], [1620604800000, 58213.926293744385], [1620691200000, 55901.66759456145], [1620777600000, 56928.97235660158], [1620864000000, 50004.76218313686], [1620950400000, 49972.795398372444], [1621036800000, 49913.26314398822], [1621123200000, 46780.902304688214], [1621209600000, 46585.06413538846], [1621296000000, 43780.82559782485], [1621382400000, 43091.04144756922], [1621468800000, 37287.39140242737], [1621555200000, 40437.73009625558], [1621641600000, 37339.63420134628], [1621728000000, 37520.31767547438], [1621814400000, 34977.527801962424], [1621900800000, 38642.4422973396], [1621987200000, 38210.80521759389], [1622073600000, 39406.934244116186], [1622160000000, 38642.89555434217], [1622246400000, 35710.220936529884], [1622332800000, 34775.07941347281], [1622419200000, 35714.75202018067], [1622505600000, 37340.6792659422], [1622592000000, 36680.06820180283], [1622678400000, 37685.71798174826], [1622764800000, 39151.316183800314], [1622851200000, 36938.72031139185], [1622937600000, 35546.98596483364], [1623024000000, 35834.4744744042], [1623110400000, 33451.38340714534], [1623196800000, 33557.15068309184], [1623283200000, 37553.64375851888], [1623369600000, 36903.304906778896], [1623456000000, 37289.43252210222], [1623542400000, 35666.14977877309], [1623628800000, 39147.70567879997], [1623715200000, 40624.51305316154], [1623801600000, 40378.20254971114], [1623888000000, 38321.454656574206], [1623974400000, 38193.619141943964], [1624060800000, 35881.11137137543], [1624147200000, 35581.64226773201], [1624233600000, 35787.07740523232], [1624320000000, 31711.935467727006], [1624406400000, 32507.738707026845], [1624492800000, 33702.61779371396], [1624579200000, 34675.44910530336], [1624665600000, 31943.136467863093], [1624752000000, 32023.973901774916], [1624838400000, 34607.293274083386], [1624924800000, 34556.8421269449], [1625011200000, 35968.993667970346], [1625097600000, 35171.486304850056], [1625184000000, 33677.566235922626], [1625270400000, 33950.786555181], [1625356800000, 34729.89077801021], [1625443200000, 35394.213074080915], [1625529600000, 33928.13421837147], [1625616000000, 34149.98981477412], [1625702400000, 33932.25463831538], [1625788800000, 32933.578199292395], [1625875200000, 33971.297749757956], [1625961600000, 33704.53847440508], [1626048000000, 34299.938016517444], [1626134400000, 33262.6543524462], [1626220800000, 32676.38380096018], [1626307200000, 32878.50187357147], [1626393600000, 31726.038833909817], [1626480000000, 31399.653128682126], [1626566400000, 31588.6713510167], [1626652800000, 31919.393922554555], [1626739200000, 30928.211908849244], [1626825600000, 29971.903783612062], [1626912000000, 32383.78112696872], [1626998400000, 32409.499767175417], [1627084800000, 33455.55127272261], [1627171200000, 34213.59993217662], [1627257600000, 35456.124660766836], [1627344000000, 37281.91809217662], [1627430400000, 39076.59377063225], [1627516800000, 40031.19824444798], [1627603200000, 39977.75693922764], [1627689600000, 41936.261478677225], [1627776000000, 41754.16660435275], [1627862400000, 39914.82975848789], [1627948800000, 39278.766508103676], [1628035200000, 38368.35401197897], [1628121600000, 39751.58457509862], [1628208000000, 40825.381940449704], [1628294400000, 42802.13717474142], [1628380800000, 44647.57760300116], [1628467200000, 43753.217862646314], [1628553600000, 46311.00189857689], [1628640000000, 45639.51401890709], [1628726400000, 45652.06012506926], [1628812800000, 44495.20979327958], [1628899200000, 47717.27052766495], [1628985600000, 47088.85313451467], [1629072000000, 47025.00323011282], [1629158400000, 46031.75754362583]]
    rawdata_array = np.asarray(rawdata)
    date = rawdata_array[...,0]
    price = rawdata_array[...,1]

    date_webshow = []
    price_webshow = []

    for x in np.nditer(date, op_flags=['readwrite']): 
        date = time.strftime("%Y-%m-%d", time.localtime(x/1000))
        print(date)
        date_webshow.append(date)

    for x in np.nditer(price, op_flags=['readwrite']): 
        price_webshow.append(np.around(x,decimals =  2))

    data = {'date':date_webshow,'price':price_webshow}
    return data
# print(get_rawdata_webshow()['date'])

In [21]:
#天数差值计算器
def diff_days(start,end):
    start_stamp = timestr_to_timestamp(start)
    end_stamp = timestr_to_timestamp(end)
    diff = end_stamp - start_stamp
    days = diff/86400
    return days


#字符串日期转成时间戳
def timestr_to_timestamp(date):
    timeArray = time.strptime(date, "%Y-%m-%d")
    return int(time.mktime(timeArray))

#将时间戳转成字符串日期
def timestamp_to_timestr(timestamp):
    timeArray = time.localtime(timestamp)
    timestr = time.strftime("%Y-%m-%d", timeArray)
    return timestr

#将utcday转成字符串
def utcday_to_timestr(utcday):
    timeArray = time.localtime((utcday-15642)*86400 + 1351468800)
    time_show = time.strftime("%Y-%m-%d", timeArray)
    return time_show

print(timestamp_to_timestr(1367107200))
print(diff_days('2012-10-29','2021-08-19'))
print(diff_days('2013-04-28','2021-08-19'))
print(utcday_to_timestr(15642))
print(utcday_to_timestr(18858))

2013-04-28
3216.0
3035.0
2012-10-29
2021-08-19


In [4]:
#获取粉丝历史数据 包括了日期和数量
def get_subscriber(name):
    #name = 'bitcoin','dogecoin'
    url = "https://subredditstats.com/api/subreddit?name=" + name
    json_str = requests.get(url).json()
    return np.asarray(json_str['subscriberCountTimeSeries'])
# get_subscriber('bitcoin')

In [70]:
def utcday_sortrule(elem):
    return elem['utcDay']
subscriber_array = '[{"utcDay":15642,"count":11274},{"utcDay":15640,"count":11292},{"utcDay":15644,"count":11301}]'
subscriber_array_json = json.loads(subscriber_array)
subscriber_array_json.sort(key=utcday_sortrule)

# print(subscriber_array_json)

[{'utcDay': 15640, 'count': 11292}, {'utcDay': 15642, 'count': 11274}, {'utcDay': 15644, 'count': 11301}]


In [23]:
#7日粉丝年化增长率  https://subredditstats.com/r/bitcoin 图表中打开chartstudio可以看到excel数据
def get_subscriber_year_incre_ratio():
    #历史粉丝数据 get_subscriber('bitcoin') btc的15642对应日期2012-10-29 08:00:00
    #缺失的日期  ['2018-09-21', '2021-01-21', '2021-01-31', '2018-03-13', '2020-12-20', '2021-01-16', '2021-01-22', '2018-03-02', '2020-12-19', '2020-09-24', '2020-09-14', '2020-12-11', '2020-12-15', '2020-12-08', '2021-01-17', '2018-02-28', '2021-02-01', '2018-03-11', '2020-09-17', '2020-09-16', '2021-01-25', '2020-12-10', '2021-01-20', '2020-09-19', '2021-01-27', '2020-10-27', '2018-03-04', '2021-01-30', '2021-01-26', '2020-09-13', '2018-09-22', '2018-09-19', '2020-09-27', '2021-01-23', '2020-09-23', '2018-03-01', '2021-02-02', '2018-09-11', '2020-09-28', '2020-09-21', '2018-09-13', '2020-10-29', '2018-03-12', '2018-03-07', '2020-09-20', '2020-12-13', '2020-11-01', '2018-03-10', '2018-09-18', '2018-09-15', '2018-09-12', '2018-03-03', '2018-03-08', '2018-09-20', '2018-09-14', '2018-09-05', '2020-11-02', '2018-09-16', '2018-09-23', '2018-03-05', '2020-09-12', '2021-01-19', '2018-09-17', '2020-10-30', '2018-03-09', '2018-03-06', '2020-12-17', '2020-09-26']
    subscriber_array = '[{"utcDay":15642,"count":11274},{"utcDay":15643,"count":11292},{"utcDay":15644,"count":11301},{"utcDay":15645,"count":11329},{"utcDay":15646,"count":11343},{"utcDay":15647,"count":11362},{"utcDay":15648,"count":11375},{"utcDay":15649,"count":11400},{"utcDay":15650,"count":11413},{"utcDay":15651,"count":11434},{"utcDay":15652,"count":11467},{"utcDay":15653,"count":11511},{"utcDay":15654,"count":11523},{"utcDay":15655,"count":11546},{"utcDay":15656,"count":11658},{"utcDay":15657,"count":11699},{"utcDay":15658,"count":11727},{"utcDay":15659,"count":11755},{"utcDay":15660,"count":11781},{"utcDay":15661,"count":11799},{"utcDay":15662,"count":11811},{"utcDay":15663,"count":11832},{"utcDay":15664,"count":11862},{"utcDay":15665,"count":11871},{"utcDay":15666,"count":11890},{"utcDay":15667,"count":11914},{"utcDay":15668,"count":11937},{"utcDay":15669,"count":11951},{"utcDay":15670,"count":11978},{"utcDay":15671,"count":12006},{"utcDay":15672,"count":12058},{"utcDay":15673,"count":12085},{"utcDay":15674,"count":12114},{"utcDay":15675,"count":12138},{"utcDay":15676,"count":12178},{"utcDay":15677,"count":12212},{"utcDay":15678,"count":12245},{"utcDay":15679,"count":12280},{"utcDay":15680,"count":12319},{"utcDay":15681,"count":12361},{"utcDay":15682,"count":12384},{"utcDay":15683,"count":12426},{"utcDay":15684,"count":12461},{"utcDay":15685,"count":12589},{"utcDay":15686,"count":12644},{"utcDay":15687,"count":12673},{"utcDay":15688,"count":12706},{"utcDay":15689,"count":12741},{"utcDay":15690,"count":12811},{"utcDay":15691,"count":12853},{"utcDay":15692,"count":12871},{"utcDay":15693,"count":12887},{"utcDay":15694,"count":12906},{"utcDay":15695,"count":12936},{"utcDay":15696,"count":12963},{"utcDay":15697,"count":12999},{"utcDay":15698,"count":13045},{"utcDay":15699,"count":13067},{"utcDay":15700,"count":13073},{"utcDay":15701,"count":13093},{"utcDay":15702,"count":13107},{"utcDay":15703,"count":13126},{"utcDay":15704,"count":13149},{"utcDay":15705,"count":13172},{"utcDay":15706,"count":13185},{"utcDay":15707,"count":13239},{"utcDay":15708,"count":13280},{"utcDay":15709,"count":13300},{"utcDay":15710,"count":13332},{"utcDay":15711,"count":13367},{"utcDay":15712,"count":13405},{"utcDay":15713,"count":13439},{"utcDay":15714,"count":13483},{"utcDay":15715,"count":13509},{"utcDay":15716,"count":13544},{"utcDay":15717,"count":13572},{"utcDay":15718,"count":13599},{"utcDay":15719,"count":13640},{"utcDay":15720,"count":13669},{"utcDay":15721,"count":13704},{"utcDay":15722,"count":13740},{"utcDay":15723,"count":13765},{"utcDay":15724,"count":13796},{"utcDay":15725,"count":13831},{"utcDay":15726,"count":13869},{"utcDay":15727,"count":13951},{"utcDay":15728,"count":14010},{"utcDay":15729,"count":14077},{"utcDay":15730,"count":14106},{"utcDay":15731,"count":14154},{"utcDay":15732,"count":14185},{"utcDay":15733,"count":14254},{"utcDay":15734,"count":14324},{"utcDay":15735,"count":14388},{"utcDay":15736,"count":14444},{"utcDay":15737,"count":14503},{"utcDay":15738,"count":14558},{"utcDay":15739,"count":14618},{"utcDay":15740,"count":14658},{"utcDay":15741,"count":14710},{"utcDay":15742,"count":14766},{"utcDay":15743,"count":14810},{"utcDay":15744,"count":14857},{"utcDay":15745,"count":14904},{"utcDay":15746,"count":14984},{"utcDay":15747,"count":15056},{"utcDay":15748,"count":15135},{"utcDay":15749,"count":15244},{"utcDay":15750,"count":15528},{"utcDay":15751,"count":15721},{"utcDay":15752,"count":15804},{"utcDay":15753,"count":15884},{"utcDay":15754,"count":15954},{"utcDay":15755,"count":16072},{"utcDay":15756,"count":16159},{"utcDay":15757,"count":16241},{"utcDay":15758,"count":16356},{"utcDay":15759,"count":16422},{"utcDay":15760,"count":16518},{"utcDay":15761,"count":16577},{"utcDay":15762,"count":16656},{"utcDay":15763,"count":16727},{"utcDay":15764,"count":16827},{"utcDay":15765,"count":16897},{"utcDay":15766,"count":16962},{"utcDay":15767,"count":17069},{"utcDay":15768,"count":17178},{"utcDay":15769,"count":17329},{"utcDay":15770,"count":17531},{"utcDay":15771,"count":17781},{"utcDay":15772,"count":17912},{"utcDay":15773,"count":18150},{"utcDay":15774,"count":18222},{"utcDay":15775,"count":18294},{"utcDay":15776,"count":18483},{"utcDay":15777,"count":18685},{"utcDay":15778,"count":18822},{"utcDay":15779,"count":18921},{"utcDay":15780,"count":19015},{"utcDay":15781,"count":19147},{"utcDay":15782,"count":19293},{"utcDay":15783,"count":19773},{"utcDay":15784,"count":19991},{"utcDay":15785,"count":20311},{"utcDay":15786,"count":20551},{"utcDay":15787,"count":20759},{"utcDay":15788,"count":21004},{"utcDay":15789,"count":21284},{"utcDay":15790,"count":21647},{"utcDay":15791,"count":21921},{"utcDay":15792,"count":22411},{"utcDay":15793,"count":22774},{"utcDay":15794,"count":23117},{"utcDay":15795,"count":23476},{"utcDay":15796,"count":23835},{"utcDay":15797,"count":24333},{"utcDay":15798,"count":25392},{"utcDay":15799,"count":25954},{"utcDay":15800,"count":26441},{"utcDay":15801,"count":26690},{"utcDay":15802,"count":27053},{"utcDay":15803,"count":27648},{"utcDay":15804,"count":29032},{"utcDay":15805,"count":31813},{"utcDay":15806,"count":33743},{"utcDay":15807,"count":34617},{"utcDay":15808,"count":35082},{"utcDay":15809,"count":35508},{"utcDay":15810,"count":35819},{"utcDay":15811,"count":36180},{"utcDay":15812,"count":36510},{"utcDay":15813,"count":36790},{"utcDay":15814,"count":36954},{"utcDay":15815,"count":37154},{"utcDay":15816,"count":37358},{"utcDay":15817,"count":37581},{"utcDay":15818,"count":37808},{"utcDay":15819,"count":38074},{"utcDay":15820,"count":38277},{"utcDay":15821,"count":38456},{"utcDay":15822,"count":38625},{"utcDay":15823,"count":38804},{"utcDay":15824,"count":38964},{"utcDay":15825,"count":39089},{"utcDay":15826,"count":39293},{"utcDay":15827,"count":39462},{"utcDay":15828,"count":39596},{"utcDay":15829,"count":39712},{"utcDay":15830,"count":39838},{"utcDay":15831,"count":39971},{"utcDay":15832,"count":40151},{"utcDay":15833,"count":40269},{"utcDay":15834,"count":40385},{"utcDay":15835,"count":40520},{"utcDay":15836,"count":40626},{"utcDay":15837,"count":40747},{"utcDay":15838,"count":40844},{"utcDay":15839,"count":40987},{"utcDay":15840,"count":41168},{"utcDay":15841,"count":41267},{"utcDay":15842,"count":41385},{"utcDay":15843,"count":41489},{"utcDay":15844,"count":41575},{"utcDay":15845,"count":41676},{"utcDay":15846,"count":41782},{"utcDay":15847,"count":41883},{"utcDay":15848,"count":41978},{"utcDay":15849,"count":42048},{"utcDay":15850,"count":42117},{"utcDay":15851,"count":42181},{"utcDay":15852,"count":42255},{"utcDay":15853,"count":42380},{"utcDay":15854,"count":42496},{"utcDay":15855,"count":42602},{"utcDay":15856,"count":42695},{"utcDay":15857,"count":42783},{"utcDay":15858,"count":42902},{"utcDay":15859,"count":42957},{"utcDay":15860,"count":43048},{"utcDay":15861,"count":43136},{"utcDay":15862,"count":43233},{"utcDay":15863,"count":43317},{"utcDay":15864,"count":43456},{"utcDay":15865,"count":43541},{"utcDay":15866,"count":43611},{"utcDay":15867,"count":43700},{"utcDay":15868,"count":43803},{"utcDay":15869,"count":43873},{"utcDay":15870,"count":43941},{"utcDay":15871,"count":43996},{"utcDay":15872,"count":44052},{"utcDay":15873,"count":44092},{"utcDay":15874,"count":44153},{"utcDay":15875,"count":44213},{"utcDay":15876,"count":44250},{"utcDay":15877,"count":44293},{"utcDay":15878,"count":44359},{"utcDay":15879,"count":44426},{"utcDay":15880,"count":44460},{"utcDay":15881,"count":44527},{"utcDay":15882,"count":44572},{"utcDay":15883,"count":44630},{"utcDay":15884,"count":44709},{"utcDay":15885,"count":44757},{"utcDay":15886,"count":44842},{"utcDay":15887,"count":44930},{"utcDay":15888,"count":45029},{"utcDay":15889,"count":45145},{"utcDay":15890,"count":45262},{"utcDay":15891,"count":45340},{"utcDay":15892,"count":45410},{"utcDay":15893,"count":45448},{"utcDay":15894,"count":45516},{"utcDay":15895,"count":45604},{"utcDay":15896,"count":45644},{"utcDay":15897,"count":45703},{"utcDay":15898,"count":45751},{"utcDay":15899,"count":45797},{"utcDay":15900,"count":45849},{"utcDay":15901,"count":45894},{"utcDay":15902,"count":45961},{"utcDay":15903,"count":46008},{"utcDay":15904,"count":46057},{"utcDay":15905,"count":46115},{"utcDay":15906,"count":46161},{"utcDay":15907,"count":46229},{"utcDay":15908,"count":46267},{"utcDay":15909,"count":46335},{"utcDay":15910,"count":46394},{"utcDay":15911,"count":46451},{"utcDay":15912,"count":46531},{"utcDay":15913,"count":46562},{"utcDay":15914,"count":46621},{"utcDay":15915,"count":46692},{"utcDay":15916,"count":46757},{"utcDay":15917,"count":46827},{"utcDay":15918,"count":46888},{"utcDay":15919,"count":46925},{"utcDay":15920,"count":46975},{"utcDay":15921,"count":47066},{"utcDay":15922,"count":47136},{"utcDay":15923,"count":47187},{"utcDay":15924,"count":47229},{"utcDay":15925,"count":47315},{"utcDay":15926,"count":47353},{"utcDay":15927,"count":47381},{"utcDay":15928,"count":47449},{"utcDay":15929,"count":47516},{"utcDay":15930,"count":47598},{"utcDay":15931,"count":47748},{"utcDay":15932,"count":47818},{"utcDay":15933,"count":47898},{"utcDay":15934,"count":47952},{"utcDay":15935,"count":48023},{"utcDay":15936,"count":48108},{"utcDay":15937,"count":48191},{"utcDay":15938,"count":48250},{"utcDay":15939,"count":48665},{"utcDay":15940,"count":48785},{"utcDay":15941,"count":48859},{"utcDay":15942,"count":48923},{"utcDay":15943,"count":48981},{"utcDay":15944,"count":49061},{"utcDay":15945,"count":49142},{"utcDay":15946,"count":49215},{"utcDay":15947,"count":49275},{"utcDay":15948,"count":49351},{"utcDay":15949,"count":49396},{"utcDay":15950,"count":49484},{"utcDay":15951,"count":49537},{"utcDay":15952,"count":49580},{"utcDay":15953,"count":49660},{"utcDay":15954,"count":49704},{"utcDay":15955,"count":49755},{"utcDay":15956,"count":49829},{"utcDay":15957,"count":49932},{"utcDay":15958,"count":50056},{"utcDay":15959,"count":50133},{"utcDay":15960,"count":50174},{"utcDay":15961,"count":50205},{"utcDay":15962,"count":50246},{"utcDay":15963,"count":50299},{"utcDay":15964,"count":50347},{"utcDay":15965,"count":50394},{"utcDay":15966,"count":50443},{"utcDay":15967,"count":50489},{"utcDay":15968,"count":50556},{"utcDay":15969,"count":50624},{"utcDay":15970,"count":50679},{"utcDay":15971,"count":50716},{"utcDay":15972,"count":50789},{"utcDay":15973,"count":50824},{"utcDay":15974,"count":50901},{"utcDay":15975,"count":50940},{"utcDay":15976,"count":51002},{"utcDay":15977,"count":51041},{"utcDay":15978,"count":51085},{"utcDay":15979,"count":51141},{"utcDay":15980,"count":51400},{"utcDay":15981,"count":51665},{"utcDay":15982,"count":51872},{"utcDay":15983,"count":52008},{"utcDay":15984,"count":52130},{"utcDay":15985,"count":52264},{"utcDay":15986,"count":52366},{"utcDay":15987,"count":52439},{"utcDay":15988,"count":52520},{"utcDay":15989,"count":52591},{"utcDay":15990,"count":52650},{"utcDay":15991,"count":52738},{"utcDay":15992,"count":52828},{"utcDay":15993,"count":52925},{"utcDay":15994,"count":53033},{"utcDay":15995,"count":53130},{"utcDay":15996,"count":53229},{"utcDay":15997,"count":53363},{"utcDay":15998,"count":53446},{"utcDay":15999,"count":53560},{"utcDay":16000,"count":53694},{"utcDay":16001,"count":53840},{"utcDay":16002,"count":53951},{"utcDay":16003,"count":54076},{"utcDay":16004,"count":54186},{"utcDay":16005,"count":54283},{"utcDay":16006,"count":54384},{"utcDay":16007,"count":55285},{"utcDay":16008,"count":55688},{"utcDay":16009,"count":55922},{"utcDay":16010,"count":56122},{"utcDay":16011,"count":56285},{"utcDay":16012,"count":56496},{"utcDay":16013,"count":56709},{"utcDay":16014,"count":56954},{"utcDay":16015,"count":57571},{"utcDay":16016,"count":58307},{"utcDay":16017,"count":58994},{"utcDay":16018,"count":59492},{"utcDay":16019,"count":59995},{"utcDay":16020,"count":60549},{"utcDay":16021,"count":61059},{"utcDay":16022,"count":61934},{"utcDay":16023,"count":62434},{"utcDay":16024,"count":62754},{"utcDay":16025,"count":63173},{"utcDay":16026,"count":63824},{"utcDay":16027,"count":65120},{"utcDay":16028,"count":66296},{"utcDay":16029,"count":67022},{"utcDay":16030,"count":67651},{"utcDay":16031,"count":68625},{"utcDay":16032,"count":69234},{"utcDay":16033,"count":69796},{"utcDay":16034,"count":70645},{"utcDay":16035,"count":71375},{"utcDay":16036,"count":72480},{"utcDay":16037,"count":73488},{"utcDay":16038,"count":74089},{"utcDay":16039,"count":74630},{"utcDay":16040,"count":75530},{"utcDay":16041,"count":76275},{"utcDay":16042,"count":76818},{"utcDay":16043,"count":77486},{"utcDay":16044,"count":78408},{"utcDay":16045,"count":79201},{"utcDay":16046,"count":79999},{"utcDay":16047,"count":80638},{"utcDay":16048,"count":81392},{"utcDay":16049,"count":82028},{"utcDay":16050,"count":82581},{"utcDay":16051,"count":83085},{"utcDay":16052,"count":83408},{"utcDay":16053,"count":83746},{"utcDay":16054,"count":84352},{"utcDay":16055,"count":84913},{"utcDay":16056,"count":85351},{"utcDay":16057,"count":87164},{"utcDay":16058,"count":87765},{"utcDay":16059,"count":88069},{"utcDay":16060,"count":88474},{"utcDay":16061,"count":88768},{"utcDay":16062,"count":89087},{"utcDay":16063,"count":89336},{"utcDay":16064,"count":89583},{"utcDay":16065,"count":89922},{"utcDay":16066,"count":90195},{"utcDay":16067,"count":90495},{"utcDay":16068,"count":90782},{"utcDay":16069,"count":90977},{"utcDay":16070,"count":91092},{"utcDay":16071,"count":91313},{"utcDay":16072,"count":91604},{"utcDay":16073,"count":91899},{"utcDay":16074,"count":92172},{"utcDay":16075,"count":92507},{"utcDay":16076,"count":92828},{"utcDay":16077,"count":93175},{"utcDay":16078,"count":93479},{"utcDay":16079,"count":93851},{"utcDay":16080,"count":94178},{"utcDay":16081,"count":94502},{"utcDay":16082,"count":94828},{"utcDay":16083,"count":95072},{"utcDay":16084,"count":95392},{"utcDay":16085,"count":95708},{"utcDay":16086,"count":96041},{"utcDay":16087,"count":96272},{"utcDay":16088,"count":96449},{"utcDay":16089,"count":96702},{"utcDay":16090,"count":97105},{"utcDay":16091,"count":97431},{"utcDay":16092,"count":97823},{"utcDay":16093,"count":98181},{"utcDay":16094,"count":98484},{"utcDay":16095,"count":98759},{"utcDay":16096,"count":99018},{"utcDay":16097,"count":99419},{"utcDay":16098,"count":99823},{"utcDay":16099,"count":100304},{"utcDay":16100,"count":100768},{"utcDay":16101,"count":101071},{"utcDay":16102,"count":101311},{"utcDay":16103,"count":101509},{"utcDay":16104,"count":101776},{"utcDay":16105,"count":102022},{"utcDay":16106,"count":102280},{"utcDay":16107,"count":102648},{"utcDay":16108,"count":103028},{"utcDay":16109,"count":103273},{"utcDay":16110,"count":103553},{"utcDay":16111,"count":104007},{"utcDay":16112,"count":104403},{"utcDay":16113,"count":104667},{"utcDay":16114,"count":105131},{"utcDay":16115,"count":105555},{"utcDay":16116,"count":105800},{"utcDay":16117,"count":106082},{"utcDay":16118,"count":106283},{"utcDay":16119,"count":106490},{"utcDay":16120,"count":106725},{"utcDay":16121,"count":107006},{"utcDay":16122,"count":107214},{"utcDay":16123,"count":107420},{"utcDay":16124,"count":107652},{"utcDay":16125,"count":108068},{"utcDay":16126,"count":108968},{"utcDay":16127,"count":109647},{"utcDay":16128,"count":110084},{"utcDay":16129,"count":110571},{"utcDay":16130,"count":110809},{"utcDay":16131,"count":111107},{"utcDay":16132,"count":111375},{"utcDay":16133,"count":111636},{"utcDay":16134,"count":111872},{"utcDay":16135,"count":112261},{"utcDay":16136,"count":112616},{"utcDay":16137,"count":113064},{"utcDay":16138,"count":113259},{"utcDay":16139,"count":113454},{"utcDay":16140,"count":113667},{"utcDay":16141,"count":113840},{"utcDay":16142,"count":114016},{"utcDay":16143,"count":114107},{"utcDay":16144,"count":114177},{"utcDay":16145,"count":114327},{"utcDay":16146,"count":114458},{"utcDay":16147,"count":114606},{"utcDay":16148,"count":114718},{"utcDay":16149,"count":114842},{"utcDay":16150,"count":114970},{"utcDay":16151,"count":115101},{"utcDay":16152,"count":115237},{"utcDay":16153,"count":115371},{"utcDay":16154,"count":115502},{"utcDay":16155,"count":115654},{"utcDay":16156,"count":115835},{"utcDay":16157,"count":115940},{"utcDay":16158,"count":116061},{"utcDay":16159,"count":116214},{"utcDay":16160,"count":116341},{"utcDay":16161,"count":116472},{"utcDay":16162,"count":116576},{"utcDay":16163,"count":116668},{"utcDay":16164,"count":116731},{"utcDay":16165,"count":116827},{"utcDay":16166,"count":116884},{"utcDay":16167,"count":116981},{"utcDay":16168,"count":117061},{"utcDay":16169,"count":117133},{"utcDay":16170,"count":117294},{"utcDay":16171,"count":117379},{"utcDay":16172,"count":117452},{"utcDay":16173,"count":117541},{"utcDay":16174,"count":117599},{"utcDay":16175,"count":117698},{"utcDay":16176,"count":117806},{"utcDay":16177,"count":117878},{"utcDay":16178,"count":117984},{"utcDay":16179,"count":118036},{"utcDay":16180,"count":118084},{"utcDay":16181,"count":118159},{"utcDay":16182,"count":118262},{"utcDay":16183,"count":118377},{"utcDay":16184,"count":118464},{"utcDay":16185,"count":118602},{"utcDay":16186,"count":118676},{"utcDay":16187,"count":118778},{"utcDay":16188,"count":118889},{"utcDay":16189,"count":119009},{"utcDay":16190,"count":119153},{"utcDay":16191,"count":119254},{"utcDay":16192,"count":119328},{"utcDay":16193,"count":119414},{"utcDay":16194,"count":119503},{"utcDay":16195,"count":119585},{"utcDay":16196,"count":119675},{"utcDay":16197,"count":119766},{"utcDay":16198,"count":119886},{"utcDay":16199,"count":119968},{"utcDay":16200,"count":120096},{"utcDay":16201,"count":120116},{"utcDay":16202,"count":120181},{"utcDay":16203,"count":120287},{"utcDay":16204,"count":120366},{"utcDay":16205,"count":120455},{"utcDay":16206,"count":120567},{"utcDay":16207,"count":120663},{"utcDay":16208,"count":120753},{"utcDay":16209,"count":120805},{"utcDay":16210,"count":120914},{"utcDay":16211,"count":121003},{"utcDay":16212,"count":121101},{"utcDay":16213,"count":121198},{"utcDay":16214,"count":121252},{"utcDay":16215,"count":121349},{"utcDay":16216,"count":121407},{"utcDay":16217,"count":121501},{"utcDay":16218,"count":121592},{"utcDay":16219,"count":121687},{"utcDay":16220,"count":121760},{"utcDay":16221,"count":121824},{"utcDay":16222,"count":121925},{"utcDay":16223,"count":121997},{"utcDay":16224,"count":122114},{"utcDay":16225,"count":122224},{"utcDay":16226,"count":122307},{"utcDay":16227,"count":122358},{"utcDay":16228,"count":122425},{"utcDay":16229,"count":122501},{"utcDay":16230,"count":122561},{"utcDay":16231,"count":122628},{"utcDay":16232,"count":122719},{"utcDay":16233,"count":122841},{"utcDay":16234,"count":122901},{"utcDay":16235,"count":122942},{"utcDay":16236,"count":123011},{"utcDay":16237,"count":123097},{"utcDay":16238,"count":123178},{"utcDay":16239,"count":123272},{"utcDay":16240,"count":123323},{"utcDay":16241,"count":123390},{"utcDay":16242,"count":123449},{"utcDay":16243,"count":123508},{"utcDay":16244,"count":123569},{"utcDay":16245,"count":123642},{"utcDay":16246,"count":123667},{"utcDay":16247,"count":123755},{"utcDay":16248,"count":123814},{"utcDay":16249,"count":123876},{"utcDay":16250,"count":123941},{"utcDay":16251,"count":124025},{"utcDay":16252,"count":124176},{"utcDay":16253,"count":124296},{"utcDay":16254,"count":124345},{"utcDay":16255,"count":124436},{"utcDay":16256,"count":124485},{"utcDay":16257,"count":124576},{"utcDay":16258,"count":125148},{"utcDay":16259,"count":127640},{"utcDay":16260,"count":128919},{"utcDay":16261,"count":129021},{"utcDay":16262,"count":129127},{"utcDay":16263,"count":129207},{"utcDay":16264,"count":129301},{"utcDay":16265,"count":129393},{"utcDay":16266,"count":129476},{"utcDay":16267,"count":129589},{"utcDay":16268,"count":129673},{"utcDay":16269,"count":129758},{"utcDay":16270,"count":129849},{"utcDay":16271,"count":129956},{"utcDay":16272,"count":130042},{"utcDay":16273,"count":130131},{"utcDay":16274,"count":130190},{"utcDay":16275,"count":130295},{"utcDay":16276,"count":130354},{"utcDay":16277,"count":130424},{"utcDay":16278,"count":130485},{"utcDay":16279,"count":130561},{"utcDay":16280,"count":130664},{"utcDay":16281,"count":130792},{"utcDay":16282,"count":130892},{"utcDay":16283,"count":130983},{"utcDay":16284,"count":131056},{"utcDay":16285,"count":131137},{"utcDay":16286,"count":131184},{"utcDay":16287,"count":131317},{"utcDay":16288,"count":131408},{"utcDay":16289,"count":131526},{"utcDay":16290,"count":131590},{"utcDay":16291,"count":131676},{"utcDay":16292,"count":131735},{"utcDay":16293,"count":131815},{"utcDay":16294,"count":131909},{"utcDay":16295,"count":132040},{"utcDay":16296,"count":132152},{"utcDay":16297,"count":132282},{"utcDay":16298,"count":132348},{"utcDay":16299,"count":132447},{"utcDay":16300,"count":132561},{"utcDay":16301,"count":132655},{"utcDay":16302,"count":132762},{"utcDay":16303,"count":132846},{"utcDay":16304,"count":132924},{"utcDay":16305,"count":133004},{"utcDay":16306,"count":133099},{"utcDay":16307,"count":133196},{"utcDay":16308,"count":133281},{"utcDay":16309,"count":133370},{"utcDay":16310,"count":133455},{"utcDay":16311,"count":133577},{"utcDay":16312,"count":133627},{"utcDay":16313,"count":133708},{"utcDay":16314,"count":133843},{"utcDay":16315,"count":133973},{"utcDay":16316,"count":134067},{"utcDay":16317,"count":134172},{"utcDay":16318,"count":134257},{"utcDay":16319,"count":134347},{"utcDay":16320,"count":134411},{"utcDay":16321,"count":134513},{"utcDay":16322,"count":134660},{"utcDay":16323,"count":134776},{"utcDay":16324,"count":134882},{"utcDay":16325,"count":134979},{"utcDay":16326,"count":135191},{"utcDay":16327,"count":135304},{"utcDay":16328,"count":135372},{"utcDay":16329,"count":135467},{"utcDay":16330,"count":135556},{"utcDay":16331,"count":135638},{"utcDay":16332,"count":135760},{"utcDay":16333,"count":135812},{"utcDay":16334,"count":135897},{"utcDay":16335,"count":135995},{"utcDay":16336,"count":136140},{"utcDay":16337,"count":136254},{"utcDay":16338,"count":136339},{"utcDay":16339,"count":136421},{"utcDay":16340,"count":136517},{"utcDay":16341,"count":136641},{"utcDay":16342,"count":136763},{"utcDay":16343,"count":136906},{"utcDay":16344,"count":137051},{"utcDay":16345,"count":137181},{"utcDay":16346,"count":137262},{"utcDay":16347,"count":137357},{"utcDay":16348,"count":137514},{"utcDay":16349,"count":137674},{"utcDay":16350,"count":137804},{"utcDay":16351,"count":137960},{"utcDay":16352,"count":138082},{"utcDay":16353,"count":138151},{"utcDay":16354,"count":138253},{"utcDay":16355,"count":138349},{"utcDay":16356,"count":138459},{"utcDay":16357,"count":138580},{"utcDay":16358,"count":138718},{"utcDay":16359,"count":138852},{"utcDay":16360,"count":138962},{"utcDay":16361,"count":139029},{"utcDay":16362,"count":139123},{"utcDay":16363,"count":139216},{"utcDay":16364,"count":139320},{"utcDay":16365,"count":139407},{"utcDay":16366,"count":139493},{"utcDay":16367,"count":139569},{"utcDay":16368,"count":139639},{"utcDay":16369,"count":139726},{"utcDay":16370,"count":139779},{"utcDay":16371,"count":139867},{"utcDay":16372,"count":139960},{"utcDay":16373,"count":140014},{"utcDay":16374,"count":140086},{"utcDay":16375,"count":140172},{"utcDay":16376,"count":140353},{"utcDay":16377,"count":140484},{"utcDay":16378,"count":140562},{"utcDay":16379,"count":140943},{"utcDay":16380,"count":141222},{"utcDay":16381,"count":141377},{"utcDay":16382,"count":141475},{"utcDay":16383,"count":141604},{"utcDay":16384,"count":141750},{"utcDay":16385,"count":141869},{"utcDay":16386,"count":142027},{"utcDay":16387,"count":142192},{"utcDay":16388,"count":142296},{"utcDay":16389,"count":142375},{"utcDay":16390,"count":142458},{"utcDay":16391,"count":142575},{"utcDay":16392,"count":142684},{"utcDay":16393,"count":142772},{"utcDay":16394,"count":142856},{"utcDay":16395,"count":142919},{"utcDay":16396,"count":143007},{"utcDay":16397,"count":143086},{"utcDay":16398,"count":143150},{"utcDay":16399,"count":143253},{"utcDay":16400,"count":143349},{"utcDay":16401,"count":143443},{"utcDay":16402,"count":143502},{"utcDay":16403,"count":143557},{"utcDay":16404,"count":143666},{"utcDay":16405,"count":143757},{"utcDay":16406,"count":143895},{"utcDay":16407,"count":143987},{"utcDay":16408,"count":144096},{"utcDay":16409,"count":144163},{"utcDay":16410,"count":144212},{"utcDay":16411,"count":144272},{"utcDay":16412,"count":144382},{"utcDay":16413,"count":144486},{"utcDay":16414,"count":144598},{"utcDay":16415,"count":144872},{"utcDay":16416,"count":145005},{"utcDay":16417,"count":145108},{"utcDay":16418,"count":145234},{"utcDay":16419,"count":145359},{"utcDay":16420,"count":145515},{"utcDay":16421,"count":145635},{"utcDay":16422,"count":145733},{"utcDay":16423,"count":145800},{"utcDay":16424,"count":145854},{"utcDay":16425,"count":145920},{"utcDay":16426,"count":146028},{"utcDay":16427,"count":146137},{"utcDay":16428,"count":146240},{"utcDay":16429,"count":146304},{"utcDay":16430,"count":146405},{"utcDay":16431,"count":146492},{"utcDay":16432,"count":146600},{"utcDay":16433,"count":146716},{"utcDay":16434,"count":146796},{"utcDay":16435,"count":146863},{"utcDay":16436,"count":146961},{"utcDay":16437,"count":147073},{"utcDay":16438,"count":147220},{"utcDay":16439,"count":147365},{"utcDay":16440,"count":147524},{"utcDay":16441,"count":147650},{"utcDay":16442,"count":147761},{"utcDay":16443,"count":147868},{"utcDay":16444,"count":147963},{"utcDay":16445,"count":148062},{"utcDay":16446,"count":148162},{"utcDay":16447,"count":148271},{"utcDay":16448,"count":148420},{"utcDay":16449,"count":148707},{"utcDay":16450,"count":148924},{"utcDay":16451,"count":149038},{"utcDay":16452,"count":149121},{"utcDay":16453,"count":149252},{"utcDay":16454,"count":149371},{"utcDay":16455,"count":149462},{"utcDay":16456,"count":149562},{"utcDay":16457,"count":149662},{"utcDay":16458,"count":149760},{"utcDay":16459,"count":149909},{"utcDay":16460,"count":150130},{"utcDay":16461,"count":150406},{"utcDay":16462,"count":150551},{"utcDay":16463,"count":150684},{"utcDay":16464,"count":150803},{"utcDay":16465,"count":150890},{"utcDay":16466,"count":150957},{"utcDay":16467,"count":151037},{"utcDay":16468,"count":151134},{"utcDay":16469,"count":151250},{"utcDay":16470,"count":151383},{"utcDay":16471,"count":151484},{"utcDay":16472,"count":151578},{"utcDay":16473,"count":151684},{"utcDay":16474,"count":151739},{"utcDay":16475,"count":151808},{"utcDay":16476,"count":151918},{"utcDay":16477,"count":152043},{"utcDay":16478,"count":152248},{"utcDay":16479,"count":152393},{"utcDay":16480,"count":152543},{"utcDay":16481,"count":152729},{"utcDay":16482,"count":152895},{"utcDay":16483,"count":153076},{"utcDay":16484,"count":153218},{"utcDay":16485,"count":153473},{"utcDay":16486,"count":153683},{"utcDay":16487,"count":153789},{"utcDay":16488,"count":153861},{"utcDay":16489,"count":153981},{"utcDay":16490,"count":154097},{"utcDay":16491,"count":154207},{"utcDay":16492,"count":154324},{"utcDay":16493,"count":154413},{"utcDay":16494,"count":154503},{"utcDay":16495,"count":154597},{"utcDay":16496,"count":154692},{"utcDay":16497,"count":154798},{"utcDay":16498,"count":154932},{"utcDay":16499,"count":155129},{"utcDay":16500,"count":155220},{"utcDay":16501,"count":155457},{"utcDay":16502,"count":155507},{"utcDay":16503,"count":155558},{"utcDay":16504,"count":155730},{"utcDay":16505,"count":155868},{"utcDay":16506,"count":156087},{"utcDay":16507,"count":156315},{"utcDay":16508,"count":156514},{"utcDay":16509,"count":156733},{"utcDay":16510,"count":156953},{"utcDay":16511,"count":157223},{"utcDay":16512,"count":157444},{"utcDay":16513,"count":157639},{"utcDay":16514,"count":157917},{"utcDay":16515,"count":158150},{"utcDay":16516,"count":158410},{"utcDay":16517,"count":158633},{"utcDay":16518,"count":158894},{"utcDay":16519,"count":159074},{"utcDay":16520,"count":159227},{"utcDay":16521,"count":159347},{"utcDay":16522,"count":159532},{"utcDay":16523,"count":159725},{"utcDay":16524,"count":159949},{"utcDay":16525,"count":160072},{"utcDay":16526,"count":160169},{"utcDay":16527,"count":160434},{"utcDay":16528,"count":160662},{"utcDay":16529,"count":160852},{"utcDay":16530,"count":161004},{"utcDay":16531,"count":161100},{"utcDay":16532,"count":161321},{"utcDay":16533,"count":161491},{"utcDay":16534,"count":161709},{"utcDay":16535,"count":161903},{"utcDay":16536,"count":162122},{"utcDay":16537,"count":162298},{"utcDay":16538,"count":162462},{"utcDay":16539,"count":162666},{"utcDay":16540,"count":162701},{"utcDay":16541,"count":162747},{"utcDay":16542,"count":162895},{"utcDay":16543,"count":163114},{"utcDay":16544,"count":163271},{"utcDay":16545,"count":163519},{"utcDay":16546,"count":163800},{"utcDay":16547,"count":164022},{"utcDay":16548,"count":164154},{"utcDay":16549,"count":164220},{"utcDay":16550,"count":164284},{"utcDay":16551,"count":164351},{"utcDay":16552,"count":164422},{"utcDay":16553,"count":164463},{"utcDay":16554,"count":164539},{"utcDay":16555,"count":164571},{"utcDay":16556,"count":164596},{"utcDay":16557,"count":164637},{"utcDay":16558,"count":164683},{"utcDay":16559,"count":164730},{"utcDay":16560,"count":164774},{"utcDay":16561,"count":164811},{"utcDay":16562,"count":164814},{"utcDay":16563,"count":164835},{"utcDay":16564,"count":164845},{"utcDay":16565,"count":164869},{"utcDay":16566,"count":164895},{"utcDay":16567,"count":164946},{"utcDay":16568,"count":164994},{"utcDay":16569,"count":165044},{"utcDay":16570,"count":165133},{"utcDay":16571,"count":165186},{"utcDay":16572,"count":165229},{"utcDay":16573,"count":165270},{"utcDay":16574,"count":165343},{"utcDay":16575,"count":165409},{"utcDay":16576,"count":165477},{"utcDay":16577,"count":165538},{"utcDay":16578,"count":165580},{"utcDay":16579,"count":165636},{"utcDay":16580,"count":165695},{"utcDay":16581,"count":165763},{"utcDay":16582,"count":165842},{"utcDay":16583,"count":165911},{"utcDay":16584,"count":165997},{"utcDay":16585,"count":166080},{"utcDay":16586,"count":166157},{"utcDay":16587,"count":166203},{"utcDay":16588,"count":166292},{"utcDay":16589,"count":166358},{"utcDay":16590,"count":166426},{"utcDay":16591,"count":166475},{"utcDay":16592,"count":166788},{"utcDay":16593,"count":166823},{"utcDay":16594,"count":166883},{"utcDay":16595,"count":166911},{"utcDay":16596,"count":166944},{"utcDay":16597,"count":166988},{"utcDay":16598,"count":167026},{"utcDay":16599,"count":167064},{"utcDay":16600,"count":167115},{"utcDay":16601,"count":167163},{"utcDay":16602,"count":167203},{"utcDay":16603,"count":167255},{"utcDay":16604,"count":167315},{"utcDay":16605,"count":167349},{"utcDay":16606,"count":167404},{"utcDay":16607,"count":167426},{"utcDay":16608,"count":167448},{"utcDay":16609,"count":167500},{"utcDay":16610,"count":167535},{"utcDay":16611,"count":167623},{"utcDay":16612,"count":167665},{"utcDay":16613,"count":167705},{"utcDay":16614,"count":167767},{"utcDay":16615,"count":167847},{"utcDay":16616,"count":167958},{"utcDay":16617,"count":168029},{"utcDay":16618,"count":168106},{"utcDay":16619,"count":168166},{"utcDay":16620,"count":168244},{"utcDay":16621,"count":168353},{"utcDay":16622,"count":168494},{"utcDay":16623,"count":168618},{"utcDay":16624,"count":168731},{"utcDay":16625,"count":168824},{"utcDay":16626,"count":168968},{"utcDay":16627,"count":169051},{"utcDay":16628,"count":169193},{"utcDay":16629,"count":169308},{"utcDay":16630,"count":169421},{"utcDay":16631,"count":169522},{"utcDay":16632,"count":169632},{"utcDay":16633,"count":169719},{"utcDay":16634,"count":169811},{"utcDay":16635,"count":169870},{"utcDay":16636,"count":169948},{"utcDay":16637,"count":170072},{"utcDay":16638,"count":170182},{"utcDay":16639,"count":170275},{"utcDay":16640,"count":170371},{"utcDay":16641,"count":170438},{"utcDay":16642,"count":170516},{"utcDay":16643,"count":170626},{"utcDay":16644,"count":170735},{"utcDay":16645,"count":170827},{"utcDay":16646,"count":170929},{"utcDay":16647,"count":170987},{"utcDay":16648,"count":171092},{"utcDay":16649,"count":171150},{"utcDay":16650,"count":171225},{"utcDay":16651,"count":171327},{"utcDay":16652,"count":171415},{"utcDay":16653,"count":171493},{"utcDay":16654,"count":171577},{"utcDay":16655,"count":171639},{"utcDay":16656,"count":171709},{"utcDay":16657,"count":171763},{"utcDay":16658,"count":171817},{"utcDay":16659,"count":171878},{"utcDay":16660,"count":171955},{"utcDay":16661,"count":171993},{"utcDay":16662,"count":171979},{"utcDay":16663,"count":171778},{"utcDay":16664,"count":171646},{"utcDay":16665,"count":171678},{"utcDay":16666,"count":171741},{"utcDay":16667,"count":171772},{"utcDay":16668,"count":171817},{"utcDay":16669,"count":171856},{"utcDay":16670,"count":171891},{"utcDay":16671,"count":171981},{"utcDay":16672,"count":172041},{"utcDay":16673,"count":172047},{"utcDay":16674,"count":172098},{"utcDay":16675,"count":172122},{"utcDay":16676,"count":172157},{"utcDay":16677,"count":172184},{"utcDay":16678,"count":172230},{"utcDay":16679,"count":172282},{"utcDay":16680,"count":172290},{"utcDay":16681,"count":172298},{"utcDay":16682,"count":172308},{"utcDay":16683,"count":172335},{"utcDay":16684,"count":172380},{"utcDay":16685,"count":172182},{"utcDay":16686,"count":172115},{"utcDay":16687,"count":172187},{"utcDay":16688,"count":172225},{"utcDay":16689,"count":172259},{"utcDay":16690,"count":172295},{"utcDay":16691,"count":172348},{"utcDay":16692,"count":172382},{"utcDay":16693,"count":172413},{"utcDay":16694,"count":172459},{"utcDay":16695,"count":172528},{"utcDay":16696,"count":172592},{"utcDay":16697,"count":172637},{"utcDay":16698,"count":172665},{"utcDay":16699,"count":172728},{"utcDay":16700,"count":172773},{"utcDay":16701,"count":172821},{"utcDay":16702,"count":172873},{"utcDay":16703,"count":172922},{"utcDay":16704,"count":172942},{"utcDay":16705,"count":172973},{"utcDay":16706,"count":173002},{"utcDay":16707,"count":173034},{"utcDay":16708,"count":173057},{"utcDay":16709,"count":173073},{"utcDay":16710,"count":173101},{"utcDay":16711,"count":173136},{"utcDay":16712,"count":173169},{"utcDay":16713,"count":173194},{"utcDay":16714,"count":173243},{"utcDay":16715,"count":173306},{"utcDay":16716,"count":173358},{"utcDay":16717,"count":173413},{"utcDay":16718,"count":173449},{"utcDay":16719,"count":173480},{"utcDay":16720,"count":173529},{"utcDay":16721,"count":173567},{"utcDay":16722,"count":173611},{"utcDay":16723,"count":173669},{"utcDay":16724,"count":173713},{"utcDay":16725,"count":173752},{"utcDay":16726,"count":173794},{"utcDay":16727,"count":173853},{"utcDay":16728,"count":173907},{"utcDay":16729,"count":173937},{"utcDay":16730,"count":173990},{"utcDay":16731,"count":174022},{"utcDay":16732,"count":174061},{"utcDay":16733,"count":174124},{"utcDay":16734,"count":174187},{"utcDay":16735,"count":174257},{"utcDay":16736,"count":174326},{"utcDay":16737,"count":174420},{"utcDay":16738,"count":174491},{"utcDay":16739,"count":174548},{"utcDay":16740,"count":174655},{"utcDay":16741,"count":174734},{"utcDay":16742,"count":174994},{"utcDay":16743,"count":175327},{"utcDay":16744,"count":175511},{"utcDay":16745,"count":175554},{"utcDay":16746,"count":175357},{"utcDay":16747,"count":175304},{"utcDay":16748,"count":175367},{"utcDay":16749,"count":175410},{"utcDay":16750,"count":175473},{"utcDay":16751,"count":175540},{"utcDay":16752,"count":175592},{"utcDay":16753,"count":175632},{"utcDay":16754,"count":175685},{"utcDay":16755,"count":175728},{"utcDay":16756,"count":175785},{"utcDay":16757,"count":175845},{"utcDay":16758,"count":175873},{"utcDay":16759,"count":175939},{"utcDay":16760,"count":176002},{"utcDay":16761,"count":176057},{"utcDay":16762,"count":176117},{"utcDay":16763,"count":176163},{"utcDay":16764,"count":176191},{"utcDay":16765,"count":176238},{"utcDay":16766,"count":176292},{"utcDay":16767,"count":176323},{"utcDay":16768,"count":176369},{"utcDay":16769,"count":176396},{"utcDay":16770,"count":176468},{"utcDay":16771,"count":176528},{"utcDay":16772,"count":176589},{"utcDay":16773,"count":176636},{"utcDay":16774,"count":176695},{"utcDay":16775,"count":176750},{"utcDay":16776,"count":176776},{"utcDay":16777,"count":177009},{"utcDay":16778,"count":177188},{"utcDay":16779,"count":177317},{"utcDay":16780,"count":177362},{"utcDay":16781,"count":177441},{"utcDay":16782,"count":177495},{"utcDay":16783,"count":177563},{"utcDay":16784,"count":177194},{"utcDay":16785,"count":176901},{"utcDay":16786,"count":176564},{"utcDay":16787,"count":176318},{"utcDay":16788,"count":176031},{"utcDay":16789,"count":175778},{"utcDay":16790,"count":175491},{"utcDay":16791,"count":175187},{"utcDay":16792,"count":174969},{"utcDay":16793,"count":174618},{"utcDay":16794,"count":174076},{"utcDay":16795,"count":173724},{"utcDay":16796,"count":173261},{"utcDay":16797,"count":172681},{"utcDay":16798,"count":172231},{"utcDay":16799,"count":171914},{"utcDay":16800,"count":171685},{"utcDay":16801,"count":171472},{"utcDay":16802,"count":171297},{"utcDay":16803,"count":171101},{"utcDay":16804,"count":170951},{"utcDay":16805,"count":170861},{"utcDay":16806,"count":170919},{"utcDay":16807,"count":171005},{"utcDay":16808,"count":171071},{"utcDay":16809,"count":171134},{"utcDay":16810,"count":171175},{"utcDay":16811,"count":171230},{"utcDay":16812,"count":171291},{"utcDay":16813,"count":171313},{"utcDay":16814,"count":171368},{"utcDay":16815,"count":171448},{"utcDay":16816,"count":171524},{"utcDay":16817,"count":171571},{"utcDay":16818,"count":171580},{"utcDay":16819,"count":171622},{"utcDay":16820,"count":171647},{"utcDay":16821,"count":171735},{"utcDay":16822,"count":171779},{"utcDay":16823,"count":171801},{"utcDay":16824,"count":171787},{"utcDay":16825,"count":171799},{"utcDay":16826,"count":171828},{"utcDay":16827,"count":171851},{"utcDay":16828,"count":171902},{"utcDay":16829,"count":171957},{"utcDay":16830,"count":172000},{"utcDay":16831,"count":172016},{"utcDay":16832,"count":172029},{"utcDay":16833,"count":172063},{"utcDay":16834,"count":172108},{"utcDay":16835,"count":172125},{"utcDay":16836,"count":172149},{"utcDay":16837,"count":172160},{"utcDay":16838,"count":172200},{"utcDay":16839,"count":172257},{"utcDay":16840,"count":172306},{"utcDay":16841,"count":172372},{"utcDay":16842,"count":172445},{"utcDay":16843,"count":172485},{"utcDay":16844,"count":172515},{"utcDay":16845,"count":172556},{"utcDay":16846,"count":172599},{"utcDay":16847,"count":172615},{"utcDay":16848,"count":172700},{"utcDay":16849,"count":172773},{"utcDay":16850,"count":172834},{"utcDay":16851,"count":172888},{"utcDay":16852,"count":172918},{"utcDay":16853,"count":172927},{"utcDay":16854,"count":172972},{"utcDay":16855,"count":173025},{"utcDay":16856,"count":173083},{"utcDay":16857,"count":173151},{"utcDay":16858,"count":173165},{"utcDay":16859,"count":173189},{"utcDay":16860,"count":173243},{"utcDay":16861,"count":173269},{"utcDay":16862,"count":173321},{"utcDay":16863,"count":173344},{"utcDay":16864,"count":173375},{"utcDay":16865,"count":173363},{"utcDay":16866,"count":173392},{"utcDay":16867,"count":173374},{"utcDay":16868,"count":173410},{"utcDay":16869,"count":173409},{"utcDay":16870,"count":173447},{"utcDay":16871,"count":173448},{"utcDay":16872,"count":173462},{"utcDay":16873,"count":173480},{"utcDay":16874,"count":173498},{"utcDay":16875,"count":173523},{"utcDay":16876,"count":173537},{"utcDay":16877,"count":173579},{"utcDay":16878,"count":173616},{"utcDay":16879,"count":173619},{"utcDay":16880,"count":173632},{"utcDay":16881,"count":173759},{"utcDay":16882,"count":173781},{"utcDay":16883,"count":173815},{"utcDay":16884,"count":173823},{"utcDay":16885,"count":173836},{"utcDay":16886,"count":173855},{"utcDay":16887,"count":173885},{"utcDay":16888,"count":173932},{"utcDay":16889,"count":173973},{"utcDay":16890,"count":173996},{"utcDay":16891,"count":174032},{"utcDay":16892,"count":174136},{"utcDay":16893,"count":174143},{"utcDay":16894,"count":174171},{"utcDay":16895,"count":174223},{"utcDay":16896,"count":174265},{"utcDay":16897,"count":174293},{"utcDay":16898,"count":174372},{"utcDay":16899,"count":174407},{"utcDay":16900,"count":174441},{"utcDay":16901,"count":174528},{"utcDay":16902,"count":174574},{"utcDay":16903,"count":174622},{"utcDay":16904,"count":174664},{"utcDay":16905,"count":174717},{"utcDay":16906,"count":174750},{"utcDay":16907,"count":174781},{"utcDay":16908,"count":174823},{"utcDay":16909,"count":174859},{"utcDay":16910,"count":174901},{"utcDay":16911,"count":174967},{"utcDay":16912,"count":175013},{"utcDay":16913,"count":175044},{"utcDay":16914,"count":175100},{"utcDay":16915,"count":175154},{"utcDay":16916,"count":175226},{"utcDay":16917,"count":175306},{"utcDay":16918,"count":175370},{"utcDay":16919,"count":175419},{"utcDay":16920,"count":175477},{"utcDay":16921,"count":175515},{"utcDay":16922,"count":175567},{"utcDay":16923,"count":176086},{"utcDay":16924,"count":176273},{"utcDay":16925,"count":176493},{"utcDay":16926,"count":176646},{"utcDay":16927,"count":176743},{"utcDay":16928,"count":176812},{"utcDay":16929,"count":176912},{"utcDay":16930,"count":176983},{"utcDay":16931,"count":177024},{"utcDay":16932,"count":177101},{"utcDay":16933,"count":177139},{"utcDay":16934,"count":177189},{"utcDay":16935,"count":177248},{"utcDay":16936,"count":177282},{"utcDay":16937,"count":177335},{"utcDay":16938,"count":177368},{"utcDay":16939,"count":177415},{"utcDay":16940,"count":177517},{"utcDay":16941,"count":177568},{"utcDay":16942,"count":177615},{"utcDay":16943,"count":177673},{"utcDay":16944,"count":177726},{"utcDay":16945,"count":177782},{"utcDay":16946,"count":177819},{"utcDay":16947,"count":177842},{"utcDay":16948,"count":177912},{"utcDay":16949,"count":177976},{"utcDay":16950,"count":178067},{"utcDay":16951,"count":178161},{"utcDay":16952,"count":178266},{"utcDay":16953,"count":178356},{"utcDay":16954,"count":178421},{"utcDay":16955,"count":178504},{"utcDay":16956,"count":178591},{"utcDay":16957,"count":178662},{"utcDay":16958,"count":178715},{"utcDay":16959,"count":178789},{"utcDay":16960,"count":178859},{"utcDay":16961,"count":178938},{"utcDay":16962,"count":179004},{"utcDay":16963,"count":179085},{"utcDay":16964,"count":179307},{"utcDay":16965,"count":179508},{"utcDay":16966,"count":179662},{"utcDay":16967,"count":179799},{"utcDay":16968,"count":180007},{"utcDay":16969,"count":180159},{"utcDay":16970,"count":180343},{"utcDay":16971,"count":180463},{"utcDay":16972,"count":180617},{"utcDay":16973,"count":180783},{"utcDay":16974,"count":180932},{"utcDay":16975,"count":181046},{"utcDay":16976,"count":181185},{"utcDay":16977,"count":181298},{"utcDay":16978,"count":181384},{"utcDay":16979,"count":181471},{"utcDay":16980,"count":181563},{"utcDay":16981,"count":181645},{"utcDay":16982,"count":181747},{"utcDay":16983,"count":181849},{"utcDay":16984,"count":181951},{"utcDay":16985,"count":182053},{"utcDay":16986,"count":182155},{"utcDay":16987,"count":182257},{"utcDay":16988,"count":182360},{"utcDay":16989,"count":182390},{"utcDay":16990,"count":182458},{"utcDay":16991,"count":182549},{"utcDay":16992,"count":182653},{"utcDay":16993,"count":182739},{"utcDay":16994,"count":182799},{"utcDay":16995,"count":182877},{"utcDay":16996,"count":182954},{"utcDay":16997,"count":183009},{"utcDay":16998,"count":183078},{"utcDay":16999,"count":183144},{"utcDay":17000,"count":183205},{"utcDay":17001,"count":183274},{"utcDay":17002,"count":183337},{"utcDay":17003,"count":183385},{"utcDay":17004,"count":183417},{"utcDay":17005,"count":183468},{"utcDay":17006,"count":183509},{"utcDay":17007,"count":183586},{"utcDay":17008,"count":183660},{"utcDay":17009,"count":183718},{"utcDay":17010,"count":183796},{"utcDay":17011,"count":183859},{"utcDay":17012,"count":183910},{"utcDay":17013,"count":184003},{"utcDay":17014,"count":184086},{"utcDay":17015,"count":184390},{"utcDay":17016,"count":184902},{"utcDay":17017,"count":185060},{"utcDay":17018,"count":185230},{"utcDay":17019,"count":185351},{"utcDay":17020,"count":185459},{"utcDay":17021,"count":185527},{"utcDay":17022,"count":185625},{"utcDay":17023,"count":185721},{"utcDay":17024,"count":185769},{"utcDay":17025,"count":185826},{"utcDay":17026,"count":185899},{"utcDay":17027,"count":185966},{"utcDay":17028,"count":186041},{"utcDay":17029,"count":186111},{"utcDay":17030,"count":186182},{"utcDay":17031,"count":186217},{"utcDay":17032,"count":186272},{"utcDay":17033,"count":186322},{"utcDay":17034,"count":186376},{"utcDay":17035,"count":186432},{"utcDay":17036,"count":186494},{"utcDay":17037,"count":186560},{"utcDay":17038,"count":186634},{"utcDay":17039,"count":186710},{"utcDay":17040,"count":186785},{"utcDay":17041,"count":186844},{"utcDay":17042,"count":186912},{"utcDay":17043,"count":186983},{"utcDay":17044,"count":187137},{"utcDay":17045,"count":187176},{"utcDay":17046,"count":187235},{"utcDay":17047,"count":187273},{"utcDay":17048,"count":187348},{"utcDay":17049,"count":187416},{"utcDay":17050,"count":187494},{"utcDay":17051,"count":187567},{"utcDay":17052,"count":187724},{"utcDay":17053,"count":187849},{"utcDay":17054,"count":187922},{"utcDay":17055,"count":188014},{"utcDay":17056,"count":188158},{"utcDay":17057,"count":188231},{"utcDay":17058,"count":188334},{"utcDay":17059,"count":188411},{"utcDay":17060,"count":188482},{"utcDay":17061,"count":188568},{"utcDay":17062,"count":188668},{"utcDay":17063,"count":188735},{"utcDay":17064,"count":188835},{"utcDay":17065,"count":188927},{"utcDay":17066,"count":189003},{"utcDay":17067,"count":189069},{"utcDay":17068,"count":189132},{"utcDay":17069,"count":189201},{"utcDay":17070,"count":189277},{"utcDay":17071,"count":189355},{"utcDay":17072,"count":189427},{"utcDay":17073,"count":189499},{"utcDay":17074,"count":189588},{"utcDay":17075,"count":189646},{"utcDay":17076,"count":189722},{"utcDay":17077,"count":189815},{"utcDay":17078,"count":189885},{"utcDay":17079,"count":189954},{"utcDay":17080,"count":190017},{"utcDay":17081,"count":190105},{"utcDay":17082,"count":190173},{"utcDay":17083,"count":190231},{"utcDay":17084,"count":190309},{"utcDay":17085,"count":190430},{"utcDay":17086,"count":190524},{"utcDay":17087,"count":190580},{"utcDay":17088,"count":190646},{"utcDay":17089,"count":190704},{"utcDay":17090,"count":190781},{"utcDay":17091,"count":190851},{"utcDay":17092,"count":190921},{"utcDay":17093,"count":190991},{"utcDay":17094,"count":191061},{"utcDay":17095,"count":191131},{"utcDay":17096,"count":191201},{"utcDay":17097,"count":191271},{"utcDay":17098,"count":191341},{"utcDay":17099,"count":191428},{"utcDay":17100,"count":191515},{"utcDay":17101,"count":191602},{"utcDay":17102,"count":191689},{"utcDay":17103,"count":191776},{"utcDay":17104,"count":191863},{"utcDay":17105,"count":191950},{"utcDay":17106,"count":192037},{"utcDay":17107,"count":192124},{"utcDay":17108,"count":192211},{"utcDay":17109,"count":192298},{"utcDay":17110,"count":192385},{"utcDay":17111,"count":192472},{"utcDay":17112,"count":192566},{"utcDay":17113,"count":192657},{"utcDay":17114,"count":192748},{"utcDay":17115,"count":192840},{"utcDay":17116,"count":192893},{"utcDay":17117,"count":192950},{"utcDay":17118,"count":193068},{"utcDay":17119,"count":193127},{"utcDay":17120,"count":193186},{"utcDay":17121,"count":193293},{"utcDay":17122,"count":193368},{"utcDay":17123,"count":193479},{"utcDay":17124,"count":193570},{"utcDay":17125,"count":193661},{"utcDay":17126,"count":193782},{"utcDay":17127,"count":193903},{"utcDay":17128,"count":193965},{"utcDay":17129,"count":194027},{"utcDay":17130,"count":194093},{"utcDay":17131,"count":194159},{"utcDay":17132,"count":194226},{"utcDay":17133,"count":194304},{"utcDay":17134,"count":194382},{"utcDay":17135,"count":194502},{"utcDay":17136,"count":194583},{"utcDay":17137,"count":194664},{"utcDay":17138,"count":194745},{"utcDay":17139,"count":194829},{"utcDay":17140,"count":194907},{"utcDay":17141,"count":195039},{"utcDay":17142,"count":195171},{"utcDay":17143,"count":195303},{"utcDay":17144,"count":195435},{"utcDay":17145,"count":195567},{"utcDay":17146,"count":195699},{"utcDay":17147,"count":195831},{"utcDay":17148,"count":195963},{"utcDay":17149,"count":196095},{"utcDay":17150,"count":196227},{"utcDay":17151,"count":196359},{"utcDay":17152,"count":196491},{"utcDay":17153,"count":196623},{"utcDay":17154,"count":196755},{"utcDay":17155,"count":196887},{"utcDay":17156,"count":197019},{"utcDay":17157,"count":197151},{"utcDay":17158,"count":197283},{"utcDay":17159,"count":197415},{"utcDay":17160,"count":197547},{"utcDay":17161,"count":197679},{"utcDay":17162,"count":197811},{"utcDay":17163,"count":197943},{"utcDay":17164,"count":198075},{"utcDay":17165,"count":198225},{"utcDay":17166,"count":198338},{"utcDay":17167,"count":198612},{"utcDay":17168,"count":199275},{"utcDay":17169,"count":199536},{"utcDay":17170,"count":199934},{"utcDay":17171,"count":200457},{"utcDay":17172,"count":200992},{"utcDay":17173,"count":201275},{"utcDay":17174,"count":201507},{"utcDay":17175,"count":201690},{"utcDay":17176,"count":201879},{"utcDay":17177,"count":202187},{"utcDay":17178,"count":202456},{"utcDay":17179,"count":202589},{"utcDay":17180,"count":202746},{"utcDay":17181,"count":202909},{"utcDay":17182,"count":203106},{"utcDay":17183,"count":203296},{"utcDay":17184,"count":203440},{"utcDay":17185,"count":203548},{"utcDay":17186,"count":203659},{"utcDay":17187,"count":203763},{"utcDay":17188,"count":203901},{"utcDay":17189,"count":204026},{"utcDay":17190,"count":204199},{"utcDay":17191,"count":204304},{"utcDay":17192,"count":204407},{"utcDay":17193,"count":204498},{"utcDay":17194,"count":204581},{"utcDay":17195,"count":204723},{"utcDay":17196,"count":204830},{"utcDay":17197,"count":204996},{"utcDay":17198,"count":205182},{"utcDay":17199,"count":205371},{"utcDay":17200,"count":205485},{"utcDay":17201,"count":205616},{"utcDay":17202,"count":205754},{"utcDay":17203,"count":205906},{"utcDay":17204,"count":206045},{"utcDay":17205,"count":206255},{"utcDay":17206,"count":206467},{"utcDay":17207,"count":206585},{"utcDay":17208,"count":206717},{"utcDay":17209,"count":206828},{"utcDay":17210,"count":206954},{"utcDay":17211,"count":207090},{"utcDay":17212,"count":207239},{"utcDay":17213,"count":207351},{"utcDay":17214,"count":207454},{"utcDay":17215,"count":207564},{"utcDay":17216,"count":207680},{"utcDay":17217,"count":207822},{"utcDay":17218,"count":208036},{"utcDay":17219,"count":208192},{"utcDay":17220,"count":208395},{"utcDay":17221,"count":208562},{"utcDay":17222,"count":208706},{"utcDay":17223,"count":208775},{"utcDay":17224,"count":208891},{"utcDay":17225,"count":209052},{"utcDay":17226,"count":209203},{"utcDay":17227,"count":209416},{"utcDay":17228,"count":209567},{"utcDay":17229,"count":209717},{"utcDay":17230,"count":209851},{"utcDay":17231,"count":210052},{"utcDay":17232,"count":210346},{"utcDay":17233,"count":210610},{"utcDay":17234,"count":210786},{"utcDay":17235,"count":211109},{"utcDay":17236,"count":211413},{"utcDay":17237,"count":211475},{"utcDay":17238,"count":211538},{"utcDay":17239,"count":211660},{"utcDay":17240,"count":211728},{"utcDay":17241,"count":211875},{"utcDay":17242,"count":211985},{"utcDay":17243,"count":212206},{"utcDay":17244,"count":212344},{"utcDay":17245,"count":212447},{"utcDay":17246,"count":212601},{"utcDay":17247,"count":212727},{"utcDay":17248,"count":212881},{"utcDay":17249,"count":213072},{"utcDay":17250,"count":213209},{"utcDay":17251,"count":213257},{"utcDay":17252,"count":213338},{"utcDay":17253,"count":213407},{"utcDay":17254,"count":213509},{"utcDay":17255,"count":213612},{"utcDay":17256,"count":213680},{"utcDay":17257,"count":213780},{"utcDay":17258,"count":213899},{"utcDay":17259,"count":214037},{"utcDay":17260,"count":214196},{"utcDay":17261,"count":214354},{"utcDay":17262,"count":214428},{"utcDay":17263,"count":214470},{"utcDay":17264,"count":214552},{"utcDay":17265,"count":214615},{"utcDay":17266,"count":214699},{"utcDay":17267,"count":214790},{"utcDay":17268,"count":214860},{"utcDay":17269,"count":214874},{"utcDay":17270,"count":214973},{"utcDay":17271,"count":215012},{"utcDay":17272,"count":215088},{"utcDay":17273,"count":215230},{"utcDay":17274,"count":215326},{"utcDay":17275,"count":215457},{"utcDay":17276,"count":215586},{"utcDay":17277,"count":215677},{"utcDay":17278,"count":215792},{"utcDay":17279,"count":215883},{"utcDay":17280,"count":215964},{"utcDay":17281,"count":216085},{"utcDay":17282,"count":216279},{"utcDay":17283,"count":216466},{"utcDay":17284,"count":216596},{"utcDay":17285,"count":216705},{"utcDay":17286,"count":216868},{"utcDay":17287,"count":217134},{"utcDay":17288,"count":217479},{"utcDay":17289,"count":217732},{"utcDay":17290,"count":218181},{"utcDay":17291,"count":218502},{"utcDay":17292,"count":219009},{"utcDay":17293,"count":219447},{"utcDay":17294,"count":219856},{"utcDay":17295,"count":220406},{"utcDay":17296,"count":220847},{"utcDay":17297,"count":221325},{"utcDay":17298,"count":221733},{"utcDay":17299,"count":222061},{"utcDay":17300,"count":222359},{"utcDay":17301,"count":222703},{"utcDay":17302,"count":223051},{"utcDay":17303,"count":223399},{"utcDay":17304,"count":223790},{"utcDay":17305,"count":224165},{"utcDay":17306,"count":224554},{"utcDay":17307,"count":225127},{"utcDay":17308,"count":226181},{"utcDay":17309,"count":227071},{"utcDay":17310,"count":228163},{"utcDay":17311,"count":229847},{"utcDay":17312,"count":230722},{"utcDay":17313,"count":231640},{"utcDay":17314,"count":232331},{"utcDay":17315,"count":232926},{"utcDay":17316,"count":233477},{"utcDay":17317,"count":234047},{"utcDay":17318,"count":234618},{"utcDay":17319,"count":235108},{"utcDay":17320,"count":235556},{"utcDay":17321,"count":236018},{"utcDay":17322,"count":236757},{"utcDay":17323,"count":237870},{"utcDay":17324,"count":238676},{"utcDay":17325,"count":239277},{"utcDay":17326,"count":239749},{"utcDay":17327,"count":240275},{"utcDay":17328,"count":240976},{"utcDay":17329,"count":241894},{"utcDay":17330,"count":242573},{"utcDay":17331,"count":243267},{"utcDay":17332,"count":244258},{"utcDay":17333,"count":244784},{"utcDay":17334,"count":245346},{"utcDay":17335,"count":245995},{"utcDay":17336,"count":246523},{"utcDay":17337,"count":247240},{"utcDay":17338,"count":247962},{"utcDay":17339,"count":248566},{"utcDay":17340,"count":249128},{"utcDay":17341,"count":249660},{"utcDay":17342,"count":250205},{"utcDay":17343,"count":250908},{"utcDay":17344,"count":251597},{"utcDay":17345,"count":252078},{"utcDay":17346,"count":252605},{"utcDay":17347,"count":252962},{"utcDay":17348,"count":253411},{"utcDay":17349,"count":253850},{"utcDay":17350,"count":254376},{"utcDay":17351,"count":254926},{"utcDay":17352,"count":255434},{"utcDay":17353,"count":255851},{"utcDay":17354,"count":256279},{"utcDay":17355,"count":256747},{"utcDay":17356,"count":257242},{"utcDay":17357,"count":257817},{"utcDay":17358,"count":258760},{"utcDay":17359,"count":259704},{"utcDay":17360,"count":260418},{"utcDay":17361,"count":261132},{"utcDay":17362,"count":261846},{"utcDay":17363,"count":262560},{"utcDay":17364,"count":263274},{"utcDay":17365,"count":263991},{"utcDay":17366,"count":264440},{"utcDay":17367,"count":265662},{"utcDay":17368,"count":266288},{"utcDay":17369,"count":266902},{"utcDay":17370,"count":267571},{"utcDay":17371,"count":268180},{"utcDay":17372,"count":268867},{"utcDay":17373,"count":269476},{"utcDay":17374,"count":270063},{"utcDay":17375,"count":270652},{"utcDay":17376,"count":271175},{"utcDay":17377,"count":272025},{"utcDay":17378,"count":272656},{"utcDay":17379,"count":273930},{"utcDay":17380,"count":274795},{"utcDay":17381,"count":275356},{"utcDay":17382,"count":276235},{"utcDay":17383,"count":277517},{"utcDay":17384,"count":278233},{"utcDay":17385,"count":279061},{"utcDay":17386,"count":279821},{"utcDay":17387,"count":280490},{"utcDay":17388,"count":281258},{"utcDay":17389,"count":282179},{"utcDay":17390,"count":283459},{"utcDay":17391,"count":285938},{"utcDay":17392,"count":287422},{"utcDay":17393,"count":289084},{"utcDay":17394,"count":290176},{"utcDay":17395,"count":291487},{"utcDay":17396,"count":292422},{"utcDay":17397,"count":293242},{"utcDay":17398,"count":294142},{"utcDay":17399,"count":295000},{"utcDay":17400,"count":296041},{"utcDay":17401,"count":297015},{"utcDay":17402,"count":297962},{"utcDay":17403,"count":298796},{"utcDay":17404,"count":299470},{"utcDay":17405,"count":300242},{"utcDay":17406,"count":300967},{"utcDay":17407,"count":302381},{"utcDay":17408,"count":303294},{"utcDay":17409,"count":304414},{"utcDay":17410,"count":305529},{"utcDay":17411,"count":306753},{"utcDay":17412,"count":307649},{"utcDay":17413,"count":309282},{"utcDay":17414,"count":310342},{"utcDay":17415,"count":311762},{"utcDay":17416,"count":313419},{"utcDay":17417,"count":314877},{"utcDay":17418,"count":315977},{"utcDay":17419,"count":317051},{"utcDay":17420,"count":318196},{"utcDay":17421,"count":319311},{"utcDay":17422,"count":321000},{"utcDay":17423,"count":322940},{"utcDay":17424,"count":324831},{"utcDay":17425,"count":325977},{"utcDay":17426,"count":327186},{"utcDay":17427,"count":328527},{"utcDay":17428,"count":329552},{"utcDay":17429,"count":330669},{"utcDay":17430,"count":331660},{"utcDay":17431,"count":332545},{"utcDay":17432,"count":333260},{"utcDay":17433,"count":334193},{"utcDay":17434,"count":335087},{"utcDay":17435,"count":335952},{"utcDay":17436,"count":337262},{"utcDay":17437,"count":338118},{"utcDay":17438,"count":338869},{"utcDay":17439,"count":339573},{"utcDay":17440,"count":340502},{"utcDay":17441,"count":341405},{"utcDay":17442,"count":342481},{"utcDay":17443,"count":343359},{"utcDay":17444,"count":344436},{"utcDay":17445,"count":345339},{"utcDay":17446,"count":346242},{"utcDay":17447,"count":347372},{"utcDay":17448,"count":348502},{"utcDay":17449,"count":349663},{"utcDay":17450,"count":350824},{"utcDay":17451,"count":353126},{"utcDay":17452,"count":355428},{"utcDay":17453,"count":356811},{"utcDay":17454,"count":358422},{"utcDay":17455,"count":359843},{"utcDay":17456,"count":361258},{"utcDay":17457,"count":362832},{"utcDay":17458,"count":363988},{"utcDay":17459,"count":365469},{"utcDay":17460,"count":366950},{"utcDay":17461,"count":368431},{"utcDay":17462,"count":369956},{"utcDay":17463,"count":371557},{"utcDay":17464,"count":372835},{"utcDay":17465,"count":374242},{"utcDay":17466,"count":375297},{"utcDay":17467,"count":376227},{"utcDay":17468,"count":377642},{"utcDay":17469,"count":378976},{"utcDay":17470,"count":380575},{"utcDay":17471,"count":383218},{"utcDay":17472,"count":386459},{"utcDay":17473,"count":388571},{"utcDay":17474,"count":391125},{"utcDay":17475,"count":393679},{"utcDay":17476,"count":396233},{"utcDay":17477,"count":398448},{"utcDay":17478,"count":400663},{"utcDay":17479,"count":402878},{"utcDay":17480,"count":405093},{"utcDay":17481,"count":407308},{"utcDay":17482,"count":409527},{"utcDay":17483,"count":411962},{"utcDay":17484,"count":413847},{"utcDay":17485,"count":415774},{"utcDay":17486,"count":418539},{"utcDay":17487,"count":420948},{"utcDay":17488,"count":423409},{"utcDay":17489,"count":425688},{"utcDay":17490,"count":428309},{"utcDay":17491,"count":430577},{"utcDay":17492,"count":432354},{"utcDay":17493,"count":434166},{"utcDay":17494,"count":436024},{"utcDay":17495,"count":438345},{"utcDay":17496,"count":443395},{"utcDay":17497,"count":447659},{"utcDay":17498,"count":454706},{"utcDay":17499,"count":464274},{"utcDay":17500,"count":469646},{"utcDay":17501,"count":472960},{"utcDay":17502,"count":476149},{"utcDay":17503,"count":479663},{"utcDay":17504,"count":483062},{"utcDay":17505,"count":486554},{"utcDay":17506,"count":492703},{"utcDay":17507,"count":504933},{"utcDay":17508,"count":511849},{"utcDay":17509,"count":516995},{"utcDay":17510,"count":522307},{"utcDay":17511,"count":527659},{"utcDay":17512,"count":532942},{"utcDay":17513,"count":538168},{"utcDay":17514,"count":542416},{"utcDay":17515,"count":546092},{"utcDay":17516,"count":550343},{"utcDay":17517,"count":555085},{"utcDay":17518,"count":559680},{"utcDay":17519,"count":565402},{"utcDay":17520,"count":570843},{"utcDay":17521,"count":576224},{"utcDay":17522,"count":584215},{"utcDay":17523,"count":588532},{"utcDay":17524,"count":591906},{"utcDay":17525,"count":595284},{"utcDay":17526,"count":598849},{"utcDay":17527,"count":602823},{"utcDay":17528,"count":606468},{"utcDay":17529,"count":609662},{"utcDay":17530,"count":613387},{"utcDay":17531,"count":616104},{"utcDay":17532,"count":619161},{"utcDay":17533,"count":622677},{"utcDay":17534,"count":626577},{"utcDay":17535,"count":630250},{"utcDay":17536,"count":634137},{"utcDay":17537,"count":638201},{"utcDay":17538,"count":642493},{"utcDay":17539,"count":646335},{"utcDay":17540,"count":649930},{"utcDay":17541,"count":653670},{"utcDay":17542,"count":657219},{"utcDay":17543,"count":659961},{"utcDay":17544,"count":662702},{"utcDay":17545,"count":665661},{"utcDay":17546,"count":668465},{"utcDay":17547,"count":674040},{"utcDay":17548,"count":679022},{"utcDay":17549,"count":682222},{"utcDay":17550,"count":684679},{"utcDay":17551,"count":686923},{"utcDay":17552,"count":689364},{"utcDay":17553,"count":691853},{"utcDay":17554,"count":694115},{"utcDay":17555,"count":696378},{"utcDay":17556,"count":698499},{"utcDay":17557,"count":700489},{"utcDay":17558,"count":702807},{"utcDay":17559,"count":706195},{"utcDay":17560,"count":708655},{"utcDay":17561,"count":711039},{"utcDay":17562,"count":713391},{"utcDay":17563,"count":715922},{"utcDay":17564,"count":717998},{"utcDay":17565,"count":719642},{"utcDay":17566,"count":721827},{"utcDay":17567,"count":724381},{"utcDay":17568,"count":726711},{"utcDay":17569,"count":728483},{"utcDay":17570,"count":730708},{"utcDay":17571,"count":732277},{"utcDay":17572,"count":733502},{"utcDay":17573,"count":734716},{"utcDay":17574,"count":736082},{"utcDay":17575,"count":737256},{"utcDay":17576,"count":738454},{"utcDay":17577,"count":739828},{"utcDay":17578,"count":741110},{"utcDay":17579,"count":742551},{"utcDay":17580,"count":744025},{"utcDay":17581,"count":745395},{"utcDay":17582,"count":746720},{"utcDay":17583,"count":748066},{"utcDay":17584,"count":749237},{"utcDay":17585,"count":750276},{"utcDay":17586,"count":751315},{"utcDay":17587,"count":752354},{"utcDay":17588,"count":753393},{"utcDay":17589,"count":754432},{"utcDay":17604,"count":770600},{"utcDay":17605,"count":771444},{"utcDay":17606,"count":772446},{"utcDay":17607,"count":773124},{"utcDay":17608,"count":773854},{"utcDay":17609,"count":774766},{"utcDay":17610,"count":775633},{"utcDay":17611,"count":776459},{"utcDay":17612,"count":777224},{"utcDay":17613,"count":777844},{"utcDay":17614,"count":778428},{"utcDay":17615,"count":778460},{"utcDay":17616,"count":779858},{"utcDay":17617,"count":780887},{"utcDay":17618,"count":781960},{"utcDay":17619,"count":783124},{"utcDay":17620,"count":784372},{"utcDay":17621,"count":785267},{"utcDay":17622,"count":786247},{"utcDay":17623,"count":787566},{"utcDay":17624,"count":788509},{"utcDay":17625,"count":789215},{"utcDay":17626,"count":790108},{"utcDay":17627,"count":791141},{"utcDay":17628,"count":791974},{"utcDay":17629,"count":792919},{"utcDay":17630,"count":793862},{"utcDay":17631,"count":794853},{"utcDay":17632,"count":795908},{"utcDay":17633,"count":796928},{"utcDay":17634,"count":798032},{"utcDay":17635,"count":798893},{"utcDay":17636,"count":799809},{"utcDay":17637,"count":800836},{"utcDay":17638,"count":801670},{"utcDay":17639,"count":802567},{"utcDay":17640,"count":803326},{"utcDay":17641,"count":804139},{"utcDay":17642,"count":805100},{"utcDay":17643,"count":806028},{"utcDay":17644,"count":807083},{"utcDay":17645,"count":808363},{"utcDay":17646,"count":809404},{"utcDay":17647,"count":810396},{"utcDay":17648,"count":811227},{"utcDay":17649,"count":812076},{"utcDay":17650,"count":813004},{"utcDay":17651,"count":813927},{"utcDay":17652,"count":815183},{"utcDay":17653,"count":816049},{"utcDay":17654,"count":817109},{"utcDay":17655,"count":818184},{"utcDay":17656,"count":819166},{"utcDay":17657,"count":820179},{"utcDay":17658,"count":821111},{"utcDay":17659,"count":822267},{"utcDay":17660,"count":823441},{"utcDay":17661,"count":824555},{"utcDay":17662,"count":825630},{"utcDay":17663,"count":826547},{"utcDay":17664,"count":827520},{"utcDay":17665,"count":828621},{"utcDay":17666,"count":829593},{"utcDay":17667,"count":830623},{"utcDay":17668,"count":831548},{"utcDay":17669,"count":832456},{"utcDay":17670,"count":833455},{"utcDay":17671,"count":834356},{"utcDay":17672,"count":835445},{"utcDay":17673,"count":836500},{"utcDay":17674,"count":837577},{"utcDay":17675,"count":838604},{"utcDay":17676,"count":839600},{"utcDay":17677,"count":840545},{"utcDay":17678,"count":841493},{"utcDay":17679,"count":843258},{"utcDay":17680,"count":843983},{"utcDay":17681,"count":844932},{"utcDay":17682,"count":845923},{"utcDay":17683,"count":847062},{"utcDay":17684,"count":848099},{"utcDay":17685,"count":849336},{"utcDay":17686,"count":849617},{"utcDay":17687,"count":851593},{"utcDay":17688,"count":852597},{"utcDay":17689,"count":853730},{"utcDay":17690,"count":854734},{"utcDay":17691,"count":855726},{"utcDay":17692,"count":856767},{"utcDay":17693,"count":857778},{"utcDay":17694,"count":858599},{"utcDay":17695,"count":859633},{"utcDay":17696,"count":860560},{"utcDay":17697,"count":861409},{"utcDay":17698,"count":862198},{"utcDay":17699,"count":863253},{"utcDay":17700,"count":864253},{"utcDay":17701,"count":865035},{"utcDay":17702,"count":866057},{"utcDay":17703,"count":866838},{"utcDay":17704,"count":867748},{"utcDay":17705,"count":868673},{"utcDay":17706,"count":869582},{"utcDay":17707,"count":870667},{"utcDay":17708,"count":871604},{"utcDay":17709,"count":872584},{"utcDay":17710,"count":873805},{"utcDay":17711,"count":874715},{"utcDay":17712,"count":875551},{"utcDay":17713,"count":876449},{"utcDay":17714,"count":877447},{"utcDay":17715,"count":878476},{"utcDay":17716,"count":879448},{"utcDay":17717,"count":880503},{"utcDay":17718,"count":881465},{"utcDay":17719,"count":882323},{"utcDay":17720,"count":883409},{"utcDay":17721,"count":884591},{"utcDay":17722,"count":885767},{"utcDay":17723,"count":886775},{"utcDay":17724,"count":887822},{"utcDay":17725,"count":888767},{"utcDay":17726,"count":889781},{"utcDay":17727,"count":890684},{"utcDay":17728,"count":891665},{"utcDay":17729,"count":892467},{"utcDay":17730,"count":893483},{"utcDay":17731,"count":894533},{"utcDay":17732,"count":895595},{"utcDay":17733,"count":896805},{"utcDay":17734,"count":897475},{"utcDay":17735,"count":898414},{"utcDay":17736,"count":899323},{"utcDay":17737,"count":900180},{"utcDay":17738,"count":900994},{"utcDay":17739,"count":901942},{"utcDay":17740,"count":902773},{"utcDay":17741,"count":903712},{"utcDay":17742,"count":904596},{"utcDay":17743,"count":905525},{"utcDay":17744,"count":906606},{"utcDay":17745,"count":907486},{"utcDay":17746,"count":907693},{"utcDay":17747,"count":907887},{"utcDay":17748,"count":908155},{"utcDay":17749,"count":908476},{"utcDay":17750,"count":909219},{"utcDay":17751,"count":909482},{"utcDay":17752,"count":909819},{"utcDay":17753,"count":910099},{"utcDay":17754,"count":910226},{"utcDay":17755,"count":910454},{"utcDay":17756,"count":910617},{"utcDay":17757,"count":910842},{"utcDay":17758,"count":911102},{"utcDay":17759,"count":911450},{"utcDay":17760,"count":911771},{"utcDay":17761,"count":912000},{"utcDay":17762,"count":912171},{"utcDay":17763,"count":912393},{"utcDay":17764,"count":912594},{"utcDay":17765,"count":912815},{"utcDay":17766,"count":913000},{"utcDay":17767,"count":913141},{"utcDay":17768,"count":913339},{"utcDay":17769,"count":913519},{"utcDay":17770,"count":913707},{"utcDay":17771,"count":914074},{"utcDay":17772,"count":914380},{"utcDay":17773,"count":915234},{"utcDay":17774,"count":916056},{"utcDay":17775,"count":916914},{"utcDay":17776,"count":917698},{"utcDay":17777,"count":918783},{"utcDay":17778,"count":919544},{"utcDay":17780,"count":921730},{"utcDay":17781,"count":922608},{"utcDay":17782,"count":923271},{"utcDay":17783,"count":924121},{"utcDay":17784,"count":924802},{"utcDay":17798,"count":937226},{"utcDay":17799,"count":938233},{"utcDay":17800,"count":939016},{"utcDay":17801,"count":939927},{"utcDay":17802,"count":940718},{"utcDay":17803,"count":941516},{"utcDay":17804,"count":942437},{"utcDay":17805,"count":943428},{"utcDay":17806,"count":944352},{"utcDay":17807,"count":945286},{"utcDay":17808,"count":946268},{"utcDay":17809,"count":947063},{"utcDay":17810,"count":947845},{"utcDay":17811,"count":948877},{"utcDay":17812,"count":949710},{"utcDay":17813,"count":950770},{"utcDay":17814,"count":951703},{"utcDay":17815,"count":952722},{"utcDay":17816,"count":953522},{"utcDay":17817,"count":954332},{"utcDay":17818,"count":955347},{"utcDay":17819,"count":956358},{"utcDay":17820,"count":957400},{"utcDay":17821,"count":958490},{"utcDay":17822,"count":959427},{"utcDay":17823,"count":960233},{"utcDay":17824,"count":961109},{"utcDay":17825,"count":962133},{"utcDay":17826,"count":963139},{"utcDay":17827,"count":964172},{"utcDay":17828,"count":965101},{"utcDay":17829,"count":965966},{"utcDay":17830,"count":966916},{"utcDay":17831,"count":967690},{"utcDay":17832,"count":968586},{"utcDay":17833,"count":969686},{"utcDay":17834,"count":970890},{"utcDay":17835,"count":971812},{"utcDay":17836,"count":972716},{"utcDay":17837,"count":973623},{"utcDay":17838,"count":974509},{"utcDay":17839,"count":975485},{"utcDay":17840,"count":976452},{"utcDay":17841,"count":977455},{"utcDay":17842,"count":978057},{"utcDay":17843,"count":978613},{"utcDay":17844,"count":979164},{"utcDay":17845,"count":979590},{"utcDay":17846,"count":980132},{"utcDay":17847,"count":980756},{"utcDay":17848,"count":981378},{"utcDay":17849,"count":982062},{"utcDay":17850,"count":983001},{"utcDay":17851,"count":983786},{"utcDay":17852,"count":984647},{"utcDay":17853,"count":985546},{"utcDay":17854,"count":986555},{"utcDay":17855,"count":987704},{"utcDay":17856,"count":988801},{"utcDay":17857,"count":989776},{"utcDay":17858,"count":990599},{"utcDay":17859,"count":991431},{"utcDay":17860,"count":992578},{"utcDay":17861,"count":993619},{"utcDay":17862,"count":994737},{"utcDay":17863,"count":995779},{"utcDay":17864,"count":996926},{"utcDay":17865,"count":997853},{"utcDay":17866,"count":998765},{"utcDay":17867,"count":999869},{"utcDay":17868,"count":1001190},{"utcDay":17869,"count":1001819},{"utcDay":17870,"count":1002063},{"utcDay":17871,"count":1002339},{"utcDay":17872,"count":1002697},{"utcDay":17873,"count":1003040},{"utcDay":17874,"count":1003302},{"utcDay":17875,"count":1003784},{"utcDay":17876,"count":1004054},{"utcDay":17877,"count":1004412},{"utcDay":17878,"count":1004568},{"utcDay":17879,"count":1004810},{"utcDay":17880,"count":1005019},{"utcDay":17881,"count":1005198},{"utcDay":17882,"count":1005447},{"utcDay":17883,"count":1005759},{"utcDay":17884,"count":1006084},{"utcDay":17885,"count":1006355},{"utcDay":17886,"count":1006660},{"utcDay":17887,"count":1006822},{"utcDay":17888,"count":1006973},{"utcDay":17889,"count":1007145},{"utcDay":17890,"count":1007258},{"utcDay":17891,"count":1007392},{"utcDay":17892,"count":1007639},{"utcDay":17893,"count":1007962},{"utcDay":17894,"count":1008135},{"utcDay":17895,"count":1008291},{"utcDay":17896,"count":1008396},{"utcDay":17897,"count":1008570},{"utcDay":17898,"count":1008780},{"utcDay":17899,"count":1009065},{"utcDay":17900,"count":1009398},{"utcDay":17901,"count":1009568},{"utcDay":17902,"count":1009599},{"utcDay":17903,"count":1009814},{"utcDay":17904,"count":1010134},{"utcDay":17905,"count":1010383},{"utcDay":17906,"count":1010629},{"utcDay":17907,"count":1010896},{"utcDay":17908,"count":1011147},{"utcDay":17909,"count":1011320},{"utcDay":17910,"count":1011522},{"utcDay":17911,"count":1011847},{"utcDay":17912,"count":1012130},{"utcDay":17913,"count":1012294},{"utcDay":17914,"count":1012459},{"utcDay":17915,"count":1012566},{"utcDay":17916,"count":1012664},{"utcDay":17917,"count":1012812},{"utcDay":17918,"count":1013008},{"utcDay":17919,"count":1013239},{"utcDay":17920,"count":1013461},{"utcDay":17921,"count":1013586},{"utcDay":17922,"count":1013748},{"utcDay":17923,"count":1013883},{"utcDay":17924,"count":1014040},{"utcDay":17925,"count":1014278},{"utcDay":17926,"count":1014405},{"utcDay":17927,"count":1014544},{"utcDay":17928,"count":1014687},{"utcDay":17929,"count":1014823},{"utcDay":17930,"count":1014969},{"utcDay":17931,"count":1015163},{"utcDay":17932,"count":1015401},{"utcDay":17933,"count":1015585},{"utcDay":17934,"count":1015736},{"utcDay":17935,"count":1015950},{"utcDay":17936,"count":1016173},{"utcDay":17937,"count":1016345},{"utcDay":17938,"count":1016585},{"utcDay":17939,"count":1016818},{"utcDay":17940,"count":1017079},{"utcDay":17941,"count":1017412},{"utcDay":17942,"count":1017585},{"utcDay":17943,"count":1017777},{"utcDay":17944,"count":1017955},{"utcDay":17945,"count":1018060},{"utcDay":17946,"count":1018314},{"utcDay":17947,"count":1018486},{"utcDay":17948,"count":1018664},{"utcDay":17949,"count":1018784},{"utcDay":17950,"count":1018937},{"utcDay":17951,"count":1019154},{"utcDay":17952,"count":1019402},{"utcDay":17953,"count":1019636},{"utcDay":17954,"count":1019792},{"utcDay":17955,"count":1019981},{"utcDay":17956,"count":1020145},{"utcDay":17957,"count":1020312},{"utcDay":17958,"count":1020460},{"utcDay":17959,"count":1020673},{"utcDay":17960,"count":1020858},{"utcDay":17961,"count":1021046},{"utcDay":17962,"count":1021200},{"utcDay":17963,"count":1021337},{"utcDay":17964,"count":1021434},{"utcDay":17965,"count":1021585},{"utcDay":17966,"count":1021728},{"utcDay":17967,"count":1021949},{"utcDay":17968,"count":1022120},{"utcDay":17969,"count":1022357},{"utcDay":17970,"count":1022597},{"utcDay":17971,"count":1022800},{"utcDay":17972,"count":1023006},{"utcDay":17973,"count":1023614},{"utcDay":17974,"count":1023768},{"utcDay":17975,"count":1023959},{"utcDay":17976,"count":1024332},{"utcDay":17977,"count":1024533},{"utcDay":17978,"count":1024680},{"utcDay":17979,"count":1024838},{"utcDay":17980,"count":1025053},{"utcDay":17981,"count":1025312},{"utcDay":17982,"count":1025529},{"utcDay":17983,"count":1025745},{"utcDay":17984,"count":1025925},{"utcDay":17985,"count":1026132},{"utcDay":17986,"count":1026301},{"utcDay":17987,"count":1026511},{"utcDay":17988,"count":1026979},{"utcDay":17989,"count":1027538},{"utcDay":17990,"count":1027918},{"utcDay":17991,"count":1028172},{"utcDay":17992,"count":1028440},{"utcDay":17993,"count":1028797},{"utcDay":17994,"count":1029153},{"utcDay":17995,"count":1029535},{"utcDay":17996,"count":1029902},{"utcDay":17997,"count":1030175},{"utcDay":17998,"count":1030512},{"utcDay":17999,"count":1030787},{"utcDay":18000,"count":1031072},{"utcDay":18001,"count":1031395},{"utcDay":18002,"count":1031714},{"utcDay":18003,"count":1031993},{"utcDay":18004,"count":1032309},{"utcDay":18005,"count":1032440},{"utcDay":18006,"count":1032569},{"utcDay":18007,"count":1032720},{"utcDay":18008,"count":1032891},{"utcDay":18009,"count":1033131},{"utcDay":18010,"count":1033359},{"utcDay":18011,"count":1033581},{"utcDay":18012,"count":1033804},{"utcDay":18013,"count":1033962},{"utcDay":18014,"count":1034183},{"utcDay":18015,"count":1034347},{"utcDay":18016,"count":1034518},{"utcDay":18017,"count":1034636},{"utcDay":18018,"count":1034841},{"utcDay":18019,"count":1035074},{"utcDay":18020,"count":1035253},{"utcDay":18021,"count":1035449},{"utcDay":18022,"count":1035694},{"utcDay":18023,"count":1035917},{"utcDay":18024,"count":1036168},{"utcDay":18025,"count":1036342},{"utcDay":18026,"count":1036750},{"utcDay":18027,"count":1037229},{"utcDay":18028,"count":1037658},{"utcDay":18029,"count":1038233},{"utcDay":18030,"count":1038884},{"utcDay":18031,"count":1039391},{"utcDay":18032,"count":1039936},{"utcDay":18033,"count":1040419},{"utcDay":18034,"count":1040658},{"utcDay":18035,"count":1041068},{"utcDay":18036,"count":1041484},{"utcDay":18037,"count":1041838},{"utcDay":18038,"count":1042145},{"utcDay":18039,"count":1042551},{"utcDay":18040,"count":1042825},{"utcDay":18041,"count":1043080},{"utcDay":18042,"count":1043330},{"utcDay":18043,"count":1043786},{"utcDay":18044,"count":1044191},{"utcDay":18045,"count":1044664},{"utcDay":18046,"count":1045066},{"utcDay":18047,"count":1045410},{"utcDay":18048,"count":1045685},{"utcDay":18049,"count":1045912},{"utcDay":18050,"count":1046234},{"utcDay":18051,"count":1046691},{"utcDay":18052,"count":1046966},{"utcDay":18053,"count":1047250},{"utcDay":18054,"count":1047565},{"utcDay":18055,"count":1047754},{"utcDay":18056,"count":1048007},{"utcDay":18057,"count":1048451},{"utcDay":18058,"count":1048741},{"utcDay":18059,"count":1049101},{"utcDay":18060,"count":1049513},{"utcDay":18061,"count":1049870},{"utcDay":18062,"count":1050165},{"utcDay":18063,"count":1050688},{"utcDay":18064,"count":1051113},{"utcDay":18065,"count":1051588},{"utcDay":18066,"count":1052121},{"utcDay":18067,"count":1052674},{"utcDay":18068,"count":1053168},{"utcDay":18069,"count":1054012},{"utcDay":18070,"count":1054677},{"utcDay":18071,"count":1055388},{"utcDay":18072,"count":1056074},{"utcDay":18073,"count":1057676},{"utcDay":18074,"count":1058858},{"utcDay":18075,"count":1059503},{"utcDay":18076,"count":1060307},{"utcDay":18077,"count":1060874},{"utcDay":18078,"count":1061539},{"utcDay":18079,"count":1062678},{"utcDay":18080,"count":1063872},{"utcDay":18081,"count":1064927},{"utcDay":18082,"count":1065907},{"utcDay":18083,"count":1066996},{"utcDay":18084,"count":1067990},{"utcDay":18085,"count":1069141},{"utcDay":18086,"count":1070278},{"utcDay":18087,"count":1071459},{"utcDay":18088,"count":1072623},{"utcDay":18089,"count":1073868},{"utcDay":18090,"count":1075011},{"utcDay":18091,"count":1076624},{"utcDay":18092,"count":1077833},{"utcDay":18093,"count":1079335},{"utcDay":18094,"count":1080717},{"utcDay":18095,"count":1081931},{"utcDay":18096,"count":1082901},{"utcDay":18097,"count":1083778},{"utcDay":18098,"count":1084878},{"utcDay":18099,"count":1086083},{"utcDay":18100,"count":1087115},{"utcDay":18101,"count":1088626},{"utcDay":18102,"count":1089829},{"utcDay":18103,"count":1090891},{"utcDay":18104,"count":1092004},{"utcDay":18105,"count":1092979},{"utcDay":18106,"count":1093964},{"utcDay":18107,"count":1095061},{"utcDay":18108,"count":1096208},{"utcDay":18109,"count":1097281},{"utcDay":18110,"count":1098196},{"utcDay":18111,"count":1098971},{"utcDay":18112,"count":1100045},{"utcDay":18113,"count":1101172},{"utcDay":18114,"count":1102334},{"utcDay":18115,"count":1103516},{"utcDay":18116,"count":1104614},{"utcDay":18117,"count":1105627},{"utcDay":18118,"count":1106453},{"utcDay":18119,"count":1107388},{"utcDay":18120,"count":1108411},{"utcDay":18121,"count":1109518},{"utcDay":18122,"count":1110524},{"utcDay":18123,"count":1111426},{"utcDay":18124,"count":1112358},{"utcDay":18125,"count":1113418},{"utcDay":18126,"count":1114462},{"utcDay":18127,"count":1115578},{"utcDay":18128,"count":1116552},{"utcDay":18129,"count":1117397},{"utcDay":18130,"count":1118453},{"utcDay":18131,"count":1119275},{"utcDay":18132,"count":1120204},{"utcDay":18133,"count":1121238},{"utcDay":18134,"count":1122240},{"utcDay":18135,"count":1123206},{"utcDay":18136,"count":1124098},{"utcDay":18137,"count":1125143},{"utcDay":18138,"count":1126118},{"utcDay":18139,"count":1126824},{"utcDay":18140,"count":1127608},{"utcDay":18141,"count":1128439},{"utcDay":18142,"count":1129395},{"utcDay":18143,"count":1130370},{"utcDay":18144,"count":1131145},{"utcDay":18145,"count":1132020},{"utcDay":18146,"count":1132834},{"utcDay":18147,"count":1133807},{"utcDay":18148,"count":1134657},{"utcDay":18149,"count":1135581},{"utcDay":18150,"count":1136415},{"utcDay":18151,"count":1137282},{"utcDay":18152,"count":1138036},{"utcDay":18153,"count":1138827},{"utcDay":18154,"count":1139696},{"utcDay":18155,"count":1140504},{"utcDay":18156,"count":1141395},{"utcDay":18157,"count":1142197},{"utcDay":18158,"count":1142906},{"utcDay":18159,"count":1143781},{"utcDay":18160,"count":1144571},{"utcDay":18161,"count":1145377},{"utcDay":18162,"count":1146257},{"utcDay":18163,"count":1147219},{"utcDay":18164,"count":1148202},{"utcDay":18165,"count":1149010},{"utcDay":18166,"count":1149846},{"utcDay":18167,"count":1150665},{"utcDay":18168,"count":1151512},{"utcDay":18169,"count":1152212},{"utcDay":18170,"count":1152976},{"utcDay":18171,"count":1153730},{"utcDay":18172,"count":1154503},{"utcDay":18173,"count":1155288},{"utcDay":18174,"count":1155994},{"utcDay":18175,"count":1156700},{"utcDay":18176,"count":1157507},{"utcDay":18177,"count":1158189},{"utcDay":18178,"count":1159035},{"utcDay":18179,"count":1159748},{"utcDay":18180,"count":1160335},{"utcDay":18181,"count":1160859},{"utcDay":18182,"count":1161591},{"utcDay":18183,"count":1162301},{"utcDay":18184,"count":1162999},{"utcDay":18185,"count":1163714},{"utcDay":18186,"count":1164519},{"utcDay":18187,"count":1165255},{"utcDay":18188,"count":1165984},{"utcDay":18189,"count":1166727},{"utcDay":18190,"count":1167526},{"utcDay":18191,"count":1168353},{"utcDay":18192,"count":1169337},{"utcDay":18193,"count":1170057},{"utcDay":18194,"count":1170750},{"utcDay":18195,"count":1171585},{"utcDay":18196,"count":1172513},{"utcDay":18197,"count":1173346},{"utcDay":18198,"count":1174129},{"utcDay":18199,"count":1174934},{"utcDay":18200,"count":1175699},{"utcDay":18201,"count":1176373},{"utcDay":18202,"count":1177062},{"utcDay":18203,"count":1178036},{"utcDay":18204,"count":1178846},{"utcDay":18205,"count":1179825},{"utcDay":18206,"count":1180598},{"utcDay":18207,"count":1181365},{"utcDay":18208,"count":1182083},{"utcDay":18209,"count":1182719},{"utcDay":18210,"count":1183510},{"utcDay":18211,"count":1184220},{"utcDay":18212,"count":1184994},{"utcDay":18213,"count":1185748},{"utcDay":18214,"count":1186529},{"utcDay":18215,"count":1187257},{"utcDay":18216,"count":1187902},{"utcDay":18217,"count":1188589},{"utcDay":18218,"count":1189229},{"utcDay":18219,"count":1189963},{"utcDay":18220,"count":1190745},{"utcDay":18221,"count":1191698},{"utcDay":18222,"count":1192665},{"utcDay":18223,"count":1193605},{"utcDay":18224,"count":1194670},{"utcDay":18225,"count":1195608},{"utcDay":18226,"count":1196782},{"utcDay":18227,"count":1197602},{"utcDay":18228,"count":1198513},{"utcDay":18229,"count":1199457},{"utcDay":18230,"count":1200399},{"utcDay":18231,"count":1201464},{"utcDay":18232,"count":1202493},{"utcDay":18233,"count":1203503},{"utcDay":18234,"count":1204496},{"utcDay":18235,"count":1205467},{"utcDay":18236,"count":1206291},{"utcDay":18237,"count":1207245},{"utcDay":18238,"count":1208082},{"utcDay":18239,"count":1209217},{"utcDay":18240,"count":1210022},{"utcDay":18241,"count":1210939},{"utcDay":18242,"count":1212147},{"utcDay":18243,"count":1213199},{"utcDay":18244,"count":1214079},{"utcDay":18245,"count":1215040},{"utcDay":18246,"count":1216118},{"utcDay":18247,"count":1217020},{"utcDay":18248,"count":1217957},{"utcDay":18249,"count":1219018},{"utcDay":18250,"count":1219980},{"utcDay":18251,"count":1220901},{"utcDay":18252,"count":1221869},{"utcDay":18253,"count":1222884},{"utcDay":18254,"count":1223756},{"utcDay":18255,"count":1224727},{"utcDay":18256,"count":1225704},{"utcDay":18257,"count":1226784},{"utcDay":18258,"count":1227694},{"utcDay":18259,"count":1228944},{"utcDay":18260,"count":1230326},{"utcDay":18261,"count":1231271},{"utcDay":18262,"count":1232356},{"utcDay":18263,"count":1233586},{"utcDay":18264,"count":1234726},{"utcDay":18265,"count":1235839},{"utcDay":18266,"count":1236964},{"utcDay":18267,"count":1238457},{"utcDay":18268,"count":1239874},{"utcDay":18269,"count":1241370},{"utcDay":18270,"count":1242643},{"utcDay":18271,"count":1243724},{"utcDay":18272,"count":1244801},{"utcDay":18273,"count":1245902},{"utcDay":18274,"count":1247056},{"utcDay":18275,"count":1248131},{"utcDay":18276,"count":1249325},{"utcDay":18277,"count":1250371},{"utcDay":18278,"count":1251299},{"utcDay":18279,"count":1252405},{"utcDay":18280,"count":1253699},{"utcDay":18281,"count":1254803},{"utcDay":18282,"count":1255972},{"utcDay":18283,"count":1256985},{"utcDay":18284,"count":1258048},{"utcDay":18285,"count":1259275},{"utcDay":18286,"count":1260200},{"utcDay":18287,"count":1261160},{"utcDay":18288,"count":1262144},{"utcDay":18289,"count":1263211},{"utcDay":18290,"count":1264342},{"utcDay":18291,"count":1265547},{"utcDay":18292,"count":1266602},{"utcDay":18293,"count":1267536},{"utcDay":18294,"count":1267734},{"utcDay":18295,"count":1269863},{"utcDay":18296,"count":1270893},{"utcDay":18297,"count":1272025},{"utcDay":18298,"count":1273174},{"utcDay":18299,"count":1274350},{"utcDay":18300,"count":1275420},{"utcDay":18301,"count":1276789},{"utcDay":18302,"count":1277890},{"utcDay":18303,"count":1279138},{"utcDay":18304,"count":1280255},{"utcDay":18305,"count":1281524},{"utcDay":18306,"count":1282565},{"utcDay":18307,"count":1283590},{"utcDay":18308,"count":1284862},{"utcDay":18309,"count":1286040},{"utcDay":18310,"count":1287225},{"utcDay":18311,"count":1288474},{"utcDay":18312,"count":1289587},{"utcDay":18313,"count":1290890},{"utcDay":18314,"count":1292024},{"utcDay":18315,"count":1293227},{"utcDay":18316,"count":1294421},{"utcDay":18317,"count":1295487},{"utcDay":18318,"count":1296685},{"utcDay":18319,"count":1297988},{"utcDay":18320,"count":1299160},{"utcDay":18321,"count":1300300},{"utcDay":18322,"count":1301727},{"utcDay":18323,"count":1302848},{"utcDay":18324,"count":1304166},{"utcDay":18325,"count":1305273},{"utcDay":18326,"count":1306502},{"utcDay":18327,"count":1307576},{"utcDay":18328,"count":1308738},{"utcDay":18329,"count":1309953},{"utcDay":18330,"count":1311243},{"utcDay":18331,"count":1312482},{"utcDay":18332,"count":1313426},{"utcDay":18333,"count":1314897},{"utcDay":18334,"count":1316787},{"utcDay":18335,"count":1318195},{"utcDay":18336,"count":1319830},{"utcDay":18337,"count":1321288},{"utcDay":18338,"count":1322569},{"utcDay":18339,"count":1324115},{"utcDay":18340,"count":1325713},{"utcDay":18341,"count":1327210},{"utcDay":18342,"count":1328698},{"utcDay":18343,"count":1330328},{"utcDay":18344,"count":1331833},{"utcDay":18345,"count":1333403},{"utcDay":18346,"count":1335252},{"utcDay":18347,"count":1337105},{"utcDay":18348,"count":1338981},{"utcDay":18349,"count":1340651},{"utcDay":18350,"count":1342277},{"utcDay":18351,"count":1343712},{"utcDay":18352,"count":1345336},{"utcDay":18353,"count":1346894},{"utcDay":18354,"count":1348396},{"utcDay":18355,"count":1350068},{"utcDay":18356,"count":1351612},{"utcDay":18357,"count":1353356},{"utcDay":18358,"count":1355115},{"utcDay":18359,"count":1356908},{"utcDay":18360,"count":1358250},{"utcDay":18361,"count":1359871},{"utcDay":18362,"count":1361772},{"utcDay":18363,"count":1363371},{"utcDay":18364,"count":1364870},{"utcDay":18365,"count":1366829},{"utcDay":18366,"count":1368632},{"utcDay":18367,"count":1370519},{"utcDay":18368,"count":1371995},{"utcDay":18369,"count":1373366},{"utcDay":18370,"count":1374776},{"utcDay":18371,"count":1376668},{"utcDay":18372,"count":1378102},{"utcDay":18373,"count":1379842},{"utcDay":18374,"count":1381048},{"utcDay":18375,"count":1382413},{"utcDay":18376,"count":1383882},{"utcDay":18377,"count":1385355},{"utcDay":18378,"count":1386969},{"utcDay":18379,"count":1388654},{"utcDay":18380,"count":1390083},{"utcDay":18381,"count":1391773},{"utcDay":18382,"count":1393907},{"utcDay":18383,"count":1395574},{"utcDay":18384,"count":1397105},{"utcDay":18385,"count":1398713},{"utcDay":18386,"count":1400463},{"utcDay":18387,"count":1402237},{"utcDay":18388,"count":1404015},{"utcDay":18389,"count":1405989},{"utcDay":18390,"count":1407860},{"utcDay":18391,"count":1409598},{"utcDay":18392,"count":1411659},{"utcDay":18393,"count":1414026},{"utcDay":18394,"count":1416318},{"utcDay":18395,"count":1418274},{"utcDay":18396,"count":1419969},{"utcDay":18397,"count":1421935},{"utcDay":18398,"count":1423766},{"utcDay":18399,"count":1425591},{"utcDay":18400,"count":1427333},{"utcDay":18401,"count":1428986},{"utcDay":18402,"count":1430660},{"utcDay":18403,"count":1432308},{"utcDay":18404,"count":1433902},{"utcDay":18405,"count":1435323},{"utcDay":18406,"count":1436930},{"utcDay":18407,"count":1438577},{"utcDay":18408,"count":1440273},{"utcDay":18409,"count":1441841},{"utcDay":18410,"count":1443148},{"utcDay":18411,"count":1444775},{"utcDay":18412,"count":1445872},{"utcDay":18413,"count":1447319},{"utcDay":18414,"count":1449029},{"utcDay":18415,"count":1450672},{"utcDay":18416,"count":1452229},{"utcDay":18417,"count":1453921},{"utcDay":18418,"count":1455633},{"utcDay":18419,"count":1457294},{"utcDay":18420,"count":1458983},{"utcDay":18421,"count":1460533},{"utcDay":18422,"count":1462017},{"utcDay":18423,"count":1463303},{"utcDay":18424,"count":1464721},{"utcDay":18425,"count":1466309},{"utcDay":18426,"count":1467724},{"utcDay":18427,"count":1469141},{"utcDay":18428,"count":1470456},{"utcDay":18429,"count":1471908},{"utcDay":18430,"count":1473311},{"utcDay":18431,"count":1474512},{"utcDay":18432,"count":1475595},{"utcDay":18433,"count":1476888},{"utcDay":18434,"count":1478180},{"utcDay":18435,"count":1479595},{"utcDay":18436,"count":1481138},{"utcDay":18437,"count":1482470},{"utcDay":18438,"count":1483887},{"utcDay":18439,"count":1485544},{"utcDay":18440,"count":1486723},{"utcDay":18441,"count":1488168},{"utcDay":18442,"count":1489617},{"utcDay":18443,"count":1490912},{"utcDay":18444,"count":1492264},{"utcDay":18445,"count":1493580},{"utcDay":18446,"count":1494707},{"utcDay":18447,"count":1496048},{"utcDay":18448,"count":1497229},{"utcDay":18449,"count":1498491},{"utcDay":18450,"count":1500006},{"utcDay":18451,"count":1501294},{"utcDay":18452,"count":1502671},{"utcDay":18453,"count":1504133},{"utcDay":18454,"count":1505750},{"utcDay":18455,"count":1507579},{"utcDay":18456,"count":1509419},{"utcDay":18457,"count":1510520},{"utcDay":18458,"count":1512181},{"utcDay":18459,"count":1513764},{"utcDay":18460,"count":1515187},{"utcDay":18461,"count":1516488},{"utcDay":18462,"count":1517820},{"utcDay":18463,"count":1519028},{"utcDay":18464,"count":1520374},{"utcDay":18465,"count":1521863},{"utcDay":18466,"count":1523516},{"utcDay":18467,"count":1524886},{"utcDay":18468,"count":1526343},{"utcDay":18469,"count":1528234},{"utcDay":18470,"count":1530032},{"utcDay":18471,"count":1532101},{"utcDay":18472,"count":1533860},{"utcDay":18473,"count":1535368},{"utcDay":18474,"count":1537124},{"utcDay":18475,"count":1538783},{"utcDay":18476,"count":1540949},{"utcDay":18477,"count":1542545},{"utcDay":18478,"count":1544659},{"utcDay":18479,"count":1546384},{"utcDay":18480,"count":1548650},{"utcDay":18481,"count":1551459},{"utcDay":18482,"count":1553048},{"utcDay":18483,"count":1554768},{"utcDay":18484,"count":1556666},{"utcDay":18485,"count":1559342},{"utcDay":18486,"count":1561869},{"utcDay":18487,"count":1564337},{"utcDay":18488,"count":1566608},{"utcDay":18489,"count":1569144},{"utcDay":18490,"count":1571725},{"utcDay":18491,"count":1574473},{"utcDay":18492,"count":1577008},{"utcDay":18493,"count":1579740},{"utcDay":18494,"count":1582392},{"utcDay":18495,"count":1584952},{"utcDay":18496,"count":1587752},{"utcDay":18497,"count":1590769},{"utcDay":18498,"count":1593389},{"utcDay":18499,"count":1595780},{"utcDay":18500,"count":1598308},{"utcDay":18501,"count":1601217},{"utcDay":18502,"count":1603655},{"utcDay":18503,"count":1606086},{"utcDay":18504,"count":1608307},{"utcDay":18505,"count":1610119},{"utcDay":18506,"count":1613014},{"utcDay":18507,"count":1615005},{"utcDay":18508,"count":1617161},{"utcDay":18509,"count":1619364},{"utcDay":18510,"count":1621506},{"utcDay":18511,"count":1623804},{"utcDay":18512,"count":1626323},{"utcDay":18513,"count":1628589},{"utcDay":18514,"count":1630579},{"utcDay":18515,"count":1632106},{"utcDay":18516,"count":1634373},{"utcDay":18520,"count":1642015},{"utcDay":18523,"count":1650562},{"utcDay":18527,"count":1657686},{"utcDay":18530,"count":1663864},{"utcDay":18534,"count":1670135},{"utcDay":18535,"count":1671591},{"utcDay":18536,"count":1672813},{"utcDay":18537,"count":1674098},{"utcDay":18538,"count":1675623},{"utcDay":18539,"count":1676925},{"utcDay":18540,"count":1678402},{"utcDay":18541,"count":1679738},{"utcDay":18542,"count":1681343},{"utcDay":18543,"count":1682976},{"utcDay":18544,"count":1684759},{"utcDay":18545,"count":1686516},{"utcDay":18546,"count":1688437},{"utcDay":18547,"count":1690640},{"utcDay":18548,"count":1692934},{"utcDay":18549,"count":1695042},{"utcDay":18550,"count":1696936},{"utcDay":18551,"count":1698813},{"utcDay":18552,"count":1700691},{"utcDay":18553,"count":1702820},{"utcDay":18554,"count":1705017},{"utcDay":18555,"count":1707136},{"utcDay":18556,"count":1709619},{"utcDay":18557,"count":1711858},{"utcDay":18558,"count":1713831},{"utcDay":18559,"count":1716056},{"utcDay":18560,"count":1718317},{"utcDay":18561,"count":1719570},{"utcDay":18563,"count":1723431},{"utcDay":18566,"count":1729717},{"utcDay":18569,"count":1734824},{"utcDay":18570,"count":1736346},{"utcDay":18571,"count":1739556},{"utcDay":18572,"count":1741325},{"utcDay":18573,"count":1742967},{"utcDay":18574,"count":1744536},{"utcDay":18575,"count":1746406},{"utcDay":18576,"count":1747962},{"utcDay":18577,"count":1749907},{"utcDay":18578,"count":1751590},{"utcDay":18579,"count":1753432},{"utcDay":18580,"count":1754980},{"utcDay":18581,"count":1756641},{"utcDay":18582,"count":1758801},{"utcDay":18583,"count":1760738},{"utcDay":18584,"count":1763040},{"utcDay":18585,"count":1764955},{"utcDay":18586,"count":1766872},{"utcDay":18587,"count":1768982},{"utcDay":18588,"count":1771127},{"utcDay":18589,"count":1773187},{"utcDay":18590,"count":1775413},{"utcDay":18591,"count":1777499},{"utcDay":18592,"count":1779788},{"utcDay":18593,"count":1781746},{"utcDay":18594,"count":1783712},{"utcDay":18595,"count":1785640},{"utcDay":18596,"count":1787647},{"utcDay":18597,"count":1789905},{"utcDay":18598,"count":1790049},{"utcDay":18599,"count":1793807},{"utcDay":18600,"count":1793862},{"utcDay":18601,"count":1797162},{"utcDay":18602,"count":1797396},{"utcDay":18603,"count":1800791},{"utcDay":18605,"count":1805883},{"utcDay":18608,"count":1809847},{"utcDay":18610,"count":1813233},{"utcDay":18612,"count":1819805},{"utcDay":18614,"count":1824295},{"utcDay":18617,"count":1833112},{"utcDay":18618,"count":1835635},{"utcDay":18619,"count":1838330},{"utcDay":18620,"count":1840647},{"utcDay":18621,"count":1843115},{"utcDay":18622,"count":1846079},{"utcDay":18623,"count":1849974},{"utcDay":18624,"count":1853238},{"utcDay":18625,"count":1856341},{"utcDay":18626,"count":1859460},{"utcDay":18627,"count":1862253},{"utcDay":18628,"count":1865036},{"utcDay":18629,"count":1870663},{"utcDay":18630,"count":1877757},{"utcDay":18631,"count":1883120},{"utcDay":18632,"count":1887395},{"utcDay":18633,"count":1891522},{"utcDay":18634,"count":1897127},{"utcDay":18635,"count":1902854},{"utcDay":18636,"count":1907288},{"utcDay":18637,"count":1912618},{"utcDay":18638,"count":1919491},{"utcDay":18639,"count":1923708},{"utcDay":18640,"count":1927461},{"utcDay":18641,"count":1931173},{"utcDay":18642,"count":1934367},{"utcDay":18645,"count":1943545},{"utcDay":18651,"count":1962419},{"utcDay":18655,"count":2053940},{"utcDay":18656,"count":2120504},{"utcDay":18661,"count":2285825},{"utcDay":18662,"count":2298606},{"utcDay":18663,"count":2309301},{"utcDay":18664,"count":2319901},{"utcDay":18665,"count":2330617},{"utcDay":18666,"count":2346437},{"utcDay":18667,"count":2363683},{"utcDay":18668,"count":2376146},{"utcDay":18669,"count":2388268},{"utcDay":18670,"count":2396997},{"utcDay":18671,"count":2405424},{"utcDay":18672,"count":2414009},{"utcDay":18673,"count":2421772},{"utcDay":18674,"count":2429327},{"utcDay":18675,"count":2437103},{"utcDay":18676,"count":2444944},{"utcDay":18677,"count":2452316},{"utcDay":18678,"count":2460646},{"utcDay":18679,"count":2468651},{"utcDay":18680,"count":2477261},{"utcDay":18681,"count":2485837},{"utcDay":18682,"count":2492166},{"utcDay":18683,"count":2498988},{"utcDay":18684,"count":2507988},{"utcDay":18685,"count":2514105},{"utcDay":18686,"count":2519535},{"utcDay":18687,"count":2524620},{"utcDay":18688,"count":2530102},{"utcDay":18689,"count":2535141},{"utcDay":18690,"count":2539171},{"utcDay":18691,"count":2542640},{"utcDay":18692,"count":2545972},{"utcDay":18693,"count":2549628},{"utcDay":18694,"count":2553255},{"utcDay":18695,"count":2557354},{"utcDay":18696,"count":2561584},{"utcDay":18697,"count":2565680},{"utcDay":18698,"count":2569576},{"utcDay":18699,"count":2574541},{"utcDay":18700,"count":2579399},{"utcDay":18701,"count":2584005},{"utcDay":18702,"count":2588771},{"utcDay":18703,"count":2593598},{"utcDay":18704,"count":2598086},{"utcDay":18705,"count":2601803},{"utcDay":18706,"count":2605657},{"utcDay":18707,"count":2609790},{"utcDay":18708,"count":2613714},{"utcDay":18709,"count":2617556},{"utcDay":18710,"count":2621718},{"utcDay":18711,"count":2625806},{"utcDay":18712,"count":2629170},{"utcDay":18713,"count":2632308},{"utcDay":18714,"count":2635646},{"utcDay":18715,"count":2639206},{"utcDay":18716,"count":2642543},{"utcDay":18717,"count":2646154},{"utcDay":18718,"count":2649393},{"utcDay":18719,"count":2652736},{"utcDay":18720,"count":2655956},{"utcDay":18721,"count":2659367},{"utcDay":18722,"count":2662987},{"utcDay":18723,"count":2666408},{"utcDay":18724,"count":2670029},{"utcDay":18725,"count":2673179},{"utcDay":18726,"count":2675991},{"utcDay":18727,"count":2679191},{"utcDay":18728,"count":2682605},{"utcDay":18729,"count":2685988},{"utcDay":18730,"count":2689804},{"utcDay":18731,"count":2694649},{"utcDay":18732,"count":2699255},{"utcDay":18733,"count":2706437},{"utcDay":18734,"count":2713173},{"utcDay":18735,"count":2722481},{"utcDay":18736,"count":2730257},{"utcDay":18737,"count":2737694},{"utcDay":18738,"count":2744226},{"utcDay":18739,"count":2750722},{"utcDay":18740,"count":2758144},{"utcDay":18741,"count":2763296},{"utcDay":18742,"count":2768900},{"utcDay":18743,"count":2774006},{"utcDay":18744,"count":2779052},{"utcDay":18745,"count":2783873},{"utcDay":18746,"count":2788350},{"utcDay":18747,"count":2791961},{"utcDay":18748,"count":2796716},{"utcDay":18749,"count":2801902},{"utcDay":18750,"count":2807218},{"utcDay":18751,"count":2813206},{"utcDay":18752,"count":2819744},{"utcDay":18753,"count":2826037},{"utcDay":18754,"count":2832073},{"utcDay":18755,"count":2838728},{"utcDay":18756,"count":2845825},{"utcDay":18757,"count":2852822},{"utcDay":18758,"count":2859794},{"utcDay":18759,"count":2866521},{"utcDay":18760,"count":2875203},{"utcDay":18761,"count":2883711},{"utcDay":18762,"count":2891553},{"utcDay":18763,"count":2900366},{"utcDay":18764,"count":2909842},{"utcDay":18765,"count":2917186},{"utcDay":18766,"count":2927676},{"utcDay":18767,"count":2935394},{"utcDay":18768,"count":2942539},{"utcDay":18769,"count":2949287},{"utcDay":18770,"count":2956750},{"utcDay":18771,"count":2964867},{"utcDay":18772,"count":2973061},{"utcDay":18773,"count":2979765},{"utcDay":18774,"count":2985871},{"utcDay":18775,"count":2992058},{"utcDay":18776,"count":2997326},{"utcDay":18777,"count":3002486},{"utcDay":18778,"count":3007385},{"utcDay":18779,"count":3012220},{"utcDay":18780,"count":3018898},{"utcDay":18781,"count":3027433},{"utcDay":18782,"count":3033514},{"utcDay":18783,"count":3039064},{"utcDay":18784,"count":3044565},{"utcDay":18785,"count":3050122},{"utcDay":18786,"count":3057154},{"utcDay":18787,"count":3064308},{"utcDay":18788,"count":3070277},{"utcDay":18789,"count":3075135},{"utcDay":18790,"count":3079413},{"utcDay":18791,"count":3083694},{"utcDay":18792,"count":3088366},{"utcDay":18793,"count":3092828},{"utcDay":18794,"count":3096923},{"utcDay":18795,"count":3100856},{"utcDay":18796,"count":3104551},{"utcDay":18797,"count":3108028},{"utcDay":18798,"count":3111568},{"utcDay":18799,"count":3115973},{"utcDay":18800,"count":3120637},{"utcDay":18801,"count":3124751},{"utcDay":18802,"count":3130469},{"utcDay":18803,"count":3134487},{"utcDay":18804,"count":3138002},{"utcDay":18805,"count":3141378},{"utcDay":18806,"count":3144683},{"utcDay":18807,"count":3147932},{"utcDay":18808,"count":3150916},{"utcDay":18809,"count":3153984},{"utcDay":18810,"count":3156703},{"utcDay":18811,"count":3159620},{"utcDay":18812,"count":3162498},{"utcDay":18813,"count":3165334},{"utcDay":18814,"count":3168249},{"utcDay":18815,"count":3171103},{"utcDay":18816,"count":3173914},{"utcDay":18817,"count":3176431},{"utcDay":18818,"count":3179052},{"utcDay":18819,"count":3181652},{"utcDay":18820,"count":3184169},{"utcDay":18821,"count":3186700},{"utcDay":18822,"count":3189197},{"utcDay":18823,"count":3191531},{"utcDay":18824,"count":3193886},{"utcDay":18825,"count":3196104},{"utcDay":18826,"count":3198500},{"utcDay":18827,"count":3201002},{"utcDay":18828,"count":3203503},{"utcDay":18829,"count":3206125},{"utcDay":18830,"count":3208539},{"utcDay":18831,"count":3210815},{"utcDay":18832,"count":3213052},{"utcDay":18833,"count":3215474},{"utcDay":18834,"count":3218658},{"utcDay":18835,"count":3221544},{"utcDay":18836,"count":3224365},{"utcDay":18837,"count":3226689},{"utcDay":18838,"count":3228805},{"utcDay":18839,"count":3230903},{"utcDay":18840,"count":3233227},{"utcDay":18841,"count":3235481},{"utcDay":18842,"count":3237806},{"utcDay":18843,"count":3240238},{"utcDay":18844,"count":3242699},{"utcDay":18845,"count":3244996},{"utcDay":18846,"count":3247611},{"utcDay":18847,"count":3250347},{"utcDay":18848,"count":3253069},{"utcDay":18849,"count":3255954},{"utcDay":18850,"count":3258641},{"utcDay":18851,"count":3261363},{"utcDay":18852,"count":3264058},{"utcDay":18853,"count":3266489},{"utcDay":18854,"count":3269142},{"utcDay":18855,"count":3271975},{"utcDay":18856,"count":3274984},{"utcDay":18857,"count":3277838},{"utcDay":18858,"count":3278527}]'
    subscriber_array_json = json.loads(subscriber_array)
    
    #补上缺失的数据
    missing_date = [17792, 17793, 17794, 17795, 17796, 17797, 18562, 18564, 18565, 18567, 18568, 18650, 18648, 18652, 18604, 18606, 18607, 18609, 18611, 18613, 17590, 17591, 17592, 17593, 17594, 17595, 17596, 17597, 17598, 17599, 17600, 17601, 17602, 17603, 18643, 18644, 18517, 18518, 18519, 18646, 18521, 18522, 18615, 18524, 18525, 18526, 18616, 18528, 18529, 18649, 18531, 18532, 18533, 18653, 18654, 18657, 18658, 18659, 18660, 17779, 18647, 17785, 17786, 17787, 17788, 17789, 17790, 17791]
    subscriber_array_padding = []
    for utcday in missing_date:
        subscriber_array_padding.append({'utcDay':utcday,'count':0})
    subscriber_array_json = subscriber_array_json + subscriber_array_padding
    subscriber_array_json.sort(key=utcday_sortrule)

    subscriber_date = []
    subscriber_date_show = []
    subscriber_count_show = []
    subscriber_year_incre_ratio = []

    #subscriber_date_show[0]对应2012-10-29  subscriber_date_show[181] 对应2013-04-28
    for x in subscriber_array_json:
        timeArray = time.localtime((x['utcDay']-15642)*86400 + 1351468800)
        time_show = time.strftime("%Y-%m-%d", timeArray)
        subscriber_date.append(x['utcDay'])
        subscriber_date_show.append(time_show)
        subscriber_count_show.append(x['count'])

    for index in range(len(subscriber_count_show)):
        if index<7:
            subscriber_year_incre_ratio.append(0)
        else:
            num_a = subscriber_count_show[index]
            num_b = subscriber_count_show[index-7]
            if num_a == 0 or num_b == 0:
                #给个差不多的60
                subscriber_year_incre_ratio.append(60)
            else:
                #计算7天增长人数
                num = (num_a - num_b)/7
                ratio = round((math.pow(num/num_b+1,365)-1)*100,2)

                subscriber_year_incre_ratio.append(ratio)  
    
    return subscriber_year_incre_ratio,subscriber_date_show,subscriber_count_show,subscriber_date

#排序规则
def utcday_sortrule(elem):
    return elem['utcDay']
        
print(get_subscriber_year_incre_ratio()[0])
# print(len(get_subscriber_year_incre_ratio()[1]))
# print(get_subscriber_year_incre_ratio()[1])
# print(get_subscriber_year_incre_ratio()[3])
# get_subscriber_year_incre_ratio()

[0, 0, 0, 0, 0, 0, 0, 79.01, 74.77, 84.62, 88.63, 116.29, 109.2, 118.81, 224.84, 268.52, 279.52, 269.6, 239.06, 247.92, 230.29, 117.59, 106.63, 89.59, 81.91, 80.07, 83.92, 85.44, 90.19, 88.22, 127.16, 134.94, 139.71, 140.36, 168.87, 176.55, 181.94, 160.84, 174.08, 189.11, 187.27, 188.73, 189.11, 331.42, 367.55, 346.07, 327.35, 348.2, 401.28, 413.79, 220.96, 172.03, 160.5, 156.68, 147.79, 114.77, 117.73, 121.04, 112.08, 112.71, 99.1, 92.53, 82.43, 66.08, 60.09, 93.77, 110.43, 115.33, 126.46, 137.14, 151.23, 172.68, 161.1, 145.48, 159.96, 155.35, 146.92, 149.17, 143.83, 134.83, 143.64, 133.93, 136.22, 143.14, 139.74, 192.75, 219.78, 258.47, 263.08, 285.97, 278.92, 324.02, 302.07, 307.21, 288.41, 332.57, 341.63, 389.5, 337.06, 306.51, 292.48, 273.92, 256.27, 244.59, 268.11, 310.85, 349.7, 438.74, 1141.88, 1948.74, 2199.37, 2161.36, 2112.78, 2387.7, 2156.64, 987.46, 716.69, 663.94, 696.72, 661.8, 561.8, 522.31, 553.1, 458.81, 453.22, 467.02, 559.01, 717.29, 1115.49, 1799.76, 2162.03, 3685.

In [17]:
#过滤缺失的日期 字符串形式
def filter_missdatestr(data):
    
    fulldata = []
    for a in range(3216):
        fulldata.append(time.strftime("%Y-%m-%d",time.localtime(1351468800 + 86400*a)))
    minus = list(set(fulldata) - set(data))
    return minus
#过滤缺失的日期 utcDay形式
def filter_missdateutc(data):
    fulldata = []
    for a in range(3216):
        fulldata.append(15642+a)
    minus = list(set(fulldata) - set(data))
    return minus
# print(filter_missdateutc(get_subscriber_year_incre_ratio()[3]))
# get_subscriber_year_incre_ratio()[0]
# filter_missdateutc(get_subscriber_year_incre_ratio()[3])
filter_missdatestr(get_subscriber_year_incre_ratio()[1])


[17792, 17793, 17794, 17795, 17796, 17797, 18562, 18564, 18565, 18567, 18568, 18650, 18648, 18652, 18604, 18606, 18607, 18609, 18611, 18613, 17590, 17591, 17592, 17593, 17594, 17595, 17596, 17597, 17598, 17599, 17600, 17601, 17602, 17603, 18643, 18644, 18517, 18518, 18519, 18646, 18521, 18522, 18615, 18524, 18525, 18526, 18616, 18528, 18529, 18649, 18531, 18532, 18533, 18653, 18654, 18657, 18658, 18659, 18660, 17779, 18647, 17785, 17786, 17787, 17788, 17789, 17790, 17791]


['2021-02-01',
 '2021-01-25',
 '2021-01-20',
 '2020-09-23',
 '2021-01-22',
 '2020-09-12',
 '2018-09-11',
 '2020-12-10',
 '2020-10-30',
 '2020-09-14',
 '2018-03-12',
 '2021-01-31',
 '2021-01-19',
 '2020-10-27',
 '2020-09-28',
 '2020-12-11',
 '2018-09-13',
 '2021-01-23',
 '2021-01-21',
 '2020-09-24',
 '2020-09-13',
 '2018-03-02',
 '2020-11-01',
 '2020-09-20',
 '2018-09-23',
 '2018-03-06',
 '2018-03-11',
 '2020-09-21',
 '2021-01-16',
 '2018-03-05',
 '2020-12-17',
 '2020-09-16',
 '2018-09-05',
 '2021-01-30',
 '2020-12-13',
 '2021-02-02',
 '2020-09-19',
 '2018-09-19',
 '2018-03-09',
 '2018-09-20',
 '2018-03-07',
 '2018-09-21',
 '2020-10-29',
 '2018-09-17',
 '2018-09-12',
 '2018-09-14',
 '2020-09-27',
 '2020-09-26',
 '2020-12-19',
 '2018-09-18',
 '2018-03-01',
 '2020-12-08',
 '2018-03-08',
 '2021-01-17',
 '2018-03-04',
 '2018-03-10',
 '2021-01-26',
 '2021-01-27',
 '2020-12-15',
 '2018-02-28',
 '2018-03-03',
 '2020-11-02',
 '2018-03-13',
 '2018-09-16',
 '2018-09-15',
 '2020-09-17',
 '2020-12-

In [37]:
#计算粉丝年化域面积
def subscriber_dominarea(start_date,end_date):
    #年化大于1000的数据作为杂数据剔除
    
    subscriber_year_incre_ratio = get_subscriber_year_incre_ratio()[0]
    subscriber_date_show = get_subscriber_year_incre_ratio()[1]
    dictionary = dict(zip(subscriber_date_show, subscriber_year_incre_ratio))
    area = 0
    contain_dates = []
    start_timestamp = timestr_to_timestamp(start_date)
    end_timestamp = timestr_to_timestamp(end_date)
    for i in range(int((end_timestamp - start_timestamp)/86400)):
        contain_dates.append(timestamp_to_timestr(start_timestamp + i*86400))
        
    for date in contain_dates:
        if dictionary[date]>2000:
            area = area + 2000
        else:
            area = area + dictionary[date]
    return area
    

In [40]:
print(subscriber_dominarea('2012-11-05','2013-04-09')+subscriber_dominarea('2013-11-03','2013-11-29'))
print(subscriber_dominarea('2012-11-05','2013-11-29'))
print(subscriber_dominarea('2017-04-15','2017-12-15'))
print(subscriber_dominarea('2020-12-14','2021-04-12'))

149612.49
203885.6599999999
93723.01000000001
18807.089999999997


In [ ]:
from flask import Flask
app = Flask(__name__)
@app.route('/bitcoin')
def index():
#     return 'bitcoin'
    return btc_data
if __name__ == '__main__':
#     app.debug = True # 设置调试模式，生产模式的时候要关掉debug
    app.run(port='8891')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8891/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Aug/2021 14:40:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [17/Aug/2021 14:40:40] "GET /bitcoin HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:51:20] "GET /bitcoin?callbackparam=jQuery111205364225784739427_1629183080458&_=1629183080459 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:53:19] "GET /bitcoin?callbackparam=jQuery111204065444227152566_1629183198956&_=1629183198957 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:53:44] "GET /bitcoin?callbackparam=jQuery1112034406517965612604_1629183224657&_=1629183224658 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:54:09] "GET /bitcoin?callbackparam=jQuery111209595082916033073_1629183248760&_=1629183248761 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:54:58] "GET /bitcoin?callbackparam=jQuery11120719787286528351_1629183298968&_=1629183298969 HTTP/1.1" 200 -
